# 자동차 환경 빅데이터 3차
- 과제2 : 건설기계 배출가스 관리 체계 구축
- 과제4 : 배출가스 전문정비제도 관리체계 구축
- 과제5 : 자동차 환경 빅데이터 제작사 공동 활용체게 구축

# Library

In [608]:
import pandas as pd
import os, math
import numpy as np
from datetime import datetime
import pyexasol

In [2]:
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

In [3]:
# 경고문 없애는 코드
# UserWarning: Workbook contains no default style, apply openpyxl's default warn("Workbook contains no default style, apply openpyxl's default")
import warnings
warnings.simplefilter("ignore")

# Fold

In [4]:
pub_fold = 'D:/data/processing/big3/raw/public'
raw_fold = 'D:/data/processing/big3/raw'
raw2_fold = 'D:/data/processing/big3/raw/BD2'
raw4_fold = 'D:/data/processing/big3/raw/BD4'
raw5_fold = 'D:/data/processing/big3/raw/BD5'
an_fold = 'D:/data/processing/big3/analysis'
an2_fold = 'D:/data/processing/big3/analysis/BD2'
an4_fold = 'D:/data/processing/big3/analysis/BD4'
an5_fold = 'D:/data/processing/big3/analysis/BD5'
df_fold = 'D:/data/processing/big3/df'
df2_fold = 'D:/data/processing/big3/df/BD2'
df4_fold = 'D:/data/processing/big3/df/BD4'
df5_fold = 'D:/data/processing/big3/df/BD5'
folds = {
    'pub_fold':pub_fold, 
    'raw_fold':raw_fold, 
    'raw2_fold':raw2_fold, 
    'raw4_fold':raw4_fold, 
    'raw5_fold':raw5_fold, 
    'an_fold':an_fold, 
    'an2_fold:':an2_fold, 
    'an4_fold:':an4_fold, 
    'an5_fold:':an5_fold, 
    'df_fold':df_fold, 
    'df2_fold':df2_fold, 
    'df4_fold':df4_fold, 
    'df5_fold':df5_fold, 
    }
folds

{'pub_fold': 'D:/data/processing/big3/raw/public',
 'raw_fold': 'D:/data/processing/big3/raw',
 'raw2_fold': 'D:/data/processing/big3/raw/BD2',
 'raw4_fold': 'D:/data/processing/big3/raw/BD4',
 'raw5_fold': 'D:/data/processing/big3/raw/BD5',
 'an_fold': 'D:/data/processing/big3/analysis',
 'an2_fold:': 'D:/data/processing/big3/analysis/BD2',
 'an4_fold:': 'D:/data/processing/big3/analysis/BD4',
 'an5_fold:': 'D:/data/processing/big3/analysis/BD5',
 'df_fold': 'D:/data/processing/big3/df',
 'df2_fold': 'D:/data/processing/big3/df/BD2',
 'df4_fold': 'D:/data/processing/big3/df/BD4',
 'df5_fold': 'D:/data/processing/big3/df/BD5'}

# server

In [5]:
# exasol db
wd = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
wz = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSZ')
we = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSE')

## 분석 결과 출력 옵션

In [ ]:
# y : 출력
# 그 외 : 미출력
export_opt = 'y'

# EDA
+ 과제2
    - 건설기계 인증자료
    - 건설기계 인증대장
+ 과제4
    - 자동차전문정비 : STD_TB_ERR_RES
    - 자동차전문정비상세 : STD_TB_ERR_RES_DET
+ 과제5
    - 구조변경검사 : STD_DLM_TB_ERP_GUBUN_BAECHUL

# 과제2

## 로드

### 등록정보(STD_CEG_CAR_MIG)

In [6]:
# 5m 31.4s
car = wd.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_FBCTN_YMD, VHCL_MNG_NO, VHRNO, EXTGAS_INSP_VLD_YMD, VHCL_OWNR_CL_CD FROM STD_CEG_CAR_MIG;")
car_ch_col = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'VHCL_ERSR_YN':'차량말소YN',
    'MANP_MNG_NO':'제원관리번호', 
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
    'VHCL_FBCTN_YMD':'제작일자', 
    'VHCL_MNG_NO':'차량관리번호', 
    'VHRNO':'자동차등록번호',
    'EXTGAS_INSP_VLD_YMD':'검사유효일',
    'VHCL_OWNR_CL_CD':'소유자구분',  
}
carr = car.rename(columns=car_ch_col)

print('data load : STD_CEG_CAR_MIG')

data load : STD_CEG_CAR_MIG


#### 데이터 확인

In [7]:
carr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31131025 entries, 0 to 31131024
Data columns (total 15 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   차대번호      object
 1   법정동코드     int64 
 2   배출가스등급    object
 3   배출가스인증번호  object
 4   차량말소YN    object
 5   제원관리번호    object
 6   차량연식      int64 
 7   차종        object
 8   용도        object
 9   최초등록일자    object
 10  제작일자      object
 11  차량관리번호    object
 12  자동차등록번호   object
 13  검사유효일     int64 
 14  소유자구분     object
dtypes: int64(3), object(12)
memory usage: 3.5+ GB


In [8]:
cols_df = pd.DataFrame({'컬럼명':car_ch_col.keys(), '컬럼설명':car_ch_col.values()})
cols_df

,컬럼명,컬럼설명
0,VIN,차대번호
1,BSPL_STDG_CD,법정동코드
2,EXHST_GAS_GRD_CD,배출가스등급
3,EXHST_GAS_CERT_NO,배출가스인증번호
4,VHCL_ERSR_YN,차량말소YN
5,MANP_MNG_NO,제원관리번호
6,YRIDNW,차량연식
7,VHCTY_CD,차종
8,PURPS_CD2,용도
9,FRST_REG_YMD,최초등록일자


In [9]:
carr_stat = carr.isnull().sum().reset_index().rename(columns={'index':'컬럼설명', 0:'빈 값 개수'})
carr_stat['총 행수'] = carr.shape[0]
carr_stat.insert(0, '번호', [x for x in range(1, carr_stat.shape[0] + 1)])
carr_stat_final = carr_stat.merge(cols_df, on='컬럼설명', how='left')
carr_stat_final = carr_stat_final[['번호', '컬럼명', '컬럼설명', '빈 값 개수', '총 행수']]
carr_stat_final

,번호,컬럼명,컬럼설명,빈 값 개수,총 행수
0,1,VIN,차대번호,0,31131025
1,2,BSPL_STDG_CD,법정동코드,0,31131025
2,3,EXHST_GAS_GRD_CD,배출가스등급,1298801,31131025
3,4,EXHST_GAS_CERT_NO,배출가스인증번호,4148289,31131025
4,5,VHCL_ERSR_YN,차량말소YN,0,31131025
5,6,MANP_MNG_NO,제원관리번호,0,31131025
6,7,YRIDNW,차량연식,0,31131025
7,8,VHCTY_CD,차종,0,31131025
8,9,PURPS_CD2,용도,0,31131025
9,10,FRST_REG_YMD,최초등록일자,18,31131025


##### [출력] 데이터 정보

In [10]:
today_date = datetime.today().strftime('%Y%m%d')
today_date

'2024.01.08'

In [11]:
carr_stat_final.to_excel(os.path.join(an2_fold, f'데이터정보_등록정보({today_date}).xlsx'), index=False)

### 제원정보(STD_CEG_CAR_SRC_MIG)

In [12]:
# 4.1s
src = wd.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, VHCTY_TY_CD2, MNFCTR_NM, VHCNM, VHCL_FRM, EGIN_TY, VHCTY_CL_CD, TOTL_WGHT, CRYNG_WGHT, DSPLVL, EGIN_OTPT FROM STD_CEG_CAR_SRC_MIG;")
src_ch_col = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
    'MNFCTR_NM':'제작사명', 
    'VHCNM':'차명', 
    'VHCL_FRM':'자동차형식', 
    'EGIN_TY':'엔진형식', 
    'VHCTY_CL_CD':'차종분류',
    'TOTL_WGHT':'총중량',
    'CRYNG_WGHT':'적재중량',
    'DSPLVL':'배기량', 
    'EGIN_OTPT':'엔진출력',
}
srcr = src.rename(columns=src_ch_col)

print('data load : STD_CEG_CAR_SRC_MIG')

data load : STD_CEG_CAR_SRC_MIG


#### 데이터 확인

In [13]:
srcr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 574467 entries, 0 to 574466
Data columns (total 12 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   제원관리번호  574467 non-null  object 
 1   연료      552717 non-null  object 
 2   차종유형    560780 non-null  object 
 3   제작사명    567648 non-null  object 
 4   차명      574395 non-null  object 
 5   자동차형식   543557 non-null  object 
 6   엔진형식    523045 non-null  object 
 7   차종분류    558783 non-null  object 
 8   총중량     570937 non-null  float64
 9   적재중량    467359 non-null  float64
 10  배기량     537510 non-null  float64
 11  엔진출력    497363 non-null  float64
dtypes: float64(4), object(8)
memory usage: 52.6+ MB


In [14]:
cols_df = pd.DataFrame({'컬럼명':src_ch_col.keys(), '컬럼설명':src_ch_col.values()})
cols_df

,컬럼명,컬럼설명
0,MANP_MNG_NO,제원관리번호
1,FUEL_CD,연료
2,VHCTY_TY_CD2,차종유형
3,MNFCTR_NM,제작사명
4,VHCNM,차명
5,VHCL_FRM,자동차형식
6,EGIN_TY,엔진형식
7,VHCTY_CL_CD,차종분류
8,TOTL_WGHT,총중량
9,CRYNG_WGHT,적재중량


In [15]:
srcr_stat = srcr.isnull().sum().reset_index().rename(columns={'index':'컬럼설명', 0:'빈 값 개수'})
srcr_stat['총 행수'] = srcr.shape[0]
srcr_stat.insert(0, '번호', [x for x in range(1, srcr_stat.shape[0] + 1)])
srcr_stat_final = srcr_stat.merge(cols_df, on='컬럼설명', how='left')
srcr_stat_final = srcr_stat_final[['번호', '컬럼명', '컬럼설명', '빈 값 개수', '총 행수']]
srcr_stat_final

,번호,컬럼명,컬럼설명,빈 값 개수,총 행수
0,1,MANP_MNG_NO,제원관리번호,0,574467
1,2,FUEL_CD,연료,21750,574467
2,3,VHCTY_TY_CD2,차종유형,13687,574467
3,4,MNFCTR_NM,제작사명,6819,574467
4,5,VHCNM,차명,72,574467
5,6,VHCL_FRM,자동차형식,30910,574467
6,7,EGIN_TY,엔진형식,51422,574467
7,8,VHCTY_CL_CD,차종분류,15684,574467
8,9,TOTL_WGHT,총중량,3530,574467
9,10,CRYNG_WGHT,적재중량,107108,574467


##### [출력] 데이터 정보

In [16]:
today_date = datetime.today().strftime('%Y%m%d')
today_date

'2024.01.08'

In [17]:
srcr_stat_final.to_excel(os.path.join(an2_fold, f'데이터정보_제원정보({today_date}).xlsx'), index=False)

### 정기검사(STD_TB_JGT_HIS)

In [18]:
# 3m 34.9s
# jgt = wb.export_to_pandas("SELECT VIN, FDRM_INSP_INSP_MTHD_CD, FDRM_INSP_KND_CD, FDRM_INSP_JGMT, FDRM_NLOD_SMO_MSTVL1, FDRM_NLOD_SMO_MSTVL2, FDRM_NLOD_SMO_MSTVL3, FDRM_NLOD_SMO_JT_YN1, FDRM_INSP_YMD, FDRM_DRVNG_DSTNC, FDRM_NLOD_SMO_PRMVL1 FROM STD_TB_JGT_HIS WHERE ROWNUM <= 10000;") # 테스트용
jgt = wd.export_to_pandas("SELECT VIN, FDRM_INSP_INSP_MTHD_CD, FDRM_INSP_KND_CD, FDRM_INSP_JGMT, FDRM_NLOD_SMO_MSTVL1, FDRM_NLOD_SMO_MSTVL2, FDRM_NLOD_SMO_MSTVL3, FDRM_NLOD_SMO_MSTVL4, FDRM_NLOD_SMO_MSTVL5, FDRM_NLOD_SMO_MSTVL6, FDRM_NLOD_SMO_JT_YN1, FDRM_NLOD_SMO_JT_YN2, FDRM_NLOD_SMO_JT_YN3, FDRM_NLOD_SMO_JT_YN4, FDRM_NLOD_SMO_JT_YN5, FDRM_NLOD_SMO_JT_YN6, FDRM_INSP_YMD, FDRM_DRVNG_DSTNC, FDRM_NLOD_SMO_PRMVL1, FDRM_NLOD_SMO_PRMVL2, FDRM_NLOD_SMO_PRMVL3, FDRM_NLOD_SMO_PRMVL4, FDRM_NLOD_SMO_PRMVL5, FDRM_NLOD_SMO_PRMVL6 FROM STD_TB_JGT_HIS;")
jgt_ch_col = {
    'VIN':'차대번호', 
    'FDRM_INSP_INSP_MTHD_CD':'검사방법', 
    'FDRM_INSP_KND_CD':'검사종류', 
    'FDRM_INSP_JGMT':'검사판정', 
    'FDRM_NLOD_SMO_MSTVL1':'무부하매연측정치1', 
    'FDRM_NLOD_SMO_MSTVL2':'무부하매연측정치2', 
    'FDRM_NLOD_SMO_MSTVL3':'무부하매연측정치3', 
    'FDRM_NLOD_SMO_MSTVL4':'무부하매연측정치4', 
    'FDRM_NLOD_SMO_MSTVL5':'무부하매연측정치5', 
    'FDRM_NLOD_SMO_MSTVL6':'무부하매연측정치6', 
    'FDRM_NLOD_SMO_JT_YN1':'무부하매연판정1', 
    'FDRM_NLOD_SMO_JT_YN2':'무부하매연판정2', 
    'FDRM_NLOD_SMO_JT_YN3':'무부하매연판정3', 
    'FDRM_NLOD_SMO_JT_YN4':'무부하매연판정4', 
    'FDRM_NLOD_SMO_JT_YN5':'무부하매연판정5', 
    'FDRM_NLOD_SMO_JT_YN6':'무부하매연판정6', 
    'FDRM_NLOD_SMO_PRMVL1':'무부하매연허용치1', 
    'FDRM_NLOD_SMO_PRMVL2':'무부하매연허용치2', 
    'FDRM_NLOD_SMO_PRMVL3':'무부하매연허용치3', 
    'FDRM_NLOD_SMO_PRMVL4':'무부하매연허용치4', 
    'FDRM_NLOD_SMO_PRMVL5':'무부하매연허용치5', 
    'FDRM_NLOD_SMO_PRMVL6':'무부하매연허용치6', 
    'FDRM_INSP_YMD':'검사일자',
    'FDRM_DRVNG_DSTNC':'주행거리',
}
jgtr = jgt.rename(columns=jgt_ch_col)

print('data load : STD_CEG_CAR_SRC_MIG')

data load : STD_CEG_CAR_SRC_MIG


#### 데이터 확인

In [19]:
jgtr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67357583 entries, 0 to 67357582
Data columns (total 24 columns):
 #   Column     Dtype  
---  ------     -----  
 0   차대번호       object 
 1   검사방법       object 
 2   검사종류       object 
 3   검사판정       object 
 4   무부하매연측정치1  float64
 5   무부하매연측정치2  float64
 6   무부하매연측정치3  float64
 7   무부하매연측정치4  float64
 8   무부하매연측정치5  float64
 9   무부하매연측정치6  float64
 10  무부하매연판정1   object 
 11  무부하매연판정2   object 
 12  무부하매연판정3   object 
 13  무부하매연판정4   object 
 14  무부하매연판정5   object 
 15  무부하매연판정6   object 
 16  검사일자       int64  
 17  주행거리       float64
 18  무부하매연허용치1  float64
 19  무부하매연허용치2  float64
 20  무부하매연허용치3  float64
 21  무부하매연허용치4  float64
 22  무부하매연허용치5  float64
 23  무부하매연허용치6  float64
dtypes: float64(13), int64(1), object(10)
memory usage: 12.0+ GB


In [20]:
cols_df = pd.DataFrame({'컬럼명':jgt_ch_col.keys(), '컬럼설명':jgt_ch_col.values()})
cols_df

,컬럼명,컬럼설명
0,VIN,차대번호
1,FDRM_INSP_INSP_MTHD_CD,검사방법
2,FDRM_INSP_KND_CD,검사종류
3,FDRM_INSP_JGMT,검사판정
4,FDRM_NLOD_SMO_MSTVL1,무부하매연측정치1
5,FDRM_NLOD_SMO_MSTVL2,무부하매연측정치2
6,FDRM_NLOD_SMO_MSTVL3,무부하매연측정치3
7,FDRM_NLOD_SMO_MSTVL4,무부하매연측정치4
8,FDRM_NLOD_SMO_MSTVL5,무부하매연측정치5
9,FDRM_NLOD_SMO_MSTVL6,무부하매연측정치6


In [21]:
jgtr_stat = jgtr.isnull().sum().reset_index().rename(columns={'index':'컬럼설명', 0:'빈 값 개수'})
jgtr_stat['총 행수'] = jgtr.shape[0]
jgtr_stat.insert(0, '번호', [x for x in range(1, jgtr_stat.shape[0] + 1)])
jgtr_stat_final = jgtr_stat.merge(cols_df, on='컬럼설명', how='left')
jgtr_stat_final = jgtr_stat_final[['번호', '컬럼명', '컬럼설명', '빈 값 개수', '총 행수']]
jgtr_stat_final

,번호,컬럼명,컬럼설명,빈 값 개수,총 행수
0,1,VIN,차대번호,0,67357583
1,2,FDRM_INSP_INSP_MTHD_CD,검사방법,0,67357583
2,3,FDRM_INSP_KND_CD,검사종류,0,67357583
3,4,FDRM_INSP_JGMT,검사판정,0,67357583
4,5,FDRM_NLOD_SMO_MSTVL1,무부하매연측정치1,3558954,67357583
5,6,FDRM_NLOD_SMO_MSTVL2,무부하매연측정치2,41907082,67357583
6,7,FDRM_NLOD_SMO_MSTVL3,무부하매연측정치3,41907082,67357583
7,8,FDRM_NLOD_SMO_MSTVL4,무부하매연측정치4,46950093,67357583
8,9,FDRM_NLOD_SMO_MSTVL5,무부하매연측정치5,46950093,67357583
9,10,FDRM_NLOD_SMO_MSTVL6,무부하매연측정치6,46950093,67357583


##### [출력] 데이터 정보

In [22]:
today_date = datetime.today().strftime('%Y%m%d')
today_date

'2024.01.08'

In [23]:
jgtr_stat_final.to_excel(os.path.join(an2_fold, f'데이터정보_정기정보({today_date}).xlsx'), index=False)

### 정밀검사(STD_TB_EET_HIS_ME)

In [24]:
# 10m 30.8s
eet = wd.export_to_pandas("SELECT VIN, PRCINSP_MSRMT_MTHD_CD, PRCINSP_KND_CD, PRCINSP_JGMT, PREC_NLOD_SMO_MSTVL1, PREC_NLOD_SMO_MSTVL2, PREC_NLOD_SMO_MSTVL3, PREC_NLOD_SMO_MSTVL4, PREC_NLOD_SMO_MSTVL5, PREC_NLOD_SMO_MSTVL6, PREC_NLOD_SMO_JT_YN1, PREC_NLOD_SMO_JT_YN2, PREC_NLOD_SMO_JT_YN3, PREC_NLOD_SMO_JT_YN4, PREC_NLOD_SMO_JT_YN5, PREC_NLOD_SMO_JT_YN6, PRCINSP_YMD, PRCINSP_DRVNG_DSTNC, PREC_NLOD_SMO_PRMVL1, PREC_NLOD_SMO_PRMVL2, PREC_NLOD_SMO_PRMVL3, PREC_NLOD_SMO_PRMVL4, PREC_NLOD_SMO_PRMVL5, PREC_NLOD_SMO_PRMVL6 FROM STD_TB_EET_HIS_ME;")
eet_ch_col = {
    'VIN':'차대번호', 
    'PRCINSP_MSRMT_MTHD_CD':'검사방법', 
    'PRCINSP_KND_CD':'검사종류', 
    'PRCINSP_JGMT':'검사판정', 
    'PREC_NLOD_SMO_MSTVL1':'무부하매연측정치1', 
    'PREC_NLOD_SMO_MSTVL2':'무부하매연측정치2', 
    'PREC_NLOD_SMO_MSTVL3':'무부하매연측정치3', 
    'PREC_NLOD_SMO_MSTVL4':'무부하매연측정치4', 
    'PREC_NLOD_SMO_MSTVL5':'무부하매연측정치5', 
    'PREC_NLOD_SMO_MSTVL6':'무부하매연측정치6', 
    'PREC_NLOD_SMO_JT_YN1':'무부하매연판정1', 
    'PREC_NLOD_SMO_JT_YN2':'무부하매연판정2', 
    'PREC_NLOD_SMO_JT_YN3':'무부하매연판정3', 
    'PREC_NLOD_SMO_JT_YN4':'무부하매연판정4', 
    'PREC_NLOD_SMO_JT_YN5':'무부하매연판정5', 
    'PREC_NLOD_SMO_JT_YN6':'무부하매연판정6', 
    'PREC_NLOD_SMO_PRMVL1':'무부하매연허용치1', 
    'PREC_NLOD_SMO_PRMVL2':'무부하매연허용치2', 
    'PREC_NLOD_SMO_PRMVL3':'무부하매연허용치3', 
    'PREC_NLOD_SMO_PRMVL4':'무부하매연허용치4', 
    'PREC_NLOD_SMO_PRMVL5':'무부하매연허용치5', 
    'PREC_NLOD_SMO_PRMVL6':'무부하매연허용치6', 
    'PRCINSP_YMD':'검사일자',
    'PRCINSP_DRVNG_DSTNC':'주행거리', 
}
eetr = eet.rename(columns=eet_ch_col)

print('data load : STD_TB_EET_HIS_ME')

data load : STD_TB_EET_HIS_ME


#### 데이터 확인

In [25]:
eetr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104870677 entries, 0 to 104870676
Data columns (total 24 columns):
 #   Column     Dtype  
---  ------     -----  
 0   차대번호       object 
 1   검사방법       object 
 2   검사종류       object 
 3   검사판정       object 
 4   무부하매연측정치1  float64
 5   무부하매연측정치2  float64
 6   무부하매연측정치3  float64
 7   무부하매연측정치4  float64
 8   무부하매연측정치5  float64
 9   무부하매연측정치6  float64
 10  무부하매연판정1   object 
 11  무부하매연판정2   object 
 12  무부하매연판정3   object 
 13  무부하매연판정4   object 
 14  무부하매연판정5   object 
 15  무부하매연판정6   object 
 16  검사일자       int64  
 17  주행거리       float64
 18  무부하매연허용치1  float64
 19  무부하매연허용치2  float64
 20  무부하매연허용치3  float64
 21  무부하매연허용치4  float64
 22  무부하매연허용치5  float64
 23  무부하매연허용치6  float64
dtypes: float64(13), int64(1), object(10)
memory usage: 18.8+ GB


In [26]:
cols_df = pd.DataFrame({'컬럼명':eet_ch_col.keys(), '컬럼설명':eet_ch_col.values()})
cols_df

,컬럼명,컬럼설명
0,VIN,차대번호
1,PRCINSP_MSRMT_MTHD_CD,검사방법
2,PRCINSP_KND_CD,검사종류
3,PRCINSP_JGMT,검사판정
4,PREC_NLOD_SMO_MSTVL1,무부하매연측정치1
5,PREC_NLOD_SMO_MSTVL2,무부하매연측정치2
6,PREC_NLOD_SMO_MSTVL3,무부하매연측정치3
7,PREC_NLOD_SMO_MSTVL4,무부하매연측정치4
8,PREC_NLOD_SMO_MSTVL5,무부하매연측정치5
9,PREC_NLOD_SMO_MSTVL6,무부하매연측정치6


In [27]:
# 
eetr_stat = eetr.isnull().sum().reset_index().rename(columns={'index':'컬럼설명', 0:'빈 값 개수'})
eetr_stat['총 행수'] = eetr.shape[0]
eetr_stat.insert(0, '번호', [x for x in range(1, eetr_stat.shape[0] + 1)])
eetr_stat_final = eetr_stat.merge(cols_df, on='컬럼설명', how='left')
eetr_stat_final = eetr_stat_final[['번호', '컬럼명', '컬럼설명', '빈 값 개수', '총 행수']]
eetr_stat_final

,번호,컬럼명,컬럼설명,빈 값 개수,총 행수
0,1,VIN,차대번호,0,104870677
1,2,PRCINSP_MSRMT_MTHD_CD,검사방법,7,104870677
2,3,PRCINSP_KND_CD,검사종류,19420,104870677
3,4,PRCINSP_JGMT,검사판정,123941,104870677
4,5,PREC_NLOD_SMO_MSTVL1,무부하매연측정치1,2470225,104870677
5,6,PREC_NLOD_SMO_MSTVL2,무부하매연측정치2,28995010,104870677
6,7,PREC_NLOD_SMO_MSTVL3,무부하매연측정치3,31702404,104870677
7,8,PREC_NLOD_SMO_MSTVL4,무부하매연측정치4,44690751,104870677
8,9,PREC_NLOD_SMO_MSTVL5,무부하매연측정치5,44690811,104870677
9,10,PREC_NLOD_SMO_MSTVL6,무부하매연측정치6,58527740,104870677


##### [출력] 데이터 정보

In [28]:
today_date = datetime.today().strftime('%Y%m%d')
today_date

'2024.01.08'

In [29]:
# # 0.8s
# eetr_stat_final.to_excel(os.path.join(an2_fold, f'데이터정보_정밀정보({today_date}).xlsx'), index=False)

### 건설기계 인증자료(12개)

In [30]:
def construct_machine_certification_data_load(name, sheet_name=0):
    fold = os.path.join(raw2_fold, 'KENCIS 건설기계 인증자료_231018')
    try:
        file_name = f'{name}.xlsx'
        cert_dt = pd.read_excel(os.path.join(fold, file_name), sheet_name=sheet_name)
        return cert_dt
    except:
        file_name = f'{name}.xlsx'
        cert_dt = pd.read_excel(os.path.join(fold, file_name))
        return cert_dt

#### 정보 모음

In [31]:
cols = construct_machine_certification_data_load('KENCIS_건설기계인증정보_테이블명세서', '테이블정의서')
cols.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   테이블명    12 non-null     object
 1   테이블설명   12 non-null     object
 2   컬럼명     649 non-null    object
 3   컬럼설명    644 non-null    object
 4   데이터형식   649 non-null    object
 5   길이      499 non-null    object
 6   Null    649 non-null    object
dtypes: object(7)
memory usage: 35.6+ KB


In [32]:
cols.head()

,테이블명,테이블설명,컬럼명,컬럼설명,데이터형식,길이,Null
0,UNI_FC_TCMASTER,통합 정식/기본인증 마스터,ADM_NO,관리번호,VARCHAR2,10,N
1,NaN,NaN,AGREE_SUBMIS,교정정보 및 사후 확정정보 제출 협약,VARCHAR2,100,Y
2,NaN,NaN,AGREE_SUBMIS_CT,교정정보 및 사후 확정정보 제출 협약 첨부파일,VARCHAR2,100,Y
3,NaN,NaN,APP_WAR,보증에 관한 설명,VARCHAR2,100,Y
4,NaN,NaN,APP_WAR_CT,보증에 관한 설명첨부파일,VARCHAR2,100,Y


In [33]:
cols[['테이블명', '테이블설명']] = cols[['테이블명', '테이블설명']].ffill()

In [34]:
name_list = [
    'UNI_FC_TCMASTER', 
    'UNI_FC_TCERTIMODI', 
    'UNI_FC_ENG_TEFSPEC', 
    'UNI_FC_TEEMISSN', 
    'UNI_FC_TEEMISSN_ENG_RES', 
    'UNI_FC_TESTINFO', 
    'UNI_FC_ENG_LIST', 
    'UNI_FC_ENDUENG_EMI', 
    'UNI_FC_ENDUENGTEST', 
    'UNI_FC_ENDUENGTEST_RES', 
    'UNI_FC_KIENGTEST', 
    'UNI_FC_KIENGTEST_RES', 
    ]
len(name_list)

12

In [35]:
cols_dict = {}
for t in name_list:
    cols_dict[t] = {a:b for a, b in cols.loc[cols['테이블명'] == t, ['컬럼명', '컬럼설명']].values}

In [36]:
cols_dict

{'UNI_FC_TCMASTER': {'ADM_NO': '관리번호',
  'AGREE_SUBMIS': '교정정보 및 사후 확정정보 제출 협약',
  'AGREE_SUBMIS_CT': '교정정보 및 사후 확정정보 제출 협약 첨부파일',
  'APP_WAR': '보증에 관한 설명',
  'APP_WAR_CT': '보증에 관한 설명첨부파일',
  'APPLI_ID': '신청아이디',
  'APPREGI_DATE': '결재일',
  'APPROVAL_YN': '결재여부',
  'CERTI_DIV': '인증구분',
  'CERTI_INTRO': '인증소개',
  'CERTI_INTRO_CT': '인증소개 첨부파일',
  'CERTI_STATE': '처분내용',
  'CERTI_TYPE': '인증종류',
  'CO_DEVT': 'CO 개발목표치',
  'CO_OP': '산술기호(1:덧셈 2:곱셈)',
  'COLD_CO_DEVT': 'Cold CO 개발목표치',
  'COLD_CO_OP': 'Cold CO 연산자',
  'CONF_INFO': '기밀사항',
  'CONF_INFO_CT': '기밀사항 첨부파일',
  'CONF_ISSUE_NO': '발급문서번호',
  'DF_CO': 'CO 열화계수',
  'DF_COLD_CO': 'HC 열화계수',
  'DF_EVAP_EMIS': 'COLD_CO 열화계수',
  'DF_HC': 'HC 열화계수',
  'DF_HCNOX': 'HCNOX 열화계수',
  'DF_MILEAGE': '열화계수 사용량',
  'DF_NOX': 'NOX 열화계수',
  'DF_PM': 'PM 열화계수',
  'DF_PN': 'PN 열화계수',
  'DF_SMOKE': 'SMOKE 열화계수',
  'DURA_TEST_EF_SNO': '내구성시험 동일차종 일련번호',
  'DURA_TEST_REMARK': '내구성테스트 비고',
  'DURA_TEST_YN': '내구성시험 실시 여부',
  'E_CONFIRM': '결제상태',
  'EF_CD': '동일

In [37]:
name = 'UNI_FC_TCMASTER'
tcmaster = construct_machine_certification_data_load(name)
tcmaster.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624 entries, 0 to 623
Columns: 159 entries, ADM_NO to NOISE_STD_NOTE
dtypes: datetime64[ns](1), float64(127), int64(2), object(29)
memory usage: 775.3+ KB


In [38]:
cols_dict[name]

{'ADM_NO': '관리번호',
 'AGREE_SUBMIS': '교정정보 및 사후 확정정보 제출 협약',
 'AGREE_SUBMIS_CT': '교정정보 및 사후 확정정보 제출 협약 첨부파일',
 'APP_WAR': '보증에 관한 설명',
 'APP_WAR_CT': '보증에 관한 설명첨부파일',
 'APPLI_ID': '신청아이디',
 'APPREGI_DATE': '결재일',
 'APPROVAL_YN': '결재여부',
 'CERTI_DIV': '인증구분',
 'CERTI_INTRO': '인증소개',
 'CERTI_INTRO_CT': '인증소개 첨부파일',
 'CERTI_STATE': '처분내용',
 'CERTI_TYPE': '인증종류',
 'CO_DEVT': 'CO 개발목표치',
 'CO_OP': '산술기호(1:덧셈 2:곱셈)',
 'COLD_CO_DEVT': 'Cold CO 개발목표치',
 'COLD_CO_OP': 'Cold CO 연산자',
 'CONF_INFO': '기밀사항',
 'CONF_INFO_CT': '기밀사항 첨부파일',
 'CONF_ISSUE_NO': '발급문서번호',
 'DF_CO': 'CO 열화계수',
 'DF_COLD_CO': 'HC 열화계수',
 'DF_EVAP_EMIS': 'COLD_CO 열화계수',
 'DF_HC': 'HC 열화계수',
 'DF_HCNOX': 'HCNOX 열화계수',
 'DF_MILEAGE': '열화계수 사용량',
 'DF_NOX': 'NOX 열화계수',
 'DF_PM': 'PM 열화계수',
 'DF_PN': 'PN 열화계수',
 'DF_SMOKE': 'SMOKE 열화계수',
 'DURA_TEST_EF_SNO': '내구성시험 동일차종 일련번호',
 'DURA_TEST_REMARK': '내구성테스트 비고',
 'DURA_TEST_YN': '내구성시험 실시 여부',
 'E_CONFIRM': '결제상태',
 'EF_CD': '동일차종기호',
 'EF_CD_1': '인증년도',
 'EF_CD_2': '차종기호',
 'EF_CD

In [39]:
cols_df = pd.DataFrame({'컬럼명':cols_dict[name].keys(), '컬럼설명':cols_dict[name].values()})
cols_df

,컬럼명,컬럼설명
0,ADM_NO,관리번호
1,AGREE_SUBMIS,교정정보 및 사후 확정정보 제출 협약
2,AGREE_SUBMIS_CT,교정정보 및 사후 확정정보 제출 협약 첨부파일
3,APP_WAR,보증에 관한 설명
4,APP_WAR_CT,보증에 관한 설명첨부파일
...,...,...
154,TONG_NUM,통관대수
155,TRECOGREP,비정기정비
156,TREGREP,정기정비
157,USER_ID,사용자아이디


In [40]:
temp = tcmaster.isnull().sum().reset_index().rename(columns={'index':'컬럼명', 0:'빈 값 개수'})
temp['총 행수'] = tcmaster.shape[0]
temp.insert(0, '번호', [x for x in range(1, temp.shape[0] + 1)])
ttemp = temp.merge(cols_df, on='컬럼명', how='left')
ttemp = ttemp[['번호', '컬럼명', '컬럼설명', '빈 값 개수', '총 행수']]
ttemp

,번호,컬럼명,컬럼설명,빈 값 개수,총 행수
0,1,ADM_NO,관리번호,0,624
1,2,FACT_SNO,사업장,464,624
2,3,CERTI_DIV,인증구분,0,624
3,4,REG_DATETIME,등록일시,0,624
4,5,RECT_DATE,접수일,175,624
...,...,...,...,...,...
154,155,MOD_DATETIME,수정일시,333,624
155,156,ENG_CAR_GB,건설기계/농업기계구분,131,624
156,157,EMIS_NOTE2,변경인증관련비고내용,363,624
157,158,EMIS_STD_NOTE,배출기준정보,619,624


In [41]:
today_date = datetime.today().strftime('%Y%m%d')
today_date

'2024.01.08'

#### [출력] 컬럼, 데이터 정보들

In [42]:
# # 41.2s
# with pd.ExcelWriter(os.path.join(an2_fold, f'건설기계_인증자료_데이터정보({today_date}).xlsx')) as writer:
#     for i, name in enumerate(name_list):
#         cols_df = pd.DataFrame({'컬럼명':cols_dict[name].keys(), '컬럼설명':cols_dict[name].values()})
#         ct = construct_machine_certification_data_load(name)
#         temp = ct.isnull().sum().reset_index().rename(columns={'index':'컬럼명', 0:'빈 값 개수'})
#         temp['총 행수'] = ct.shape[0]
#         temp.insert(0, '번호', [x for x in range(1, temp.shape[0] + 1)])
#         ttemp = temp.merge(cols_df, on='컬럼명', how='left')
#         ttemp = ttemp[['번호', '컬럼명', '컬럼설명', '빈 값 개수', '총 행수']]
#         ttemp.to_excel(writer, sheet_name=f'{i+1}. {name}', index=False)

In [43]:
name_list

['UNI_FC_TCMASTER',
 'UNI_FC_TCERTIMODI',
 'UNI_FC_ENG_TEFSPEC',
 'UNI_FC_TEEMISSN',
 'UNI_FC_TEEMISSN_ENG_RES',
 'UNI_FC_TESTINFO',
 'UNI_FC_ENG_LIST',
 'UNI_FC_ENDUENG_EMI',
 'UNI_FC_ENDUENGTEST',
 'UNI_FC_ENDUENGTEST_RES',
 'UNI_FC_KIENGTEST',
 'UNI_FC_KIENGTEST_RES']

#### 1\. UNI_FC_TCMASTER(통합 정식/기본인증 마스터)

In [44]:
name01 = name_list[0]
name01

'UNI_FC_TCMASTER'

In [45]:
# 1.3s
tcmaster = construct_machine_certification_data_load(name01)
tcmaster.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624 entries, 0 to 623
Columns: 159 entries, ADM_NO to NOISE_STD_NOTE
dtypes: datetime64[ns](1), float64(127), int64(2), object(29)
memory usage: 775.3+ KB


In [46]:
# CERTI_DIV(인증구분) : 1, 2
# EMIS_CERTI_NO(배출가스인증번호) : 00EN*AA*00 형식(숫자, 영문, 특수문자 혼합 10자리)
# EMIS_VEH_CLASS(배출가스 차종) : CE
# CERTI_STATE 처분내용 : nan, '90', 'C0', '10', '42', '46', '41'
# RECEIVE_STATE 반영 전 데이타 : nan,  0.
# NATION 제작국가 : nan, 'SW', 'US', 'KR', 'JP', 'GM', 'IT', 'FN', 'UK', 'CN', 'FR', 'SP', 'SU', 'ID'
# SELF_TEST 시설확인유무 : nan, 'Y', 'N'
# EMIS_CERTI_SNO 배출가스인증번호 : nan,  1.,  2.,  7.,  3.,  5., 12., 13.,  4.,  6., 10.,  8., 11., 9.
# OFFICE_CD 업체코드 : 'DDDC', 'VVIC', 'HIIC', 'KJDC', 'SSIC', 'DSDC', 'HDDC', 'WYIC', 'SJIC', 'ASIC', 'DPIC', 'LSIC', 'KBIC', 'CMIC', 'WTIC', 'MEIC', 'WJIC', 'HCDC', 'TMIC', 'LHIC', 'MXIC', 'HWDC', 'SCIC', 'EXAN', 'RADC', 'TYDC'
# OLDCERTI 과거자료여부 : nan, 'Y'
# REG2_YN 재신청여부 : nan, 'N'
# SNOTE 취소반납사유 : nan, '권리 및 의무 승계', '권리 및 의무 승계 신고', '권리 및 의무 신고에 따른 인증서 재교부'
# EF_CNT 동일차량 대수 : nan,  5.,  2.,  1.,  6.,  4.,  3., 22.,  9.,  0., 31.,  8., 14., 23.,  7., 15., 16., 11., 51., 13., 17., 10., 18.
# ELEC_CAR_GB 전기차 구분 ( N:일반,E:전기차(사용안함),H:하이브리드(사용안함), C:친환경차 ) : nan, 'N'
# CONF_ISSUE_NO 발급문서번호 : nan,  821.,  941.,  901., 1141., 1022., 1122.,  822.
# APPROVAL_YN 결재여부 : 'N', 'Y'
# ENG_CAR_GB 건설기계/농업기계구분 : nan, 'C', 'A', 'CA'

In [47]:
# ADM_NO 관리번호
col01 = 'ADM_NO'
print(cols_dict[name01][col01])
set(tcmaster[col01].unique())

관리번호


{52346,
 52347,
 52348,
 52349,
 52350,
 52351,
 52352,
 52353,
 52354,
 52355,
 52356,
 52359,
 52360,
 52361,
 52362,
 52366,
 52367,
 52370,
 52372,
 52376,
 52379,
 52381,
 52382,
 52385,
 52386,
 52388,
 52390,
 52391,
 52393,
 52395,
 52397,
 52398,
 52399,
 52401,
 52405,
 52406,
 52407,
 52408,
 52409,
 52410,
 52411,
 52412,
 52413,
 52414,
 52415,
 52416,
 52417,
 52418,
 52419,
 52420,
 52421,
 52422,
 52423,
 52424,
 52425,
 52426,
 52427,
 52428,
 52429,
 52431,
 52432,
 52433,
 52434,
 52435,
 52436,
 52437,
 52439,
 52452,
 52454,
 52456,
 52457,
 52458,
 52462,
 52466,
 52473,
 52474,
 52475,
 52476,
 52479,
 52480,
 52481,
 52483,
 52484,
 52485,
 52486,
 52487,
 52492,
 52493,
 52494,
 52495,
 52497,
 52505,
 52510,
 52511,
 52513,
 52514,
 52516,
 52517,
 52518,
 52520,
 52523,
 52534,
 52535,
 52536,
 52540,
 52549,
 52551,
 52552,
 52554,
 52560,
 52561,
 52562,
 52567,
 52573,
 52578,
 52583,
 52587,
 52589,
 52593,
 52597,
 52598,
 52599,
 52600,
 52601,
 52602,


In [48]:
tcmaster[col01].notnull().sum(), len(set(tcmaster[col01].unique()))

(624, 624)

#### 2\. UNI_FC_TCERTIMODI(통합 정식/변경인증 마스터)

In [49]:
name02 = name_list[1]
name02

'UNI_FC_TCERTIMODI'

In [50]:
# 5.0s
tcertimodi = construct_machine_certification_data_load(name02)
tcertimodi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830 entries, 0 to 2829
Data columns (total 91 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   CERTI_MODI_SNO        2830 non-null   int64         
 1   REG_DATETIME          2830 non-null   object        
 2   CERTI_MODI_DIV        2375 non-null   float64       
 3   EMIS_NOISE_DIV        2670 non-null   float64       
 4   EMIS_MODI_ITEM        0 non-null      float64       
 5   NOISE_MODI_ITEM       0 non-null      float64       
 6   EMIS_MODI_REASON      2346 non-null   object        
 7   NOISE_MODI_REASON     0 non-null      float64       
 8   MODI_ITEM_ETC         0 non-null      float64       
 9   RECT_DATE             2828 non-null   object        
 10  APPLI_ID              2670 non-null   object        
 11  OFFICER_ID            0 non-null      float64       
 12  MODI_EXPLAIN          0 non-null      float64       
 13  EMIS_CHG_EXAM     

In [51]:
# CERTI_MODI_SNO 변경인증일련번호 : 0 ~ 77
# CERTI_MODI_DIV 변경구분(1:변경인증, 2:변경보고) : 1.,  2., nan
# EMIS_NOISE_DIV 배출소음구분(1:배출 2:소음 3:배출+소음) : 1., nan
# MODIF_PROC_STATE 변경인증 진행상태 : nan, 0.0
# CERTI_STATE 인증진행상태 : nan, '10', '20', '30', '42', '46', '90', 'C', 'C0', 'C2'
# OLDCERTI 과거자료여부 : nan, 'N', 'Y'
# CERTI_SNO TCERTIMODI의IDX : nan, 1.0 ~ 48.0
# EMIS_DOC 배출가스인증서교부 공문 내용 : 1.0, nan
# CERTI_DIV 인증구분(1:국내,2:수입) : 1.0, 2.0, nan
# APPROVAL_YN 결재여부 : 'N', 'Y'
# MAKE_MTRL_NO 인증신청 작성자료번호 : 1.0, 3.0, 4.0, 5.0, 6.0, nan
# STANDARD_SLT 배출허용기준 : nan, 1.0, 2.0, 3.0, 4.0, 6.0, 7.0, 8.0, 9.0, 10.0
# STANDARD_SLT2 배출허용기준2 : nan, 1.0, 2.0, 3.0, 4.0, 6.0, 7.0, 8.0, 9.0, 10.0
# EMIS_ROWS_CODE 배출가스저감기술 : nan, '1', '1/2', '1/2/3', '1/2/3/4', '1/2/3/4/5', '1/2/4', '1/2/4/5', '1/3', '1/3/4', '1/3/4/5', '1/4', '1/4/5', '1/5', '2', '2/3/4', '2/4', '3/4', '5'
# EMIS_LOWS 배출가스저감기술코드 : nan, 'In-cylinder 방식', 'LNC', 'Turbocharger, Common rail', 'VGT, common rail', '연료장치, 터보챠저', '전자제어 연료분사 시스템', '전자제어 연료분사시스템', '전자제어장치,커먼레일', '커먼레일', '터보', '터보, 기계식', '터보차저', '터보차져'

In [52]:
# ADM_NO 관리번호
col02 = 'TC_NO'
print(cols_dict[name02][col02])
set(tcertimodi[col02].unique())

고유번호


{nan}

In [53]:
tcertimodi[col02].notnull().sum(), len(set(tcertimodi[col02].unique()))

(0, 1)

In [54]:
tcertimodi.loc[tcertimodi['ADM_NO'].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
1068,52346,3
1066,52346,1
1067,52346,2
1065,52346,0
1070,52346,5
...,...,...
2829,59491,1
2786,59492,2
2799,59492,0
2824,59492,1


In [55]:
# 2컬럼('ADM_NO', 'EF_SNO') 중복 존재 : X
tcertimodi.loc[tcertimodi[['ADM_NO', 'CERTI_MODI_SNO']].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO


#### 3\. UNI_FC_ENG_TEFSPEC(건설(농)기계 원동기 제원)

In [56]:
name03 = name_list[2]
name03

'UNI_FC_ENG_TEFSPEC'

In [250]:
# 16.1s
eng_tefspec = construct_machine_certification_data_load(name03)
eng_tefspec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8646 entries, 0 to 8645
Columns: 108 entries, ADM_NO to MAX_OUTT_POWER_UNIT
dtypes: float64(23), int64(3), object(82)
memory usage: 7.1+ MB


In [58]:
# CERTI_MODI_SNO 변경인증일련번호 : 0 ~ 77
# EF_SNO 동일원동기일련번호 : 0 ~ 256
# CYCLE 싸이클(행정) : nan, '4', '4cycle', '4사이클', '4행정', '5'
# COMB_TYPE 연소방식 : nan, 1.0, 2.0
# COOL_TYPE 냉각매체 : nan, 1.0
# CAT_TYPE 종류 : nan, 1.0, 2.0, 8.0, 9.0, 12.0, 15.0, 19.0, 20.0, 21.0, 22.0, 24.0, 99.0
# ENG_CAR_GB 건설기계/농업기계구분 : nan, 'A', 'C', 'CA'

In [59]:
# ADM_NO 관리번호
# EF_SNO 동일원동기일련번호
col03 = 'ADM_NO'
print(cols_dict[name03][col03])
set(eng_tefspec[col03].unique())

관리번호


{52346,
 52347,
 52348,
 52349,
 52350,
 52351,
 52352,
 52353,
 52354,
 52355,
 52356,
 52359,
 52360,
 52361,
 52362,
 52366,
 52367,
 52370,
 52372,
 52376,
 52379,
 52381,
 52382,
 52385,
 52386,
 52388,
 52390,
 52391,
 52393,
 52398,
 52399,
 52401,
 52405,
 52406,
 52407,
 52408,
 52409,
 52410,
 52411,
 52412,
 52413,
 52414,
 52418,
 52419,
 52420,
 52421,
 52422,
 52423,
 52424,
 52425,
 52426,
 52429,
 52431,
 52432,
 52433,
 52437,
 52439,
 52452,
 52454,
 52456,
 52457,
 52458,
 52466,
 52474,
 52476,
 52479,
 52486,
 52492,
 52495,
 52497,
 52505,
 52510,
 52511,
 52513,
 52517,
 52520,
 52534,
 52535,
 52540,
 52549,
 52552,
 52554,
 52567,
 52573,
 52578,
 52587,
 52589,
 52602,
 52606,
 52607,
 52612,
 52613,
 52614,
 52622,
 52640,
 52643,
 52644,
 52645,
 52646,
 52647,
 52648,
 52655,
 52656,
 52657,
 52664,
 52665,
 52666,
 52672,
 52689,
 52690,
 52702,
 52704,
 52707,
 52708,
 52718,
 52719,
 52721,
 52722,
 52723,
 52727,
 52739,
 52761,
 52763,
 52764,
 52765,


In [60]:
eng_tefspec[col03].notnull().sum(), len(set(eng_tefspec[col03].unique()))

(8646, 472)

In [61]:
eng_tefspec.loc[eng_tefspec['ADM_NO'].duplicated(keep=False), ['ADM_NO', 'EF_SNO']].sort_values('ADM_NO')

,ADM_NO,EF_SNO
5028,52346,16
5018,52346,1
5020,52346,3
5021,52346,0
5022,52346,8
...,...,...
7868,59500,0
8314,59571,0
8315,59571,1
7904,59573,1


In [62]:
# 2컬럼('ADM_NO', 'EF_SNO') 중복 존재 : O
eng_tefspec.loc[eng_tefspec[['ADM_NO', 'EF_SNO']].duplicated(keep=False), ['ADM_NO', 'EF_SNO']].sort_values('ADM_NO')

,ADM_NO,EF_SNO
5022,52346,8
5024,52346,8
4803,52589,8
4805,52589,8


In [63]:
# 2컬럼('ADM_NO', 'EF_SNO') 중복 존재 : O
eng_tefspec.loc[eng_tefspec[['ADM_NO', 'EF_SNO']].duplicated(keep=False), ['ADM_NO', 'EF_SNO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,EF_SNO,CERTI_MODI_SNO
5022,52346,8,1
5024,52346,8,2
4803,52589,8,3
4805,52589,8,4


In [64]:
# 3컬럼('ADM_NO', 'EF_SNO', 'CERTI_MODI_SNO') 중복 존재 : X
eng_tefspec.loc[eng_tefspec[['ADM_NO', 'EF_SNO', 'CERTI_MODI_SNO']].duplicated(keep=False), ['ADM_NO', 'EF_SNO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,EF_SNO,CERTI_MODI_SNO


#### 4\. UNI_FC_TEEMISSN(통합정식인증 배출일반정보)

In [65]:
name04 = name_list[3]
name04

'UNI_FC_TEEMISSN'

In [66]:
# 0.3s
teemissn = construct_machine_certification_data_load(name04)
teemissn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1058 entries, 0 to 1057
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ADM_NO           1058 non-null   int64  
 1   CERTI_MODI_SNO   1058 non-null   int64  
 2   FUEL             882 non-null    object 
 3   SEL_STAND        592 non-null    float64
 4   F_STAND          609 non-null    float64
 5   GU_TERM          577 non-null    float64
 6   TEST_DIV         0 non-null      float64
 7   TEST_METHOD_CHK  1053 non-null   object 
 8   COL              0 non-null      float64
 9   REG_ID           1058 non-null   object 
 10  REG_DATETIME     1058 non-null   object 
 11  MOD_ID           538 non-null    object 
 12  MOD_DATETIME     538 non-null    object 
 13  TBL_PK_SEQ_ID    1058 non-null   int64  
 14  LEV_CRTR_CD      0 non-null      float64
dtypes: float64(6), int64(3), object(6)
memory usage: 124.1+ KB


In [67]:
# CERTI_MODI_SNO 인증일련번호 : 0 ~ 70(빈 번호 존재)
# FUEL 사용연료코드 : nan, ‘D’
# SEL_STAND 적용기준 : nan, 3.0, 5.0, 6.0, 7.0, 8.0
# F_STAND 외국기준 : nan, 5.0, 6.0, 9.0, 10.0, 11.0

In [68]:
# ADM_NO 관리번호
col04 = 'ADM_NO'
print(cols_dict[name04][col04])
set(teemissn[col04].unique())

관리번호


{52346,
 52347,
 52348,
 52349,
 52350,
 52351,
 52352,
 52353,
 52354,
 52355,
 52356,
 52359,
 52360,
 52361,
 52362,
 52366,
 52367,
 52370,
 52372,
 52376,
 52379,
 52381,
 52382,
 52385,
 52386,
 52388,
 52390,
 52391,
 52393,
 52398,
 52399,
 52401,
 52405,
 52406,
 52407,
 52408,
 52409,
 52410,
 52411,
 52412,
 52413,
 52414,
 52418,
 52419,
 52420,
 52421,
 52422,
 52423,
 52424,
 52425,
 52426,
 52429,
 52431,
 52432,
 52433,
 52437,
 52439,
 52452,
 52454,
 52456,
 52457,
 52458,
 52466,
 52474,
 52476,
 52479,
 52486,
 52492,
 52495,
 52497,
 52505,
 52510,
 52511,
 52513,
 52517,
 52520,
 52534,
 52535,
 52540,
 52549,
 52552,
 52554,
 52567,
 52573,
 52578,
 52587,
 52589,
 52602,
 52603,
 52604,
 52605,
 52606,
 52607,
 52608,
 52612,
 52613,
 52614,
 52622,
 52640,
 52643,
 52644,
 52645,
 52646,
 52647,
 52648,
 52655,
 52656,
 52657,
 52664,
 52665,
 52666,
 52672,
 52689,
 52690,
 52702,
 52703,
 52704,
 52705,
 52706,
 52707,
 52708,
 52718,
 52719,
 52721,
 52722,


In [69]:
teemissn[col04].notnull().sum(), len(set(teemissn[col04].unique()))

(1058, 467)

In [70]:
teemissn.loc[teemissn['ADM_NO'].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
695,52510,0
1025,52510,12
708,52578,0
285,52578,3
521,52612,19
...,...,...
296,59491,2
879,59492,3
32,59492,0
1051,59492,2


In [71]:
# 2컬럼('ADM_NO', 'EF_SNO') 중복 존재 : X
teemissn.loc[teemissn[['ADM_NO', 'CERTI_MODI_SNO']].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO


#### 5\. UNI_FC_TEEMISSN_ENG_RES(건설(농)기계 원동기 배출시험정보)

In [72]:
name05 = name_list[4]
name05

'UNI_FC_TEEMISSN_ENG_RES'

In [73]:
# 6.4s
teemissn_eng_res = construct_machine_certification_data_load(name05)
teemissn_eng_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2761 entries, 0 to 2760
Columns: 101 entries, ADM_NO to NH3_ST_PERMSS
dtypes: float64(77), int64(5), object(19)
memory usage: 2.1+ MB


In [74]:
# CERTI_MODI_SNO 인증일련번호 : 0 ~ 35(빈 번호 존재)
# TEST_RST_CODE 시험결과구분코드(차량별시험결과=00,종합결과=99) : 0, 99
# TEST_MOD 시험모드(CVS-75,Highway,US06,SC03,Cold CO...) : 1, 2, 3, 99

In [75]:
# ADM_NO 관리번호
col05 = 'ADM_NO'
print(cols_dict[name05][col05])
set(teemissn_eng_res[col05].unique())

관리번호


{52346,
 52347,
 52348,
 52349,
 52350,
 52351,
 52352,
 52353,
 52354,
 52355,
 52356,
 52359,
 52360,
 52361,
 52362,
 52366,
 52367,
 52370,
 52372,
 52376,
 52379,
 52381,
 52382,
 52385,
 52386,
 52388,
 52390,
 52391,
 52393,
 52398,
 52399,
 52401,
 52405,
 52406,
 52407,
 52408,
 52409,
 52410,
 52411,
 52412,
 52413,
 52414,
 52418,
 52419,
 52420,
 52421,
 52422,
 52423,
 52424,
 52425,
 52426,
 52429,
 52432,
 52433,
 52437,
 52439,
 52452,
 52454,
 52456,
 52457,
 52458,
 52466,
 52474,
 52476,
 52479,
 52486,
 52492,
 52495,
 52497,
 52505,
 52510,
 52511,
 52513,
 52517,
 52520,
 52534,
 52535,
 52540,
 52549,
 52552,
 52554,
 52567,
 52573,
 52578,
 52587,
 52589,
 52602,
 52603,
 52604,
 52605,
 52606,
 52607,
 52608,
 52612,
 52613,
 52614,
 52622,
 52640,
 52643,
 52644,
 52645,
 52646,
 52647,
 52648,
 52655,
 52656,
 52657,
 52664,
 52665,
 52666,
 52672,
 52689,
 52690,
 52702,
 52703,
 52704,
 52705,
 52706,
 52707,
 52708,
 52718,
 52719,
 52721,
 52722,
 52723,


In [76]:
teemissn_eng_res[col05].notnull().sum(), len(set(teemissn_eng_res[col05].unique()))

(2761, 487)

#### 6\. UNI_FC_TESTINFO(통합 정식인증 부착장치정보)

In [77]:
name06 = name_list[5]
name06

'UNI_FC_TESTINFO'

In [78]:
# 1.3s
testinfo = construct_machine_certification_data_load(name06)
testinfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1044 entries, 0 to 1043
Data columns (total 67 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   ADM_NO                         1044 non-null   int64  
 1   EF_SNO                         0 non-null      float64
 2   CERTI_MODI_SNO                 1044 non-null   int64  
 3   INFO1                          177 non-null    object 
 4   INFO2                          1044 non-null   object 
 5   INFO3                          0 non-null      float64
 6   TC_TYPE                        1044 non-null   int64  
 7   INFO4                          0 non-null      float64
 8   ELEC_INFO1                     0 non-null      float64
 9   ELEC_TECH                      0 non-null      float64
 10  ELEC_INFO1_TXT                 0 non-null      float64
 11  ELEC_WARR_ACCUMU               0 non-null      float64
 12  ELEC_WARR_CHARGER              0 non-null      f

In [79]:
# CERTI_MODI_SNO 인증일련번호 : 0 ~ 70(빈 번호 존재)
# TC_TYPE 적용기준(기본/변경) : 1

In [80]:
# ADM_NO 관리번호
col06 = 'ADM_NO'
print(cols_dict[name06][col06])
set(testinfo[col06].unique())

관리번호


{52346,
 52347,
 52348,
 52349,
 52350,
 52351,
 52352,
 52353,
 52354,
 52355,
 52356,
 52359,
 52360,
 52361,
 52362,
 52366,
 52367,
 52370,
 52372,
 52376,
 52379,
 52381,
 52382,
 52385,
 52386,
 52388,
 52390,
 52391,
 52393,
 52398,
 52399,
 52401,
 52405,
 52406,
 52407,
 52408,
 52409,
 52410,
 52411,
 52412,
 52413,
 52414,
 52418,
 52419,
 52420,
 52421,
 52422,
 52423,
 52424,
 52425,
 52426,
 52429,
 52432,
 52433,
 52437,
 52439,
 52452,
 52454,
 52456,
 52457,
 52458,
 52466,
 52474,
 52476,
 52479,
 52486,
 52492,
 52495,
 52497,
 52505,
 52510,
 52511,
 52513,
 52517,
 52520,
 52534,
 52535,
 52540,
 52549,
 52552,
 52554,
 52567,
 52573,
 52578,
 52587,
 52589,
 52602,
 52606,
 52607,
 52612,
 52613,
 52614,
 52622,
 52640,
 52643,
 52644,
 52645,
 52646,
 52647,
 52648,
 52655,
 52656,
 52657,
 52664,
 52665,
 52666,
 52672,
 52689,
 52690,
 52702,
 52704,
 52707,
 52708,
 52718,
 52719,
 52721,
 52722,
 52723,
 52727,
 52739,
 52761,
 52763,
 52764,
 52765,
 52768,


In [81]:
testinfo[col06].notnull().sum(), len(set(testinfo[col06].unique()))

(1044, 453)

In [82]:
testinfo.loc[testinfo['ADM_NO'].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
1028,52510,12
330,52510,0
579,52578,3
343,52578,0
92,52612,19
...,...,...
1038,59491,1
777,59492,0
961,59492,3
1040,59492,2


In [83]:
# 2컬럼('ADM_NO', 'EF_SNO') 중복 존재 : X
testinfo.loc[testinfo[['ADM_NO', 'CERTI_MODI_SNO']].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO


#### 7\. UNI_FC_ENG_LIST(건설(농)기계 우너동기 장착정보)

In [84]:
name07 = name_list[6]
name07

'UNI_FC_ENG_LIST'

In [85]:
# 3.7s
eng_list = construct_machine_certification_data_load(name07)
eng_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9521 entries, 0 to 9520
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ENG_SNO          9521 non-null   int64  
 1   ADM_NO           9521 non-null   int64  
 2   CERTI_MODI_SNO   9521 non-null   int64  
 3   EF_SNO           9521 non-null   int64  
 4   ENG_TYPE         5 non-null      object 
 5   ENG_ID_NO        5 non-null      object 
 6   MAX_PWR_KW       5 non-null      float64
 7   MAX_PWR_RPM      5 non-null      float64
 8   ENG_DISPL        5 non-null      float64
 9   VEH_IN_COM       9281 non-null   object 
 10  VEH_PRE_NM       9279 non-null   object 
 11  VEH_NM           9287 non-null   object 
 12  VEH_MAKER        9291 non-null   object 
 13  VEH_TYPE         9290 non-null   object 
 14  ETC              58 non-null     object 
 15  REG_DATETIME     9521 non-null   object 
 16  REG_ID           2986 non-null   object 
 17  MOD_DATETIME  

In [86]:
# ENG_SNO 원동기장착일련번호 : 0 ~ 21
# CERTI_MODI_SNO 변경인증일련번호 : 0 ~ 77
# EF_SNO 동일차종일련번호 : 0 ~ 256
# ENG_ID_NO 대표원동기명(원동기코드) : nan, 'CFYI520/CFYI520U', 'F5GFL414J*B', 'F5HFL464A*F',
# VEH_TYPE_YN 인증서 표출 여부 : nan, 'Y'

In [87]:
# ADM_NO 관리번호
col07 = 'EF_SNO'
print(cols_dict[name07][col07])
set(eng_list[col07].unique())

동일차종일련번호


{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [88]:
eng_list[col07].notnull().sum(), len(set(eng_list[col07].unique()))

(9521, 257)

In [89]:
eng_list.loc[eng_list['ADM_NO'].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
345,52346,3
337,52346,0
338,52346,0
339,52346,0
340,52346,0
...,...,...
8891,59500,0
8969,59571,0
8970,59571,0
8437,59573,0


In [90]:
# 2컬럼('ADM_NO', 'EF_SNO') 중복 존재 : O
eng_list.loc[eng_list[['ADM_NO', 'CERTI_MODI_SNO']].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
337,52346,0
338,52346,0
339,52346,0
340,52346,0
341,52346,1
...,...,...
8397,59500,0
8969,59571,0
8970,59571,0
8436,59573,0


In [91]:
# 2컬럼('ADM_NO', 'EF_SNO') 중복 존재 : O
eng_list.loc[eng_list[['ADM_NO', 'CERTI_MODI_SNO']].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO', 'EF_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO,EF_SNO
337,52346,0,0
338,52346,0,1
339,52346,0,2
340,52346,0,3
341,52346,1,7
...,...,...,...
8397,59500,0,2
8969,59571,0,0
8970,59571,0,1
8436,59573,0,0


In [92]:
# 3컬럼('ADM_NO', 'EF_SNO', 'EF_SNO') 중복 존재 : O
eng_list.loc[eng_list[['ADM_NO', 'CERTI_MODI_SNO', 'EF_SNO']].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO', 'EF_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO,EF_SNO
8480,52510,11,23
8479,52510,11,23
8481,52510,11,23
8482,52510,11,23
8483,52510,11,23
...,...,...,...
8554,59048,1,15
9452,59491,3,4
8615,59491,3,4
8891,59500,0,2


#### 8\. UNI_FC_ENDUENG_EMI(건설(농)기계 내구 원동기 배출가스량 희귀분석결과)

In [93]:
name08 = name_list[7]
name08

'UNI_FC_ENDUENG_EMI'

In [94]:
# 0.2s
endueng_emi = construct_machine_certification_data_load(name08)
endueng_emi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411 entries, 0 to 410
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ADM_NO           411 non-null    int64  
 1   CERTI_MODI_SNO   411 non-null    int64  
 2   TEST_METHOD_CHK  411 non-null    object 
 3   TEST_MOD         411 non-null    int64  
 4   TEST_RST_SNO     411 non-null    int64  
 5   NOX_GRAD         167 non-null    float64
 6   NMHC_GRAD        153 non-null    float64
 7   CO_GRAD          163 non-null    float64
 8   PM_GRAD          166 non-null    float64
 9   NOX_YITCT        166 non-null    float64
 10  NMHC_YITCT       153 non-null    float64
 11  CO_YITCT         165 non-null    float64
 12  PM_YITCT         165 non-null    float64
 13  REG_ID           411 non-null    object 
 14  REG_DATETIME     375 non-null    object 
 15  MOD_ID           57 non-null     object 
 16  MOD_DATETIME     21 non-null     object 
 17  PN_GRAD         

In [95]:
# CERTI_MODI_SNO 변경인증일련번호 : 0 ~ 22(빈 번호 존재)
# TEST_METHOD_CHK 시험방법체크값 : 'NNNN', 'NNNY', 'NNYN', 'NNYY', 'NYNN', 'NYYN', 'YNNN', 'YNYN', 'YYYY'
# TEST_MOD 배출가스시험모드 : 1, 3, 4
# TEST_RST_SNO 시험결과일련번호 : 1

In [96]:
# ADM_NO 관리번호
col08 = 'ADM_NO'
print(cols_dict[name08][col08])
set(endueng_emi[col08].unique())

관리번호


{52664,
 52665,
 52727,
 52768,
 52786,
 52825,
 52852,
 52927,
 52936,
 52998,
 53000,
 53146,
 53254,
 53277,
 53299,
 53460,
 53464,
 55118,
 55119,
 55142,
 55143,
 55325,
 55329,
 55401,
 55432,
 55673,
 55685,
 55725,
 55926,
 56057,
 56083,
 56101,
 56120,
 56163,
 56191,
 56211,
 56237,
 56250,
 56362,
 56367,
 56475,
 56500,
 56539,
 56572,
 56577,
 56623,
 56800,
 56928,
 57043,
 57059,
 57218,
 57252,
 57254,
 57288,
 57294,
 57303,
 57330,
 57383,
 57415,
 57445,
 57447,
 57455,
 57530,
 57531,
 57533,
 57555,
 57599,
 57600,
 57668,
 57670,
 57743,
 57751,
 57761,
 57762,
 57764,
 57787,
 57789,
 57794,
 57826,
 57836,
 57843,
 57859,
 57876,
 57903,
 57923,
 57929,
 57930,
 57937,
 57956,
 57966,
 57985,
 57988,
 57994,
 58002,
 58006,
 58040,
 58051,
 58058,
 58091,
 58104,
 58110,
 58117,
 58119,
 58126,
 58129,
 58136,
 58137,
 58142,
 58144,
 58145,
 58148,
 58175,
 58212,
 58214,
 58223,
 58237,
 58378,
 58419,
 58420,
 58443,
 58505,
 58523,
 58524,
 58568,
 58576,


In [97]:
endueng_emi[col08].notnull().sum(), len(set(endueng_emi[col08].unique()))

(411, 156)

In [98]:
endueng_emi.loc[endueng_emi['ADM_NO'].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
222,52664,22
223,52664,22
275,52665,16
276,52665,16
16,52727,12
...,...,...
403,59491,0
407,59492,0
408,59492,0
332,59500,0


In [99]:
# 2컬럼('ADM_NO', 'EF_SNO') 중복 존재 : O
endueng_emi.loc[endueng_emi[['ADM_NO', 'CERTI_MODI_SNO']].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
222,52664,22
223,52664,22
275,52665,16
276,52665,16
16,52727,12
...,...,...
403,59491,0
407,59492,0
408,59492,0
332,59500,0


#### 9\. UNI_FC_ENDUENGTEST(건설(농)기계 내구 시험원동기 정보)

In [100]:
name09 = name_list[8]
name09

'UNI_FC_ENDUENGTEST'

In [101]:
# 0.1s
enduengtest = construct_machine_certification_data_load(name09)
enduengtest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430 entries, 0 to 429
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ADM_NO          430 non-null    int64 
 1   CERTI_MODI_SNO  430 non-null    int64 
 2   TEST_ENG_SNO    430 non-null    int64 
 3   ENG_TYPE        334 non-null    object
 4   ENG_ID_NO       315 non-null    object
 5   ENG_NO          319 non-null    object
 6   TEST_NO         277 non-null    object
 7   TEST_DATE       318 non-null    object
 8   REG_ID          407 non-null    object
 9   REG_DATETIME    430 non-null    object
 10  MOD_ID          56 non-null     object
 11  MOD_DATETIME    51 non-null     object
dtypes: int64(3), object(9)
memory usage: 40.4+ KB


In [102]:
# CERTI_MODI_SNO 변경인증일련번호 : 0 ~ 22(빈 번호 존재)
# TEST_ENG_SNO 시험원동기일련번호 : 0 ~ 19

In [103]:
# ADM_NO 관리번호
col09 = 'ADM_NO'
print(cols_dict[name09][col09])
set(enduengtest[col09].unique())

관리번호


{52664,
 52665,
 52727,
 52768,
 52786,
 52825,
 52852,
 52927,
 52936,
 52998,
 53000,
 53146,
 53254,
 53277,
 53299,
 53460,
 53464,
 55118,
 55119,
 55142,
 55143,
 55325,
 55329,
 55401,
 55432,
 55673,
 55685,
 55725,
 55926,
 56057,
 56083,
 56101,
 56120,
 56163,
 56191,
 56211,
 56237,
 56250,
 56362,
 56367,
 56475,
 56500,
 56539,
 56572,
 56577,
 56623,
 56800,
 56928,
 57043,
 57059,
 57218,
 57252,
 57254,
 57288,
 57294,
 57303,
 57330,
 57383,
 57415,
 57445,
 57447,
 57455,
 57530,
 57531,
 57533,
 57555,
 57599,
 57600,
 57668,
 57670,
 57743,
 57751,
 57761,
 57762,
 57764,
 57787,
 57789,
 57794,
 57826,
 57836,
 57843,
 57859,
 57876,
 57903,
 57923,
 57929,
 57930,
 57937,
 57956,
 57966,
 57985,
 57988,
 57994,
 58002,
 58006,
 58040,
 58051,
 58058,
 58091,
 58104,
 58110,
 58117,
 58119,
 58126,
 58129,
 58136,
 58137,
 58142,
 58144,
 58145,
 58148,
 58175,
 58212,
 58214,
 58223,
 58237,
 58378,
 58419,
 58420,
 58443,
 58505,
 58523,
 58524,
 58568,
 58576,


In [104]:
enduengtest[col09].notnull().sum(), len(set(enduengtest[col09].unique()))

(430, 155)

In [105]:
enduengtest.loc[enduengtest['ADM_NO'].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
183,52664,22
339,52664,20
7,52825,8
330,52825,7
79,53146,12
...,...,...
371,59018,0
415,59018,0
416,59018,0
417,59018,0


In [106]:
# 2컬럼('ADM_NO', 'EF_SNO') 중복 존재 : O
enduengtest.loc[enduengtest[['ADM_NO', 'CERTI_MODI_SNO']].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
249,55142,0
247,55142,0
246,55142,0
245,55142,0
244,55142,0
...,...,...
409,59018,0
415,59018,0
416,59018,0
371,59018,0


#### 10\. UNI_FC_ENDUENGTEST_RES(건설(농)기계 내구 원동기 시험결과정보)

In [107]:
name10 = name_list[9]
name10

'UNI_FC_ENDUENGTEST_RES'

In [108]:
# 4.2s
enduengtest_res = construct_machine_certification_data_load(name10)
enduengtest_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7581 entries, 0 to 7580
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ADM_NO            7581 non-null   int64  
 1   CERTI_MODI_SNO    7581 non-null   int64  
 2   P_ENG_YN          7581 non-null   int64  
 3   SEL_ADM_NO        4236 non-null   float64
 4   TEST_METHOD_CHK   7581 non-null   object 
 5   TEST_MOD          7581 non-null   int64  
 6   TEST_RST_SNO      7581 non-null   int64  
 7   TEST_RST_SNO_SEQ  7581 non-null   int64  
 8   FUEL              0 non-null      float64
 9   DDC_USE_YN        6724 non-null   float64
 10  TEST_CNT          0 non-null      float64
 11  TEST_TIME         7100 non-null   float64
 12  TEST_PLACE        7054 non-null   object 
 13  TEST_DATE         7025 non-null   object 
 14  CO                6034 non-null   float64
 15  NOX               5960 non-null   float64
 16  PM                5901 non-null   float64


In [109]:
# CERTI_MODI_SNO 인증 일련번호 : 0 ~ 22(빈 번호 존재)
# P_ENG_YN 내구대표차원동기여부 : 0, 1, 2
# TEST_METHOD_CHK 시험방법체크값 : 'NNNN', 'NNNY', 'NNYN', 'NNYY', 'NYNN', 'NYYN', 'YNNN', 'YNYN', 'YYYY'
# TEST_MOD 시험모드(CVS-75,Highway,US06,SC03,Cold CO...) : 1, 2
# TEST_RST_SNO 시험결과일련번호 : 1 ~ 268
# TEST_RST_SNO_SEQ 시험결과일련번호의항목순번 : 1 ~ 4
# DDC_USE_YN 지정열화계수 사용유무(사용:1,미사용:0) : nan, 1.0, 2.0

In [110]:
# ADM_NO 관리번호
col10 = 'ADM_NO'
print(cols_dict[name10][col10])
set(enduengtest_res[col10].unique())

관리번호


{52664,
 52665,
 52727,
 52768,
 52786,
 52825,
 52852,
 52927,
 52936,
 52998,
 53000,
 53146,
 53254,
 53277,
 53299,
 53460,
 53464,
 55118,
 55119,
 55142,
 55143,
 55325,
 55329,
 55401,
 55432,
 55673,
 55685,
 55725,
 55926,
 56057,
 56083,
 56101,
 56120,
 56163,
 56191,
 56211,
 56237,
 56250,
 56362,
 56367,
 56475,
 56500,
 56539,
 56572,
 56577,
 56623,
 56800,
 56928,
 57043,
 57059,
 57218,
 57252,
 57254,
 57288,
 57294,
 57303,
 57330,
 57383,
 57415,
 57445,
 57447,
 57455,
 57530,
 57531,
 57533,
 57555,
 57599,
 57600,
 57668,
 57670,
 57743,
 57751,
 57761,
 57762,
 57764,
 57787,
 57789,
 57794,
 57826,
 57836,
 57843,
 57859,
 57876,
 57903,
 57923,
 57929,
 57930,
 57937,
 57956,
 57966,
 57985,
 57988,
 57994,
 58002,
 58006,
 58040,
 58051,
 58058,
 58091,
 58104,
 58110,
 58117,
 58119,
 58126,
 58129,
 58136,
 58137,
 58142,
 58144,
 58145,
 58148,
 58175,
 58212,
 58214,
 58223,
 58237,
 58378,
 58419,
 58420,
 58443,
 58505,
 58523,
 58524,
 58568,
 58576,


In [111]:
enduengtest_res[col10].notnull().sum(), len(set(enduengtest_res[col10].unique()))

(7581, 155)

In [112]:
enduengtest_res.loc[enduengtest_res['ADM_NO'].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
1521,52664,22
1523,52664,22
1522,52664,22
1520,52664,22
1519,52664,22
...,...,...
6719,59500,0
6461,59573,0
6462,59573,0
6463,59573,0


In [113]:
# 2컬럼('ADM_NO', 'EF_SNO') 중복 존재 : O
enduengtest_res.loc[enduengtest_res[['ADM_NO', 'CERTI_MODI_SNO']].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
1521,52664,22
1523,52664,22
1522,52664,22
1520,52664,22
1519,52664,22
...,...,...
6719,59500,0
6461,59573,0
6462,59573,0
6463,59573,0


#### 11\. UNI_FC_KIENGTEST(건설(농)기계 주기적재생 시험원동기 정보)

In [114]:
name11 = name_list[10]
name11

'UNI_FC_KIENGTEST'

In [115]:
# 0.1s
kiengtest = construct_machine_certification_data_load(name11)
kiengtest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307 entries, 0 to 306
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ADM_NO          307 non-null    int64 
 1   CERTI_MODI_SNO  307 non-null    int64 
 2   TEST_ENG_SNO    307 non-null    int64 
 3   ENG_TYPE        209 non-null    object
 4   ENG_ID_NO       196 non-null    object
 5   ENG_NO          209 non-null    object
 6   TEST_NO         179 non-null    object
 7   TEST_DATE       203 non-null    object
 8   REG_ID          307 non-null    object
 9   REG_DATETIME    307 non-null    object
 10  MOD_ID          29 non-null     object
 11  MOD_DATETIME    29 non-null     object
dtypes: int64(3), object(9)
memory usage: 28.9+ KB


In [116]:
# CERTI_MODI_SNO 변경인증일련번호 : 0 ~ 35(빈 번호 존재)
# TEST_ENG_SNO 시험원동기일련번호 : 0 ~ 5

In [117]:
# ADM_NO 관리번호
col11 = 'ADM_NO'
print(cols_dict[name11][col11])
set(kiengtest[col11].unique())

관리번호


{52664,
 52665,
 52727,
 52768,
 52786,
 52825,
 52852,
 52925,
 52927,
 52998,
 53277,
 53299,
 53460,
 53464,
 53813,
 54543,
 54730,
 54913,
 55118,
 55119,
 55142,
 55143,
 55325,
 55432,
 55471,
 55673,
 55685,
 55725,
 55926,
 56057,
 56083,
 56101,
 56120,
 56163,
 56191,
 56211,
 56237,
 56250,
 56362,
 56367,
 56475,
 56539,
 56572,
 56577,
 56623,
 56800,
 56928,
 57059,
 57252,
 57254,
 57288,
 57294,
 57303,
 57330,
 57383,
 57415,
 57445,
 57447,
 57455,
 57491,
 57531,
 57535,
 57573,
 57579,
 57599,
 57600,
 57668,
 57670,
 57743,
 57751,
 57761,
 57762,
 57764,
 57787,
 57794,
 57808,
 57826,
 57837,
 57843,
 57869,
 57870,
 57876,
 57879,
 57929,
 57930,
 57937,
 57956,
 57966,
 57985,
 57988,
 57994,
 58002,
 58006,
 58040,
 58051,
 58058,
 58104,
 58110,
 58117,
 58119,
 58126,
 58129,
 58136,
 58137,
 58142,
 58144,
 58145,
 58148,
 58175,
 58212,
 58214,
 58223,
 58237,
 58309,
 58311,
 58356,
 58357,
 58365,
 58378,
 58420,
 58443,
 58523,
 58568,
 58576,
 58602,


In [118]:
kiengtest[col11].notnull().sum(), len(set(kiengtest[col11].unique()))

(307, 152)

In [119]:
kiengtest.loc[kiengtest['ADM_NO'].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
137,52664,22
237,52664,20
88,53299,9
238,53299,8
125,53464,6
...,...,...
231,59490,1
304,59490,0
83,59573,0
82,59573,0


In [120]:
# 2컬럼('ADM_NO', 'EF_SNO') 중복 존재 : O
kiengtest.loc[kiengtest[['ADM_NO', 'CERTI_MODI_SNO']].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
151,55725,0
150,55725,0
149,55725,0
143,55725,0
3,56101,0
...,...,...
298,59033,0
300,59033,0
83,59573,0
82,59573,0


#### 12\. UNI_FC_KIENGTEST_RES(건설(농)기계 주기적재생 원동기 시험결과정보)

In [121]:
name12 = name_list[11]
name12

'UNI_FC_KIENGTEST_RES'

In [122]:
# 1.3s
kiengtest_res = construct_machine_certification_data_load(name12)
kiengtest_res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2740 entries, 0 to 2739
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ADM_NO            2740 non-null   int64  
 1   CERTI_MODI_SNO    2740 non-null   int64  
 2   P_ENG_YN          2704 non-null   float64
 3   SEL_ADM_NO        30 non-null     float64
 4   TEST_METHOD_CHK   2740 non-null   object 
 5   TEST_MOD          2740 non-null   int64  
 6   TEST_RST_SNO      2740 non-null   int64  
 7   TEST_RST_SNO_SEQ  2740 non-null   int64  
 8   FUEL              0 non-null      float64
 9   CO                953 non-null    float64
 10  NMHC              924 non-null    float64
 11  NOX               943 non-null    float64
 12  PM                943 non-null    float64
 13  REG_ID            2740 non-null   object 
 14  REG_DATETIME      2740 non-null   object 
 15  MOD_ID            162 non-null    object 
 16  MOD_DATETIME      162 non-null    object 


In [123]:
# CERTI_MODI_SNO 인증 일련번호 : 0 ~ 35(빈 번호 존재)
# P_ENG_YN KI대표원동기 여부 : nan, 1.0, 2.0
# TEST_METHOD_CHK 시험방법체크값 : 'NNNN', 'NNYN', 'NYNN'
# TEST_MOD 시험모드(CVS-75,Highway,US06,SC03,Cold CO...) : 1, 2
# TEST_RST_SNO 시험결과일련번호 : 0 ~ 9
# TEST_RST_SNO_SEQ 시험결과일련번호의항목순번 : 0 ~ 2

In [124]:
# ADM_NO 관리번호
col12 = 'ADM_NO'
print(cols_dict[name12][col12])
set(kiengtest_res[col12].unique())

관리번호


{52664,
 52665,
 52727,
 52768,
 52786,
 52825,
 52852,
 52925,
 52927,
 52998,
 53277,
 53299,
 53460,
 53464,
 53813,
 54543,
 54730,
 54913,
 55118,
 55119,
 55142,
 55143,
 55325,
 55432,
 55471,
 55673,
 55685,
 55725,
 55926,
 56057,
 56083,
 56101,
 56120,
 56163,
 56191,
 56211,
 56237,
 56250,
 56362,
 56367,
 56475,
 56539,
 56572,
 56577,
 56623,
 56800,
 56928,
 57059,
 57252,
 57254,
 57288,
 57294,
 57303,
 57330,
 57383,
 57415,
 57445,
 57447,
 57455,
 57491,
 57531,
 57535,
 57573,
 57579,
 57599,
 57600,
 57668,
 57670,
 57743,
 57751,
 57761,
 57762,
 57764,
 57787,
 57794,
 57808,
 57826,
 57837,
 57843,
 57869,
 57870,
 57876,
 57879,
 57929,
 57930,
 57937,
 57956,
 57966,
 57985,
 57988,
 57994,
 58002,
 58006,
 58040,
 58051,
 58058,
 58104,
 58110,
 58117,
 58119,
 58126,
 58129,
 58136,
 58137,
 58142,
 58144,
 58145,
 58148,
 58175,
 58212,
 58214,
 58223,
 58237,
 58309,
 58311,
 58356,
 58357,
 58365,
 58378,
 58420,
 58443,
 58523,
 58568,
 58576,
 58602,


In [125]:
kiengtest_res[col12].notnull().sum(), len(set(kiengtest_res[col12].unique()))

(2740, 152)

In [126]:
kiengtest_res.loc[kiengtest_res['ADM_NO'].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
1267,52664,22
1262,52664,22
1263,52664,22
1264,52664,22
1265,52664,22
...,...,...
2526,59573,0
2527,59573,0
2518,59573,0
2525,59573,0


In [127]:
# 2컬럼('ADM_NO', 'EF_SNO') 중복 존재 : O
kiengtest_res.loc[kiengtest_res[['ADM_NO', 'CERTI_MODI_SNO']].duplicated(keep=False), ['ADM_NO', 'CERTI_MODI_SNO']].sort_values('ADM_NO')

,ADM_NO,CERTI_MODI_SNO
1267,52664,22
1262,52664,22
1263,52664,22
1264,52664,22
1265,52664,22
...,...,...
2526,59573,0
2527,59573,0
2518,59573,0
2525,59573,0


### 건설기계 인증대장

#### 로드(TIER-3~TIER-4)

In [1246]:
name = '건설기계 인증대장(Tier3~Stage5)'
file_name = f'{name}.xlsx'
cert_ld34 = pd.read_excel(os.path.join(raw2_fold, file_name), sheet_name='TIER-3~TIER-4')
cert_ld34.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5974 entries, 0 to 5973
Data columns (total 64 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   배출기준             5971 non-null   object
 1   상호명              5971 non-null   object
 2   인증번호             5971 non-null   object
 3   최초인증일자           5971 non-null   object
 4   재교부일자
(변경일자)     5971 non-null   object
 5   접수번호             5971 non-null   object
 6   접수일자             5971 non-null   object
 7   종류               5971 non-null   object
 8   구분               5971 non-null   object
 9   상세내역             4051 non-null   object
 10  엔진형식
(엔진모델명)     5971 non-null   object
 11  엔진코드             5970 non-null   object
 12  엔진배기량            5862 non-null   object
 13  최대출력             5856 non-null   object
 14  Unnamed: 14      5856 non-null   object
 15  엔진제작사            5971 non-null   object
 16  엔진제작국            5971 non-null   object
 17  건설기계 
수입사
(대표자)  5367 non-null   

##### 컬럼 수정

In [1247]:
cert_ld34.columns

Index(['배출기준', '상호명', '인증번호', '최초인증일자', '재교부일자\n(변경일자)', '접수번호', '접수일자', '종류',
       '구분', '상세내역', '엔진형식\n(엔진모델명)', '엔진코드', '엔진배기량', '최대출력', 'Unnamed: 14',
       '엔진제작사', '엔진제작국', '건설기계 \n수입사\n(대표자)', '건설기계\n제작사', '건설기계\n명칭',
       '건설/농업기계 형식', '비고', '정정내역', '기준치', 'Unnamed: 24', 'Unnamed: 25',
       'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', '시험결과', 'Unnamed: 30',
       'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34',
       'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38',
       'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42',
       'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46',
       'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50',
       'Unnamed: 51', 'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54',
       'Unnamed: 55', '자체시험여부', '적용기술', 'Unnamed: 58', 'Unnamed: 59',
       'Unnamed: 60', 'Unnamed: 61', 'Unnamed: 62', 'DF 변경보고'],
      dtype='object')

In [1248]:
cert_ld34.columns = [
    '배출기준', '상호명', '인증번호', '최초인증일자', '재교부일자(변경일자)', '접수번호', '접수일자', '종류', '구분', '상세내역', '엔진형식(엔진모델명)', '엔진코드', '엔진배기량', '최대출력_KW', '최대출력_RPM', '엔진제작사', '엔진제작국', '건설기계수입사(대표자)', '건설기계제작사', '건설기계명칭', '건설/농업기계_형식', '비고', '정정내역', 
    '기준치_CO', '기준치_NMHC+Nox', '기준치_NMHC', '기준치_Nox', '기준치_PM', '기준치_CO2', 
    '시험결과_ISO_8178_C1_CO', '시험결과_ISO_8178_C1_HC+Nox', '시험결과_ISO_8178_C1_NMHC', '시험결과_ISO_8178_C1_Nox', '시험결과_ISO_8178_C1_PM', '시험결과_ISO_8178_C1_CO2', '시험결과_ISO_8178_C1_검사일시', '시험결과_ISO_8178_C1_최대출력_kw', '시험결과_ISO_8178_C1_rpm', 
    '시험결과_NRSC_CO', '시험결과_NRSC_HC+Nox', '시험결과_NRSC_NMHC', '시험결과_NRSC_Nox', '시험결과_NRSC_PM', '시험결과_NRSC_CO2', '시험결과_NRSC_검사일시', '시험결과_NRSC_최대출력_kw', '시험결과_NRSC_최대출력_rpm', 
    '시험결과_NRTC_CO', '시험결과_NRTC_HC+Nox', '시험결과_NRTC_NMHC', '시험결과_NRTC_Nox', '시험결과_NRTC_PM', '시험결과_NRTC_CO2', '시험결과_NRTC_검사일시', '시험결과_NRTC_최대출력_kw', '시험결과_NRTC_최대출력_rpm',
    '자체시험여부', '적용기술_EGR', '적용기술_SCR', '적용기술_DPF', '적용기술_ECU', '적용기술_DOC', '적용기술_기타', 'DF_변경보고'
]
len(cert_ld34.columns)

64

##### 필요한 데이터만 추출(행 제한)

In [1249]:
cert_ld34 = cert_ld34.iloc[3:, :]
cert_ld34.head()

,배출기준,상호명,인증번호,최초인증일자,재교부일자(변경일자),접수번호,접수일자,종류,구분,상세내역,...,시험결과_NRTC_최대출력_kw,시험결과_NRTC_최대출력_rpm,자체시험여부,적용기술_EGR,적용기술_SCR,적용기술_DPF,적용기술_ECU,적용기술_DOC,적용기술_기타,DF_변경보고
3,tier-3,"㈜커민스 코리아, 윤석윤",09EN*CU*01,2009-01-14 00:00:00,-,6940,2008-12-23 00:00:00,건설기계,대표(최초),NaN,...,-,-,"TIER-2기준으로시험,3도만족\n교통입회",X,X,X,O,NaN,NaN,해당없음
4,tier-3,"㈜커민스 코리아, 윤석윤",09EN*CU*01,-,2012-02-08 00:00:00,15,2012-02-06 00:00:00,건설기계,대표(정정),대표엔진의\n건설기계 형식 정정,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
5,tier-3,"㈜커민스 코리아, 윤석윤",09EN*CU*01,-,2011-08-03 00:00:00,173,2011-07-26 00:00:00,건설기계,대표(보고),대표엔진의\n수입사 및 건설기계추가,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
6,tier-3,"㈜커민스 코리아, 윤석윤",09EN*CU*01,-,2011-09-22 00:00:00,204,2011-09-07 00:00:00,건설기계,대표(보고),대표엔진의\n수입사 및 건설기계추가,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음
7,tier-3,"㈜커민스 코리아, 윤석윤",09EN*CU*01,-,2012-01-25 00:00:00,3,2012-01-10 00:00:00,건설기계,대표(보고),대표엔진의\n수입사 및 건설기계추가,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,해당없음


In [1250]:
cert_ld34.shape

(5971, 64)

##### 데이터 확인

In [1251]:
# '대표(최초)', '대표(정정)', '대표(변경), '대표(보고)'
cert_ld34['구분'].unique()

array(['대표(최초)', '대표(정정)', '대표(보고)', '동일(보고)', '동일(최초2)', '동일(최초3)',
       '동일(최초4)', '동일(최초5)', '동일(최초6)', '동일(정정)', '동일(최초7)', '동일(최초8)',
       '동일(최초10)', '동일(최초11)', '동일(최초12)', '동일(최초13)', '동일(최초14)',
       '동일(최초15)', '동일(최초17)', '동일(최초18)', '동일(최초19)', '동일(최초20)',
       '동일(최초21)', '동일(변경)', '동일(최초1)', '동일(최초9)', '동일(최초16)', '동일(최초22)',
       '동일(최초23)', '대표(변경)', '동일(최초24)', '동일(최초25)', '동일(최초26)',
       '동일(최초27)', '동일(최초28)', '동일(최초30)', '동일(최초31)', '동일(최초29)',
       '동일(최초33)', '동일(최초34)', '동일(최초35)', '동일(최초36)', '동일(최초37)',
       '동일(최초38)', '동일(최초40)', '동일(최초41)', '동일(최초42)', '동일(최초43)',
       '동일(최초44)', '동일(최초45)', '동일(최초46)', '동일(최초47)', '동일(최초48)',
       '동일(최초49)', '동일(최초32)', '동일(최초39)', '동일(최초50)', '동일(최초51)',
       '동일(최초52)', '동일(최초53)', '동일(최초55)', '동일(최초56)', '동일(최초57)',
       '동일(최초58)', '동일(최초59)', '동일(최초60)', '동일(최초61)', '동일(최초)'],
      dtype=object)

In [1252]:
# len(cert_ld34.loc[(cert_ld34['구분'] == '대표(최초)') | (cert_ld34['구분'] == '대표(정정)') | (cert_ld34['구분'] == '대표(변경)') | (cert_ld34['구분'] == '대표(보고)'), '인증번호'].unique())
# len(cert_ld34.loc[(cert_ld34['구분'] == '대표(최초)') | (cert_ld34['구분'] == '대표(정정)') | (cert_ld34['구분'] == '대표(변경)'), '인증번호'].unique())
len(cert_ld34.loc[(cert_ld34['구분'] == '대표(최초)') | (cert_ld34['구분'] == '대표(정정)'), '인증번호'].unique())
# len(cert_ld34.loc[(cert_ld34['구분'] == '대표(최초)') | (cert_ld34['구분'] == '대표(변경)'), '인증번호'].unique())
# len(cert_ld34.loc[(cert_ld34['구분'] == '대표(변경)') | (cert_ld34['구분'] == '대표(보고)'), '인증번호'].unique())

288

In [1253]:
temp34 = cert_ld34[(cert_ld34['구분'] == '대표(최초)') | (cert_ld34['구분'] == '대표(정정)')].drop_duplicates('인증번호')
temp34.shape, temp34['인증번호'].notnull().sum()

((288, 64), 288)

##### 데이터 수정(TIER-3~TIER-4)

In [1254]:
# 줄 바꿈이 있는 열
# 상세내역, 최대출력_KW, 최대출력_RPM, 정정내역, 자체시험여부

# 줄 바꿈을 간격으로 바꿀 열
# 상세내역

# 줄 바꿈을 없앨 열
# 최대출력_KW, 최대출력_RPM, 정정내역

# 줄 바꿈을 쉼표로 바꿀 열
# 자체시험여부

In [1255]:
# 줄 바꿈을 간격으로 바꿀 열
# 상세내역
cert_ld34['상세내역'] = cert_ld34['상세내역'].str.replace('\n', ' ')
cert_ld34[['상세내역']].head()

,상세내역
3,NaN
4,대표엔진의 건설기계 형식 정정
5,대표엔진의 수입사 및 건설기계추가
6,대표엔진의 수입사 및 건설기계추가
7,대표엔진의 수입사 및 건설기계추가


In [1256]:
# 줄 바꿈을 없앨 열
# 최대출력_KW, 최대출력_RPM, 정정내역
cert_ld34['최대출력_KW'] = cert_ld34['최대출력_KW'].str.replace('\n', '')
cert_ld34['최대출력_RPM'] = cert_ld34['최대출력_RPM'].str.replace('\n', '')
cert_ld34['정정내역'] = cert_ld34['정정내역'].str.replace('\n', '')
cert_ld34[['최대출력_KW', '최대출력_RPM', '정정내역']].head()

,최대출력_KW,최대출력_RPM,정정내역
3,250(325ps),"2,100(1,900)","1. 동일엔진 추가, 출력병기 (2010.02.02, 재교부)"
4,250(325ps),"2,100(1,900)","1.건설기계 형식 정정(2012.02.,재교부)"
5,250(325ps),"2,100(1,900)",NaN
6,250(325ps),"2,100(1,900)",NaN
7,250(325ps),"2,100(1,900)",NaN


In [1257]:
# 줄 바꿈을 쉼표로 바꿀 열
# 자체시험여부
cert_ld34['자체시험여부'] = cert_ld34['자체시험여부'].str.replace('\n', ',')
cert_ld34[['자체시험여부']].head()

,자체시험여부
3,"TIER-2기준으로시험,3도만족,교통입회"
4,NaN
5,NaN
6,NaN
7,NaN


#### 로드(STAGE5)

In [1258]:
name = '건설기계 인증대장(Tier3~Stage5)'
file_name = f'{name}.xlsx'
cert_ld5 = pd.read_excel(os.path.join(raw2_fold, file_name), sheet_name='STAGE5')
cert_ld5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2058 entries, 0 to 2057
Data columns (total 74 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   배출기준             2054 non-null   object 
 1   상호명              2055 non-null   object 
 2   인증번호             2055 non-null   object 
 3   최초인증일자           2054 non-null   object 
 4   재교부일자
(변경일자)     2054 non-null   object 
 5   접수번호             2054 non-null   object 
 6   접수일자             2054 non-null   object 
 7   종류               2054 non-null   object 
 8   구분               2054 non-null   object 
 9   상세내역             783 non-null    object 
 10  엔진형식
(엔진모델명)     2054 non-null   object 
 11  엔진코드             2053 non-null   object 
 12  엔진배기량            2054 non-null   float64
 13  최대출력             2050 non-null   object 
 14  Unnamed: 14      2050 non-null   object 
 15  엔진
제작사           2054 non-null   object 
 16  엔진제작국            2054 non-null   object 
 17  건설기계 
수입사
(대표자

##### 컬럼 수정

In [1259]:
cert_ld5.head()

,배출기준,상호명,인증번호,최초인증일자,재교부일자\n(변경일자),접수번호,접수일자,종류,구분,상세내역,...,Unnamed: 64,자체시험여부,DF 대표,적용기술,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,임의설정자료제출여부
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,EGR,SCR,DPF,ECU,DOC,기타,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,rpm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,건설기계,대표(최초),NaN,...,2000,자체시험,대표,-,O,-,O,O,NaN,X
4,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1260]:
cert_ld5.columns

Index(['배출기준', '상호명', '인증번호', '최초인증일자', '재교부일자\n(변경일자)', '접수번호', '접수일자', '종류',
       '구분', '상세내역', '엔진형식\n(엔진모델명)', '엔진코드', '엔진배기량', '최대출력', 'Unnamed: 14',
       '엔진\n제작사', '엔진제작국', '건설기계 \n수입사\n(대표자)', '건설기계\n제작사', '건설기계\n명칭',
       '건설/농업기계 형식', '비고', '기준치', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25',
       'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', '시험결과', 'Unnamed: 30',
       'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34',
       'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38',
       'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42',
       'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46',
       'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50',
       'Unnamed: 51', 'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54',
       'Unnamed: 55', 'Unnamed: 56', 'Unnamed: 57', 'Unnamed: 58',
       'Unnamed: 59', 'Unnamed: 60', 'Unnamed: 61', 'Unnamed: 62',
       'Unnamed: 63', 'Unnamed: 64', '자체시험여부', 'DF 대표', '적용기술', 'Unnamed: 68',
 

In [1261]:
cert_ld5.columns = [
    '배출기준', '상호명', '인증번호', '최초인증일자', '재교부일자(변경일자)', '접수번호', '접수일자', '종류', '구분', '상세내역', '엔진형식(엔진모델명)', '엔진코드', '엔진배기량', 
    '최대출력_KW', '최대출력_RPM',
    '엔진제작사', '엔진제작국', '건설기계수입사(대표자)', '건설기계제작사', '건설기계명칭', '건설/농업기계_형식', '비고', 
    '기준치_CO', '기준치_THC+Nox', '기준치_THC', '기준치_Nox', '기준치_PM', '기준치_PN', '기준치_NH3', 
    '시험결과_ISO_8178_C1_CO', '시험결과_ISO_8178_C1_THC+Nox', '시험결과_ISO_8178_C1_THC', '시험결과_ISO_8178_C1_Nox', '시험결과_ISO_8178_C1_PM', '시험결과_ISO_8178_C1_PN', '시험결과_ISO_8178_C1_NH3', '시험결과_ISO_8178_C1_CO2', '시험결과_ISO_8178_C1_연비', '시험결과_ISO_8178_C1_검사일시',
    '시험결과_ISO_8178_C1_최대출력_kw', '시험결과_ISO_8178_C1_최대출력_rpm', 
    '시험결과_NRSC_CO', '시험결과_NRSC_THC+Nox', '시험결과_NRSC_THC', '시험결과_NRSC_Nox', '시험결과_NRSC_PM', '시험결과_NRSC_PN', '시험결과_NRSC_NH3', '시험결과_NRSC_CO2', '시험결과_NRSC_연비', '시험결과_NRSC_검사일시', '시험결과_NRSC_최대출력_kw', '시험결과_NRSC_최대출력_rpm', 
    '시험결과_NRTC_CO', '시험결과_NRTC_THC+Nox', '시험결과_NRTC_THC', '시험결과_NRTC_Nox', '시험결과_NRTC_PM', '시험결과_NRTC_PN', '시험결과_NRTC_NH3', '시험결과_NRTC_CO2', '시험결과_NRTC_연비', '시험결과_NRTC_검사일시', '시험결과_NRTC_최대출력_kw', '시험결과_NRTC_최대출력_rpm', 
    '자체시험여부', 'DF_대표', '적용기술_EGR', '적용기술_SCR', '적용기술_DPF', '적용기술_ECU', '적용기술_DOC', '적용기술_기타', 
    '임의설정자료제출여부'
]

In [1262]:
cert_ld5.head()

,배출기준,상호명,인증번호,최초인증일자,재교부일자(변경일자),접수번호,접수일자,종류,구분,상세내역,...,시험결과_NRTC_최대출력_rpm,자체시험여부,DF_대표,적용기술_EGR,적용기술_SCR,적용기술_DPF,적용기술_ECU,적용기술_DOC,적용기술_기타,임의설정자료제출여부
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,EGR,SCR,DPF,ECU,DOC,기타,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,rpm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,건설기계,대표(최초),NaN,...,2000,자체시험,대표,-,O,-,O,O,NaN,X
4,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### 필요한 데이터만 추출(행 제한)

In [1263]:
cert_ld5 = cert_ld5.iloc[3:, :]
cert_ld5.head()

,배출기준,상호명,인증번호,최초인증일자,재교부일자(변경일자),접수번호,접수일자,종류,구분,상세내역,...,시험결과_NRTC_최대출력_rpm,자체시험여부,DF_대표,적용기술_EGR,적용기술_SCR,적용기술_DPF,적용기술_ECU,적용기술_DOC,적용기술_기타,임의설정자료제출여부
3,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,건설기계,대표(최초),NaN,...,2000,자체시험,대표,-,O,-,O,O,NaN,X
4,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1264]:
cert_ld5.shape

(2055, 74)

In [1265]:
cert_ld5.iloc[:5, :20]

,배출기준,상호명,인증번호,최초인증일자,재교부일자(변경일자),접수번호,접수일자,종류,구분,상세내역,엔진형식(엔진모델명),엔진코드,엔진배기량,최대출력_KW,최대출력_RPM,엔진제작사,엔진제작국,건설기계수입사(대표자),건설기계제작사,건설기계명칭
3,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,건설기계,대표(최초),NaN,F4HGE613I,F4HGE613I*V,6728.00,191,2000,FPT Industrial S.p.A,이태리,해당없음,해당없음,해당없음
4,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,F4HGE613I,F4DGE617D*V,6728.00,221,2100,FPT Industrial S.p.A,이태리,LS엠트론\n(구자은),NH,트랙터
5,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,F4HGE613I,F4DGE617D*V,6728.00,221,2100,FPT Industrial S.p.A,이태리,삼정건설기계\n(유재흥),CASE,트랙터
6,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,F4HGE613I,F4DGE617E*V,6728.00,199,2100,FPT Industrial S.p.A,이태리,LS엠트론\n(구자은),NH,트랙터
7,Stage-5,㈜원진에이티,20EN*IV*01,2020-07-15 00:00:00,-,2020_1230,2020-06-16 00:00:00,농업기계,동일(최초),NaN,F4HGE613I,F4DGE617E*V,6728.00,199,2100,FPT Industrial S.p.A,이태리,삼정건설기계\n(유재흥),CASE,트랙터


In [1266]:
cert_ld5.iloc[:5, 20:35]

,건설/농업기계_형식,비고,기준치_CO,기준치_THC+Nox,기준치_THC,기준치_Nox,기준치_PM,기준치_PN,기준치_NH3,시험결과_ISO_8178_C1_CO,시험결과_ISO_8178_C1_THC+Nox,시험결과_ISO_8178_C1_THC,시험결과_ISO_8178_C1_Nox,시험결과_ISO_8178_C1_PM,시험결과_ISO_8178_C1_PN
3,해당없음,NaN,3.50,-,0.19,0.40,0.01,1000000000000,10,-,-,-,-,-,-
4,T7.315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Optum 300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,T7.290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Optum 270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1267]:
cert_ld5.iloc[:5, 35:50]

,시험결과_ISO_8178_C1_NH3,시험결과_ISO_8178_C1_CO2,시험결과_ISO_8178_C1_연비,시험결과_ISO_8178_C1_검사일시,시험결과_ISO_8178_C1_최대출력_kw,시험결과_ISO_8178_C1_최대출력_rpm,시험결과_NRSC_CO,시험결과_NRSC_THC+Nox,시험결과_NRSC_THC,시험결과_NRSC_Nox,시험결과_NRSC_PM,시험결과_NRSC_PN,시험결과_NRSC_NH3,시험결과_NRSC_CO2,시험결과_NRSC_연비
3,-,-,-,-,-,-,0.01,-,0.01,0.29,0.00,34600000000,1.52,686.36,212.78
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1268]:
cert_ld5.iloc[:5, 50:60]

,시험결과_NRSC_검사일시,시험결과_NRSC_최대출력_kw,시험결과_NRSC_최대출력_rpm,시험결과_NRTC_CO,시험결과_NRTC_THC+Nox,시험결과_NRTC_THC,시험결과_NRTC_Nox,시험결과_NRTC_PM,시험결과_NRTC_PN,시험결과_NRTC_NH3
3,2019-11-14 00:00:00,191,2000,0.02,-,0.01,0.21,0.00,25100000000,0.76
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1269]:
cert_ld5.iloc[:5, 60:]

,시험결과_NRTC_CO2,시험결과_NRTC_연비,시험결과_NRTC_검사일시,시험결과_NRTC_최대출력_kw,시험결과_NRTC_최대출력_rpm,자체시험여부,DF_대표,적용기술_EGR,적용기술_SCR,적용기술_DPF,적용기술_ECU,적용기술_DOC,적용기술_기타,임의설정자료제출여부
3,706.07,219.15,2019-11-13 00:00:00,191,2000,자체시험,대표,-,O,-,O,O,NaN,X
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### 데이터 확인

In [1270]:
# len(cert_ld5.loc[(cert_ld5['구분'] == '대표(최초)') | (cert_ld5['구분'] == '대표(정정)') | (cert_ld5['구분'] == '대표(보고)'), '인증번호'].unique())
len(cert_ld5.loc[(cert_ld5['구분'] == '대표(최초)') | (cert_ld5['구분'] == '대표(정정)'), '인증번호'].unique())
# len(cert_ld5.loc[(cert_ld5['구분'] == '대표(정정)') | (cert_ld5['구분'] == '대표(보고)'), '인증번호'].unique())

129

In [1271]:
temp5 = cert_ld5[(cert_ld5['구분'] == '대표(최초)') | (cert_ld5['구분'] == '대표(정정)')].drop_duplicates('인증번호')
temp5.shape, temp5['인증번호'].notnull().sum()

((129, 74), 129)

In [1272]:
set(cert_ld5['접수번호'].unique())

{'2018_1235\n2018_1235~1',
 '2018_1236\n2018_1236~1',
 '2019_1347',
 '2019_1348',
 '2019_223\n2019_223~1',
 '2019_224\n2019_224~1',
 '2019_771',
 '2019_772',
 '2020_1230',
 '2020_1417',
 '2020_1721',
 '2020_1956',
 '2020_2105',
 '2020_2200',
 '2020_2210',
 '2020_2211',
 '2020_2221\n2020_2221~1',
 '2020_2372',
 '2020_2375',
 '2020_2417\n2020_2417~1',
 '2020_2657',
 '2021_1033\n2021_1033~1',
 '2021_1034',
 '2021_1143\n2021_1143~1',
 '2021_1144',
 '2021_117',
 '2021_1194\n2021_1194~1',
 '2021_1257',
 '2021_1266',
 '2021_1277',
 '2021_1336\n2021_1336~1',
 '2021_1510',
 '2021_1511\n2021_1511~1',
 '2021_1512\n2021_1512~1',
 '2021_1609',
 '2021_1735',
 '2021_1787',
 '2021_1788',
 '2021_1795\n2021_1795~1',
 '2021_1796',
 '2021_1797\n2021_1797~1',
 '2021_1822\n2021_1822~1',
 '2021_1858',
 '2021_1866',
 '2021_1867\n2021_1867~1',
 '2021_1903\n2021_1901~1',
 '2021_1918\n2021_1918~1',
 '2021_1921\n2021_1921~1',
 '2021_1950',
 '2021_1970\n2021_1970~1',
 '2021_1984',
 '2021_2004',
 '2021_2005',
 '202

##### 데이터 수정(STAGE5)

In [1273]:
# 줄 바꿈이 있는 열
# 건설기계수입사(대표자)

# 줄 바꿈을 없앨 열
# 건설기계수입사(대표자)

In [1274]:
# 줄 바꿈을 없앨 열
# 건설기계수입사(대표자)
cert_ld5['건설기계수입사(대표자)'] = cert_ld5['건설기계수입사(대표자)'].str.replace('\n', '')
cert_ld5[['건설기계수입사(대표자)']].head()

,건설기계수입사(대표자)
3,해당없음
4,LS엠트론(구자은)
5,삼정건설기계(유재흥)
6,LS엠트론(구자은)
7,삼정건설기계(유재흥)


##### 티어 3~5 인증번호 중복 확인
- 3, 4 : 368
- 5 : 131
- 총 499
- 병합한 뒤 중복 제거 : 490

In [1275]:
temp = pd.concat([cert_ld34, cert_ld5], ignore_index=True)
temp.shape

(8026, 86)

In [1276]:
len(cert_ld34['인증번호'].unique()), len(cert_ld5['인증번호'].unique()), len(cert_ld34['인증번호'].unique()) + len(cert_ld5['인증번호'].unique()), len(temp['인증번호'].unique())

(368, 131, 499, 490)

In [1277]:
set(cert_ld34['인증번호'].unique()) & set(cert_ld5['인증번호'].unique())

{'19EN*CU*01',
 '19EN*CU*02',
 '19EN*CU*03',
 '19EN*CU*04',
 '19EN*CU*05',
 '19EN*CU*06',
 '19EN*CU*07',
 '19EN*CU*08',
 '22EN*HD*01'}

##### [출력] 컬럼, 데이터 정보들

In [1278]:
info01 = cert_ld34.isnull().sum().reset_index().rename(columns={'index':'컬럼명', 0:'빈 값 개수'})
info01['총 행수'] = cert_ld34.shape[0]
info01['번호'] = [x for x in range(1, info01.shape[0] + 1)]
info01 = info01[['번호', '컬럼명', '빈 값 개수', '총 행수']]
info01

,번호,컬럼명,빈 값 개수,총 행수
0,1,배출기준,0,5971
1,2,상호명,0,5971
2,3,인증번호,0,5971
3,4,최초인증일자,0,5971
4,5,재교부일자(변경일자),0,5971
...,...,...,...,...
59,60,적용기술_DPF,5683,5971
60,61,적용기술_ECU,5692,5971
61,62,적용기술_DOC,5771,5971
62,63,적용기술_기타,5882,5971


In [1279]:
info02 = cert_ld5.isnull().sum().reset_index().rename(columns={'index':'컬럼명', 0:'빈 값 개수'})
info02['총 행수'] = cert_ld5.shape[0]
info02['번호'] = [x for x in range(1, info02.shape[0] + 1)]
info02 = info02[['번호', '컬럼명', '빈 값 개수', '총 행수']]
info02

,번호,컬럼명,빈 값 개수,총 행수
0,1,배출기준,1,2055
1,2,상호명,0,2055
2,3,인증번호,0,2055
3,4,최초인증일자,1,2055
4,5,재교부일자(변경일자),1,2055
...,...,...,...,...
69,70,적용기술_DPF,1925,2055
70,71,적용기술_ECU,1925,2055
71,72,적용기술_DOC,1925,2055
72,73,적용기술_기타,1951,2055


In [1280]:
today_date = datetime.today().strftime('%Y%m%d')
today_date

'20240109'

In [1281]:
# with pd.ExcelWriter(os.path.join(an2_fold, f'건설기계_인증대장_데이터정보({today_date}).xlsx')) as writer:
#     info01.to_excel(writer, sheet_name='TIER-3~TIER-4', index=False)
#     info02.to_excel(writer, sheet_name='STAGE5', index=False)

### 건설기계 등록제원 정보

#### 등록_굴착기

In [18]:
# sheet_name : Data, Data_1, Data_2
name = '등록정보_굴착기'
file_name = f'{name}.xlsx'
ri_exc_all = pd.read_excel(os.path.join(raw2_fold, file_name), sheet_name=None)

In [19]:
ri_exc_all.keys()

dict_keys(['Data', 'Data_1', 'Data_2'])

In [36]:
ri_exc01 = ri_exc_all['Data']
ri_exc01.head()

,건설기계등록번호,건설기계차대번호,건설기계관리번호,건설기계코드,건설기계명,사용본거지법정동코드,사용본거지주소,대표소유자명,대표소유자주소,대표소유자번호,대표소유자전화번호,최초등록일,정비명령시작일자,정비명령종료일자,형식승인번호
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,건설기계등록번호,건설기계차대번호,건설기계관리번호,건설기계코드,건설기계명,사용본거지법정동코드,사용본거지주소,대표소유자명,대표소유자주소,대표소유자번호,대표소유자전화번호,최초등록일,정비명령시작일자,정비명령종료일자,형식승인번호
2,경북02도1245,SL55-VS-33951,경기02노7009_00,02,굴착기,4715011800,"경상북도 김천시 아홉사리길 130, 103동 1603호(응명동, 김천대신코아루아파트)",김수호,"경상북도 김천시 아홉사리길 130, 103동 1603호(응명동, 김천대신코아루아파트)",770415-1800315,NaN,20050217,NaN,NaN,40200200200
3,강원02모5196,VCEEC55CC00123601,경기02파6876_01,02,굴착기,5115012700,강원특별자치도 강릉시 성덕로176번길 22-3(두산동),최천,강원특별자치도 정선군 임계면 태봉안길 302,740716-1345719,NaN,20130429,NaN,NaN,40200720100
4,전북02사3394,VCEEC55CL00111007,경기02로5946_00,02,굴착기,4571040024,전라북도 완주군 동상면 은천길 31,배선동,전라북도 완주군 동상면 은천길 31,581103-1519931,--,20090521,NaN,NaN,40200720000


In [37]:
ri_exc01.columns = ri_exc01.iloc[1, :].values
ri_exc01 = ri_exc01.iloc[2:, :].reset_index(drop=True)
ri_exc01.head()

,건설기계등록번호,건설기계차대번호,건설기계관리번호,건설기계코드,건설기계명,사용본거지법정동코드,사용본거지주소,대표소유자명,대표소유자주소,대표소유자번호,대표소유자전화번호,최초등록일,정비명령시작일자,정비명령종료일자,형식승인번호
0,경북02도1245,SL55-VS-33951,경기02노7009_00,02,굴착기,4715011800,"경상북도 김천시 아홉사리길 130, 103동 1603호(응명동, 김천대신코아루아파트)",김수호,"경상북도 김천시 아홉사리길 130, 103동 1603호(응명동, 김천대신코아루아파트)",770415-1800315,NaN,20050217,NaN,NaN,40200200200
1,강원02모5196,VCEEC55CC00123601,경기02파6876_01,02,굴착기,5115012700,강원특별자치도 강릉시 성덕로176번길 22-3(두산동),최천,강원특별자치도 정선군 임계면 태봉안길 302,740716-1345719,NaN,20130429,NaN,NaN,40200720100
2,전북02사3394,VCEEC55CL00111007,경기02로5946_00,02,굴착기,4571040024,전라북도 완주군 동상면 은천길 31,배선동,전라북도 완주군 동상면 은천길 31,581103-1519931,--,20090521,NaN,NaN,40200720000
3,충북02도7320,VCEEC55CT00123311,충북02도7320_01,02,굴착기,4313011800,충청북도 충주시 예성로 320(연수동),전경님,"충청북도 충주시 금릉로 17, 101동 903호(칠금동, 삼일무지개아파트)",660310-2256419,NaN,20130212,20220222,20220321,40200720100
4,경남02보8670,VCEC380ET00311540,경남02보8670_01,02,굴착기,4833010700,경상남도 양산시 신기로 120(북정동),이태심,"부산광역시 동래구 시실로 73-6, 204동 702호(명륜동, 명륜2차아이파크)",600420-2919326,NaN,20180628,NaN,NaN,40201600000


In [42]:
ri_exc02 = ri_exc_all['Data_1']
ri_exc02.head()

,건설기계등록번호,건설기계차대번호,건설기계관리번호,건설기계코드,건설기계명,사용본거지법정동코드,사용본거지주소,대표소유자명,대표소유자주소,대표소유자번호,대표소유자전화번호,최초등록일,정비명령시작일자,정비명령종료일자,형식승인번호
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,건설기계등록번호,건설기계차대번호,건설기계관리번호,건설기계코드,건설기계명,사용본거지법정동코드,사용본거지주소,대표소유자명,대표소유자주소,대표소유자번호,대표소유자전화번호,최초등록일,정비명령시작일자,정비명령종료일자,형식승인번호
2,002가3934,HHIHQ404HB0000951,경기02타6962_01,02,굴착기,5178025900,강원특별자치도 철원군 동송읍 상노로 519,주식회사 철원중장비전문학원,강원특별자치도 철원군 동송읍 상노로 519,284711-0016587,--,20110720,NaN,NaN,10202200400
3,002라7816,VCEEW60ET00311669,세종02가5696_01,02,굴착기,4888025000,"경상남도 거창군 거창읍 창동로 171, 2층",김종식,경상남도 거창군 신원면 대현길 272-2,731207-1928118,010-8857-9448,20180516,NaN,NaN,10202330000
4,전북02사2762,DHKHEEAOS75005824,광주02라5121_00,02,굴착기,4514012900,전라북도 익산시 석왕동 107번지 24호,국오근,전라북도 익산시 쌍능길 23(석왕동),410819-2495511,063-831-0123,20070312,NaN,NaN,10202130200


In [43]:
ri_exc02.columns = ri_exc02.iloc[1, :].values
ri_exc02 = ri_exc02.iloc[2:, :].reset_index(drop=True)
ri_exc02.head()

,건설기계등록번호,건설기계차대번호,건설기계관리번호,건설기계코드,건설기계명,사용본거지법정동코드,사용본거지주소,대표소유자명,대표소유자주소,대표소유자번호,대표소유자전화번호,최초등록일,정비명령시작일자,정비명령종료일자,형식승인번호
0,002가3934,HHIHQ404HB0000951,경기02타6962_01,02,굴착기,5178025900,강원특별자치도 철원군 동송읍 상노로 519,주식회사 철원중장비전문학원,강원특별자치도 철원군 동송읍 상노로 519,284711-0016587,--,20110720,NaN,NaN,10202200400
1,002라7816,VCEEW60ET00311669,세종02가5696_01,02,굴착기,4888025000,"경상남도 거창군 거창읍 창동로 171, 2층",김종식,경상남도 거창군 신원면 대현길 272-2,731207-1928118,010-8857-9448,20180516,NaN,NaN,10202330000
2,전북02사2762,DHKHEEAOS75005824,광주02라5121_00,02,굴착기,4514012900,전라북도 익산시 석왕동 107번지 24호,국오근,전라북도 익산시 쌍능길 23(석왕동),410819-2495511,063-831-0123,20070312,NaN,NaN,10202130200
3,002마6742,KBCAZ25EEP3F10812,002마6742_01,02,굴착기,2914013200,광주광역시 서구 풍서우로 162(매월동),석정우,"광주광역시 광산구 목련로 330-27, 412호(신가동)",720927-1622016,NaN,20230920,NaN,NaN,50202390200
4,002나5989,DHKCEACCTN6001697,서울02더7659_01,02,굴착기,4575036000,전라북도 임실군 신덕면 불재로 622,(주)경각산수목장,전라북도 임실군 신덕면 불재로 622,214411-0008097,NaN,20220311,NaN,NaN,40202210000


In [44]:
ri_exc03 = ri_exc_all['Data_2']
ri_exc03.head()

,건설기계등록번호,건설기계차대번호,건설기계관리번호,건설기계코드,건설기계명,사용본거지법정동코드,사용본거지주소,대표소유자명,대표소유자주소,대표소유자번호,대표소유자전화번호,최초등록일,정비명령시작일자,정비명령종료일자,형식승인번호
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,건설기계등록번호,건설기계차대번호,건설기계관리번호,건설기계코드,건설기계명,사용본거지법정동코드,사용본거지주소,대표소유자명,대표소유자주소,대표소유자번호,대표소유자전화번호,최초등록일,정비명령시작일자,정비명령종료일자,형식승인번호
2,충북02모7835,DHKCEWAXKM6004678,충북02모7835_01,02,굴착기,4374025000,충청북도 영동군 영동읍 계산로 107-1,황종하,충청북도 영동군 영동읍 화신로 173,740427-1386219,NaN,20210705,NaN,NaN,10202320400
3,대구02마7885,DHKCEWAXCK6003629,대구02마7885_01,02,굴착기,2729012000,"대구광역시 달서구 성서공단로 346, 상가209호(월성동, 대구화물터미널)",김정곤,"대구광역시 달성군 옥포읍 비슬로447길 56, 107동 3201호(미진 이지비아)",731103-1768310,053-583-0306,20200110,NaN,NaN,10202320300
4,002나7928,0F046,경북02보7326_01,02,굴착기,4784037000,"경상북도 성주군 벽진면 성주로 2482, 1층",김덕남,"경상북도 울진군 평해읍 평해5길 17, 502호(정우빌라)",701004-2817611,053-527-9101,20170215,NaN,NaN,50201280200


In [45]:
ri_exc03.columns = ri_exc03.iloc[1, :].values
ri_exc03 = ri_exc03.iloc[2:, :].reset_index(drop=True)
ri_exc03.head()

,건설기계등록번호,건설기계차대번호,건설기계관리번호,건설기계코드,건설기계명,사용본거지법정동코드,사용본거지주소,대표소유자명,대표소유자주소,대표소유자번호,대표소유자전화번호,최초등록일,정비명령시작일자,정비명령종료일자,형식승인번호
0,충북02모7835,DHKCEWAXKM6004678,충북02모7835_01,02,굴착기,4374025000,충청북도 영동군 영동읍 계산로 107-1,황종하,충청북도 영동군 영동읍 화신로 173,740427-1386219,NaN,20210705,NaN,NaN,10202320400
1,대구02마7885,DHKCEWAXCK6003629,대구02마7885_01,02,굴착기,2729012000,"대구광역시 달서구 성서공단로 346, 상가209호(월성동, 대구화물터미널)",김정곤,"대구광역시 달성군 옥포읍 비슬로447길 56, 107동 3201호(미진 이지비아)",731103-1768310,053-583-0306,20200110,NaN,NaN,10202320300
2,002나7928,0F046,경북02보7326_01,02,굴착기,4784037000,"경상북도 성주군 벽진면 성주로 2482, 1층",김덕남,"경상북도 울진군 평해읍 평해5길 17, 502호(정우빌라)",701004-2817611,053-527-9101,20170215,NaN,NaN,50201280200
3,경남02소6725,DHKCEABHAL6005822,경남02소6725_01,02,굴착기,4827010200,경상남도 밀양시 밀성로 1(내이동),김기화,"경상남도 밀양시 새미2길 3, 2층(교동)",661007-2899639,NaN,20200330,NaN,NaN,40201720200
4,경북02오6939,HHKHM9A7VJ0001142,대구02마7048_01,02,굴착기,4729011200,경상북도 경산시 경안로67길 1-1(대평동),김종길,경상북도 경산시 자인면 교촌길1길 25,620602-1790318,NaN,20180517,NaN,NaN,10202350000


In [46]:
ri_exc = pd.concat([ri_exc01, ri_exc02, ri_exc03], ignore_index=True)
ri_exc.shape

(174242, 15)

In [47]:
len(ri_exc['건설기계차대번호'].unique())

173832

In [48]:
ri_exc.columns

Index(['건설기계등록번호', '건설기계차대번호', '건설기계관리번호', '건설기계코드', '건설기계명', '사용본거지법정동코드',
       '사용본거지주소', '대표소유자명', '대표소유자주소', '대표소유자번호', '대표소유자전화번호', '최초등록일',
       '정비명령시작일자', '정비명령종료일자', '형식승인번호'],
      dtype='object')

In [49]:
ri_exc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174242 entries, 0 to 174241
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   건설기계등록번호    174242 non-null  object
 1   건설기계차대번호    174242 non-null  object
 2   건설기계관리번호    174242 non-null  object
 3   건설기계코드      174242 non-null  object
 4   건설기계명       174242 non-null  object
 5   사용본거지법정동코드  174242 non-null  object
 6   사용본거지주소     174242 non-null  object
 7   대표소유자명      174241 non-null  object
 8   대표소유자주소     174242 non-null  object
 9   대표소유자번호     174170 non-null  object
 10  대표소유자전화번호   74607 non-null   object
 11  최초등록일       174242 non-null  object
 12  정비명령시작일자    17789 non-null   object
 13  정비명령종료일자    17789 non-null   object
 14  형식승인번호      174242 non-null  object
dtypes: object(15)
memory usage: 19.9+ MB


##### [출력] 등록_굴착기 시트 통합

In [50]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20240111'

In [53]:
# 0.5s
ri_exc.to_csv(os.path.join(raw2_fold, f'통합_등록정보_굴착기_{today_date}.csv'), index=False)

In [54]:
# 27.2s
ri_exc.to_excel(os.path.join(raw2_fold, f'통합_등록정보_굴착기_{today_date}.xlsx'), index=False)

#### 등록_기타

In [55]:
name = '등록정보_기타'
file_name = f'{name}.xlsx'
ri_etc = pd.read_excel(os.path.join(raw2_fold, file_name), sheet_name=None)

In [56]:
ri_etc.keys()

dict_keys(['Data'])

In [57]:
ri_etc = ri_etc['Data']
ri_etc.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,건설기계등록번호,건설기계차대번호,건설기계관리번호,건설기계코드,건설기계명,사용본거지법정동코드,사용본거지주소,대표소유자명,대표소유자주소,대표소유자번호,대표소유자전화번호,최초등록일,정비명령시작일자,정비명령종료일자,형식승인번호
2,부산22-6011,BCD-3600-0139,인천22마1074_01,22,천공기,2626010200,"부산광역시 동래구 안연로110번길 30, 2층",(주)삼정기초건설,"부산광역시 연제구 중앙대로 1069, 501호(연산동)",180111-1042918,NaN,20150529,20180531,20180830,42201430100
3,서울07가8381,25A-0506,서울07가8381_01,07,기중기,1156010800,"서울특별시 영등포구 국회대로50길 20, 103동 703호(영등포동7가, 포레나 영등포)",김상진,"경기도 고양시 일산서구 중앙로 1542, 822호(대화동, 신동아노블타워)",710817-1025137,NaN,20130104,NaN,NaN,50701510100
4,경기17고5358,VCEP7820K0H702375,서울17-5787_01,17,아스팔트 피니셔,4155012500,경기도 안성시 안성맞춤대로 1274(당왕동),(주)성림포장건설,경기도 안성시 보개면 보개원삼로 383,134211-0227341,031-671-3961,20150306,NaN,NaN,51700730100


In [58]:
ri_etc.columns = ri_etc.iloc[1, :].values
ri_etc = ri_etc.iloc[2:, :].reset_index(drop=True)
ri_etc.head()

,건설기계등록번호,건설기계차대번호,건설기계관리번호,건설기계코드,건설기계명,사용본거지법정동코드,사용본거지주소,대표소유자명,대표소유자주소,대표소유자번호,대표소유자전화번호,최초등록일,정비명령시작일자,정비명령종료일자,형식승인번호
0,부산22-6011,BCD-3600-0139,인천22마1074_01,22,천공기,2626010200,"부산광역시 동래구 안연로110번길 30, 2층",(주)삼정기초건설,"부산광역시 연제구 중앙대로 1069, 501호(연산동)",180111-1042918,NaN,20150529,20180531,20180830,42201430100
1,서울07가8381,25A-0506,서울07가8381_01,07,기중기,1156010800,"서울특별시 영등포구 국회대로50길 20, 103동 703호(영등포동7가, 포레나 영등포)",김상진,"경기도 고양시 일산서구 중앙로 1542, 822호(대화동, 신동아노블타워)",710817-1025137,NaN,20130104,NaN,NaN,50701510100
2,경기17고5358,VCEP7820K0H702375,서울17-5787_01,17,아스팔트 피니셔,4155012500,경기도 안성시 안성맞춤대로 1274(당왕동),(주)성림포장건설,경기도 안성시 보개면 보개원삼로 383,134211-0227341,031-671-3961,20150306,NaN,NaN,51700730100
3,경남07고6335,FB7510,경남07고6335_01,07,기중기,4812112900,경상남도 창원시 의창구 사화로 262(팔용동),조순자,"경상남도 김해시 율하3로 76, 802동 801호(율하동, 신리마을 중앙하이츠8단지)",570514-2894615,055-288-9999,20150413,NaN,NaN,20715000300
4,부산07가6334,24AP000210,서울07나5164_01,07,기중기,2623010200,"부산광역시 부산진구 동성로 133, 동연아미가 1308호",중앙건설산업 주식회사,"인천광역시 서구 장고개로217번길 2, 에이동 2층(가좌동)",120111-0910184,NaN,20150828,20220822,20220921,50701560200


In [71]:
ri_etc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47669 entries, 0 to 47668
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   건설기계등록번호    47669 non-null  object
 1   건설기계차대번호    47669 non-null  object
 2   건설기계관리번호    47669 non-null  object
 3   건설기계코드      47669 non-null  object
 4   건설기계명       47669 non-null  object
 5   사용본거지법정동코드  47669 non-null  object
 6   사용본거지주소     47667 non-null  object
 7   대표소유자명      47665 non-null  object
 8   대표소유자주소     47665 non-null  object
 9   대표소유자번호     47633 non-null  object
 10  대표소유자전화번호   29723 non-null  object
 11  최초등록일       47669 non-null  object
 12  정비명령시작일자    8506 non-null   object
 13  정비명령종료일자    8506 non-null   object
 14  형식승인번호      47669 non-null  object
dtypes: object(15)
memory usage: 5.5+ MB


##### [출력] 등록_기타 시트 통합

In [60]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20240111'

In [61]:
# 0.1s
ri_etc.to_csv(os.path.join(raw2_fold, f'통합_등록정보_기타_{today_date}.csv'), index=False)

In [62]:
# 7.6s
ri_etc.to_excel(os.path.join(raw2_fold, f'통합_등록정보_기타_{today_date}.xlsx'), index=False)

#### 등록_덤프트럭

In [63]:
name = '등록정보_덤프트럭'
file_name = f'{name}.xlsx'
ri_dump = pd.read_excel(os.path.join(raw2_fold, file_name), sheet_name=None)

In [64]:
ri_dump.keys()

dict_keys(['Data'])

In [65]:
ri_dump = ri_dump['Data']
ri_dump.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,건설기계등록번호,건설기계차대번호,건설기계관리번호,건설기계코드,건설기계명,사용본거지법정동코드,사용본거지주소,대표소유자명,대표소유자주소,대표소유자번호,대표소유자전화번호,최초등록일,정비명령시작일자,정비명령종료일자,형식승인번호
2,경기06오8901,WJMJ4CTS7HC364041,경기06오8901_01,06,덤프트럭,4122011800,"경기도 평택시 비전2로 79, 507호(비전동, 필그린타워)",서창효,"경기도 화성시 동탄중앙로 171, 353동 202호(반송동, 동탄시범다은마을 우남퍼...",720220-1916622,NaN,20180126,NaN,NaN,10602050000
3,006라7346,KMCSB18PPEC001881,경북06도8666_01,06,덤프트럭,3114010100,울산광역시 남구 남산로 48(무거동),김민정,"울산광역시 울주군 온양읍 연안8길 31, 506호(한그린아파트)",740909-2117715,NaN,20140415,20170411,20170610,10601760000
4,006다8576,YV2JS02G86A617633,강원06노7711_00,06,덤프트럭,2729011100,"대구광역시 달서구 장산남로 21, 1003호(용산동, 법조빌딩)",신기혁,"대구광역시 달서구 달구벌대로 1581, 101동 1104호(죽전동, 빌리브 죽전)",660825-1772519,053-572-2500,20060428,NaN,NaN,10600640000


In [66]:
ri_dump.columns = ri_dump.iloc[1, :].values
ri_dump = ri_dump.iloc[2:, :].reset_index(drop=True)
ri_dump.head()

,건설기계등록번호,건설기계차대번호,건설기계관리번호,건설기계코드,건설기계명,사용본거지법정동코드,사용본거지주소,대표소유자명,대표소유자주소,대표소유자번호,대표소유자전화번호,최초등록일,정비명령시작일자,정비명령종료일자,형식승인번호
0,경기06오8901,WJMJ4CTS7HC364041,경기06오8901_01,06,덤프트럭,4122011800,"경기도 평택시 비전2로 79, 507호(비전동, 필그린타워)",서창효,"경기도 화성시 동탄중앙로 171, 353동 202호(반송동, 동탄시범다은마을 우남퍼...",720220-1916622,NaN,20180126,NaN,NaN,10602050000
1,006라7346,KMCSB18PPEC001881,경북06도8666_01,06,덤프트럭,3114010100,울산광역시 남구 남산로 48(무거동),김민정,"울산광역시 울주군 온양읍 연안8길 31, 506호(한그린아파트)",740909-2117715,NaN,20140415,20170411,20170610,10601760000
2,006다8576,YV2JS02G86A617633,강원06노7711_00,06,덤프트럭,2729011100,"대구광역시 달서구 장산남로 21, 1003호(용산동, 법조빌딩)",신기혁,"대구광역시 달서구 달구벌대로 1581, 101동 1104호(죽전동, 빌리브 죽전)",660825-1772519,053-572-2500,20060428,NaN,NaN,10600640000
3,006라7727,KMCSB18KPKC019321,경기06모4415_01,06,덤프트럭,5113011700,강원특별자치도 원주시 늘품로 188-25(반곡동),권희재,"강원특별자치도 원주시 남원로469번길 82, 903동 904호(명륜동, 동보렉스아파트)",680823-1326517,033-735-6461,20190329,NaN,NaN,10601760100
4,경북06모7556,YV2JS02G16A617635,대전06다8879_00,06,덤프트럭,4725012200,경상북도 상주시 외답4길 2(외답동),신익민,"경상북도 구미시 고아읍 가들로 33, 101동 1203호(문성레이크자이)",910709-1120344,NaN,20060428,20221108,20221207,10600640000


In [70]:
ri_dump.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54133 entries, 0 to 54132
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   건설기계등록번호    54133 non-null  object
 1   건설기계차대번호    54133 non-null  object
 2   건설기계관리번호    54133 non-null  object
 3   건설기계코드      54133 non-null  object
 4   건설기계명       54133 non-null  object
 5   사용본거지법정동코드  54133 non-null  object
 6   사용본거지주소     54133 non-null  object
 7   대표소유자명      54132 non-null  object
 8   대표소유자주소     54133 non-null  object
 9   대표소유자번호     54122 non-null  object
 10  대표소유자전화번호   26795 non-null  object
 11  최초등록일       54133 non-null  object
 12  정비명령시작일자    8079 non-null   object
 13  정비명령종료일자    8079 non-null   object
 14  형식승인번호      54133 non-null  object
dtypes: object(15)
memory usage: 6.2+ MB


##### [출력] 등록_덤프트럭 시트 통합

In [67]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20240111'

In [68]:
# 0.1s
ri_dump.to_csv(os.path.join(raw2_fold, f'통합_등록정보_덤프트럭_{today_date}.csv'), index=False)

In [69]:
# 8.6s
ri_dump.to_excel(os.path.join(raw2_fold, f'통합_등록정보_덤프트럭_{today_date}.xlsx'), index=False)

#### 등록_로더, 콘크리트 믹서트럭

In [72]:
name = '등록정보_로더, 콘크리트 믹서트럭'
file_name = f'{name}.xlsx'
ri_lc = pd.read_excel(os.path.join(raw2_fold, file_name), sheet_name=None)

In [73]:
ri_lc.keys()

dict_keys(['Data'])

In [74]:
ri_lc = ri_lc['Data']
ri_lc.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,건설기계등록번호,건설기계차대번호,건설기계관리번호,건설기계코드,건설기계명,사용본거지법정동코드,사용본거지주소,대표소유자명,대표소유자주소,대표소유자번호,대표소유자전화번호,최초등록일,정비명령시작일자,정비명령종료일자,형식승인번호
2,울산14가7503,KMCDH18JPHC094414,광주14다8131_01,14,콘크리트 믹서트럭,3171034000,울산광역시 울주군 웅촌면 웅촌로 607-13,현대종합건설연합 주식회사,울산광역시 울주군 언양읍 북문13길 12,230111-0279040,052-263-2138,20160913,NaN,NaN,11400620100
3,서울14가3911,KMCDH18JPHC095458,서울14가3911_01,14,콘크리트 믹서트럭,1111012400,"서울특별시 종로구 종로1길 42, 15층(수송동, 이마빌딩)",삼표산업,"서울특별시 종로구 종로1길 42, 15층(수송동, 이마빌딩)",110111-5237452,02-460-7267,20161102,NaN,NaN,11400620100
4,경북03고3894,HHKHLK01AC0000388,전북03바2463_01,03,로더,4711135021,경상북도 포항시 남구 호미곶면 강사리 291번지,(주)명보산업,경상북도 포항시 남구 호미곶면 강사리 291번지,164511-0021816,041-936-2533,20120921,NaN,NaN,10301200100


In [75]:
ri_lc.columns = ri_lc.iloc[1, :].values
ri_lc = ri_lc.iloc[2:, :].reset_index(drop=True)
ri_lc.head()

,건설기계등록번호,건설기계차대번호,건설기계관리번호,건설기계코드,건설기계명,사용본거지법정동코드,사용본거지주소,대표소유자명,대표소유자주소,대표소유자번호,대표소유자전화번호,최초등록일,정비명령시작일자,정비명령종료일자,형식승인번호
0,울산14가7503,KMCDH18JPHC094414,광주14다8131_01,14,콘크리트 믹서트럭,3171034000,울산광역시 울주군 웅촌면 웅촌로 607-13,현대종합건설연합 주식회사,울산광역시 울주군 언양읍 북문13길 12,230111-0279040,052-263-2138,20160913,NaN,NaN,11400620100
1,서울14가3911,KMCDH18JPHC095458,서울14가3911_01,14,콘크리트 믹서트럭,1111012400,"서울특별시 종로구 종로1길 42, 15층(수송동, 이마빌딩)",삼표산업,"서울특별시 종로구 종로1길 42, 15층(수송동, 이마빌딩)",110111-5237452,02-460-7267,20161102,NaN,NaN,11400620100
2,경북03고3894,HHKHLK01AC0000388,전북03바2463_01,03,로더,4711135021,경상북도 포항시 남구 호미곶면 강사리 291번지,(주)명보산업,경상북도 포항시 남구 호미곶면 강사리 291번지,164511-0021816,041-936-2533,20120921,NaN,NaN,10301200100
3,경북03노1817,HHKHSE00HS0000677,경북03노1817_01,03,로더,4711331023,경상북도 포항시 북구 신광면 비학로696번길 21,김명수,경상북도 포항시 북구 신광면 비학로696번길 21,571015-1783116,NaN,20211105,NaN,NaN,10301490100
4,경북14고6884,KMCDH18TP5C003476,경기14노5360_00,14,콘크리트 믹서트럭,4717010900,"경상북도 안동시 육사로 333, 2층 201호(법흥동, 안동강변펠리시아아파트 근린생...",안중석,경상북도 안동시 안기2길 115(안기동),590112-1775414,054-841-8300,20050518,NaN,NaN,11400130600


In [76]:
ri_lc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57743 entries, 0 to 57742
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   건설기계등록번호    57743 non-null  object
 1   건설기계차대번호    57743 non-null  object
 2   건설기계관리번호    57743 non-null  object
 3   건설기계코드      57743 non-null  object
 4   건설기계명       57743 non-null  object
 5   사용본거지법정동코드  57742 non-null  object
 6   사용본거지주소     57743 non-null  object
 7   대표소유자명      57743 non-null  object
 8   대표소유자주소     57742 non-null  object
 9   대표소유자번호     57681 non-null  object
 10  대표소유자전화번호   27704 non-null  object
 11  최초등록일       57743 non-null  object
 12  정비명령시작일자    5759 non-null   object
 13  정비명령종료일자    5759 non-null   object
 14  형식승인번호      57743 non-null  object
dtypes: object(15)
memory usage: 6.6+ MB


##### [출력] 등록_로더, 콘크리트 믹서트럭 시트 통합

In [77]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20240111'

In [78]:
# 0.1s
ri_lc.to_csv(os.path.join(raw2_fold, f'통합_등록정보_로더_콘크리트믹서트럭_{today_date}.csv'), index=False)

In [79]:
# 8.6s
ri_lc.to_excel(os.path.join(raw2_fold, f'통합_등록정보_로더_콘크리트믹서트럭_{today_date}.xlsx'), index=False)

#### 등록_지게차

In [80]:
name = '등록정보_지게차'
file_name = f'{name}.xlsx'
ri_lift = pd.read_excel(os.path.join(raw2_fold, file_name), sheet_name=None)

In [81]:
ri_lift.keys()

dict_keys(['Data', 'Data_1', 'Data_2', 'Data_3'])

In [82]:
ri_lift01 = ri_lift['Data']
ri_lift01.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,건설기계등록번호,건설기계차대번호,건설기계관리번호,건설기계코드,건설기계명,사용본거지법정동코드,사용본거지주소,대표소유자명,대표소유자주소,대표소유자번호,대표소유자전화번호,최초등록일,정비명령시작일자,정비명령종료일자,형식승인번호
2,충북04고8020,HHKHHN13AJ0002562,충북04고8020_01,04,지게차,4375035000,충청북도 진천군 이월면 진광로 279-10,에이제이네트웍스(주),서울특별시 송파구 정의로8길 9(문정동),110111-1874654,NaN,20180226,NaN,NaN,40403311000
3,광주04라1314,D30S-3-4405,경기04로3924_00,04,지게차,2914011800,"광주광역시 서구 금부로95번길 42, 1층(쌍촌동)",주식회사 농심바이오,"광주광역시 서구 금부로95번길 42, 1층(쌍촌동)",200111-0449178,NaN,20040217,NaN,NaN,10402580000
4,전남04마4027,GTS232D04609820KF,전남04가9917_01,04,지게차,4691035027,전라남도 신안군 도초면 난말길 90,고재삼,전라남도 신안군 도초면 난말길 90,570818-1006118,NaN,20111007,NaN,NaN,40402370200


In [83]:
ri_lift01.columns = ri_lift01.iloc[1, :].values
ri_lift01 = ri_lift01.iloc[2:, :].reset_index(drop=True)
ri_lift01.head()

,건설기계등록번호,건설기계차대번호,건설기계관리번호,건설기계코드,건설기계명,사용본거지법정동코드,사용본거지주소,대표소유자명,대표소유자주소,대표소유자번호,대표소유자전화번호,최초등록일,정비명령시작일자,정비명령종료일자,형식승인번호
0,충북04고8020,HHKHHN13AJ0002562,충북04고8020_01,04,지게차,4375035000,충청북도 진천군 이월면 진광로 279-10,에이제이네트웍스(주),서울특별시 송파구 정의로8길 9(문정동),110111-1874654,NaN,20180226,NaN,NaN,40403311000
1,광주04라1314,D30S-3-4405,경기04로3924_00,04,지게차,2914011800,"광주광역시 서구 금부로95번길 42, 1층(쌍촌동)",주식회사 농심바이오,"광주광역시 서구 금부로95번길 42, 1층(쌍촌동)",200111-0449178,NaN,20040217,NaN,NaN,10402580000
2,전남04마4027,GTS232D04609820KF,전남04가9917_01,04,지게차,4691035027,전라남도 신안군 도초면 난말길 90,고재삼,전라남도 신안군 도초면 난말길 90,570818-1006118,NaN,20111007,NaN,NaN,40402370200
3,경남04도4198,D70S5-03173,경남04도4198_00,04,지게차,4812710300,경상남도 창원시마산회원구 봉암공단8길 77(봉암동),(주)두산중장비센터,경상남도 창원시마산회원구 봉암공단8길 77(봉암동),190111-0035657,055-294-4001,20100615,20221215,20230114,40401510100
4,전북04아1848,7RF02638,경북04-4236_00,04,지게차,4571025324,"전라북도 완주군 봉동읍 둔산3로 63, 102동 1005호(전주첨단코아루아파트)",김선열,"전라북도 완주군 봉동읍 둔산3로 63, 102동 1005호(전주첨단코아루아파트)",720315-1547717,NaN,19930805,NaN,NaN,10400690000


In [84]:
ri_lift02 = ri_lift['Data_1']
ri_lift02.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,건설기계등록번호,건설기계차대번호,건설기계관리번호,건설기계코드,건설기계명,사용본거지법정동코드,사용본거지주소,대표소유자명,대표소유자주소,대표소유자번호,대표소유자전화번호,최초등록일,정비명령시작일자,정비명령종료일자,형식승인번호
2,충북04노1777,FD35M-4706,충북04노1777_00,04,지게차,4315011600,"충청북도 제천시 신죽하로 83-24, 101동 901호(장락동, 롯데캐슬아파트)",인동진,"충청북도 제천시 신죽하로 83-24, 101동 901호(장락동, 롯데캐슬아파트)",700420-1473814,011-469-9416,19920316,20200525,20200624,10400160000
3,충북04도3490,FDA05-1850-11276,경기04하1145_01,04,지게차,4311425000,충청북도 청주시 청원구 내수읍 도원세교로 59-30,최용락,"충청북도 청주시 서원구 산남로 23, 111동 1002호(산남동, 계룡리슈빌아파트)",800514-1381019,--,20121121,NaN,NaN,40402570000
4,대구04나4250,FDA06-1850-19860,대구04나4250_01,04,지게차,2723012300,"대구광역시 북구 구암로65길 9, 305동 705호(구암동, 화성그랜드파크)",이희진,"대구광역시 북구 구암로65길 9, 305동 705호(구암동, 화성그랜드파크)",750110-1770611,NaN,20150305,NaN,NaN,40402580000


In [85]:
ri_lift02.columns = ri_lift02.iloc[1, :].values
ri_lift02 = ri_lift02.iloc[2:, :].reset_index(drop=True)
ri_lift02.head()

,건설기계등록번호,건설기계차대번호,건설기계관리번호,건설기계코드,건설기계명,사용본거지법정동코드,사용본거지주소,대표소유자명,대표소유자주소,대표소유자번호,대표소유자전화번호,최초등록일,정비명령시작일자,정비명령종료일자,형식승인번호
0,충북04노1777,FD35M-4706,충북04노1777_00,04,지게차,4315011600,"충청북도 제천시 신죽하로 83-24, 101동 901호(장락동, 롯데캐슬아파트)",인동진,"충청북도 제천시 신죽하로 83-24, 101동 901호(장락동, 롯데캐슬아파트)",700420-1473814,011-469-9416,19920316,20200525,20200624,10400160000
1,충북04도3490,FDA05-1850-11276,경기04하1145_01,04,지게차,4311425000,충청북도 청주시 청원구 내수읍 도원세교로 59-30,최용락,"충청북도 청주시 서원구 산남로 23, 111동 1002호(산남동, 계룡리슈빌아파트)",800514-1381019,--,20121121,NaN,NaN,40402570000
2,대구04나4250,FDA06-1850-19860,대구04나4250_01,04,지게차,2723012300,"대구광역시 북구 구암로65길 9, 305동 705호(구암동, 화성그랜드파크)",이희진,"대구광역시 북구 구암로65길 9, 305동 705호(구암동, 화성그랜드파크)",750110-1770611,NaN,20150305,NaN,NaN,40402580000
3,인천04바5026,D70S-00878,인천04바1412_00,04,지게차,2817010100,인천광역시 남구 숭의동 343번지 9호 701호,김태수,"인천광역시 연수구 랜드마크로 68, 107동 1305호(송도동, 랜드마크시티 센트럴...",640514-1143121,NaN,20020709,NaN,NaN,10402340000
4,전북04사3389,GTS232D07309820KF,전북04사3389_01,04,지게차,4577025025,전라북도 순창군 순창읍 백산리 798번지,농업회사법인 한국절임(주),전라북도 순창군 순창읍 백산리 798번지,214511-0003251,063-652-1900,20120508,NaN,NaN,40402380100


In [86]:
ri_lift03 = ri_lift['Data_2']
ri_lift03.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,건설기계등록번호,건설기계차대번호,건설기계관리번호,건설기계코드,건설기계명,사용본거지법정동코드,사용본거지주소,대표소유자명,대표소유자주소,대표소유자번호,대표소유자전화번호,최초등록일,정비명령시작일자,정비명령종료일자,형식승인번호
2,전남04사4670,HHKHHN51KF0003348,전남04사4670_01,04,지게차,4677037027,전라남도 고흥군 점암면 모룡길 71,김생빈,전라남도 고흥군 점암면 모룡길 71,550128-1637518,NaN,20211220,NaN,NaN,40403951400
3,대구04다3337,부산-04나2699,부산04나2699_00,04,지게차,2771026526,대구광역시 달성군 현풍읍 현풍서로 6,서정목,대구광역시 달성군 현풍읍 현풍서로 6,650221-1768621,NaN,20060203,NaN,NaN,10402580000
4,충남04로3577,D30S5-0703,충남04고6222_00,04,지게차,4420040029,충청남도 아산시 도고면 도고산로74번길 33,이기항,충청남도 아산시 도고면 도고산로74번길 33,560319-1476710,011-2900-0254,20071017,NaN,NaN,40401390100


In [87]:
ri_lift03.columns = ri_lift03.iloc[1, :].values
ri_lift03 = ri_lift03.iloc[2:, :].reset_index(drop=True)
ri_lift03.head()

,건설기계등록번호,건설기계차대번호,건설기계관리번호,건설기계코드,건설기계명,사용본거지법정동코드,사용본거지주소,대표소유자명,대표소유자주소,대표소유자번호,대표소유자전화번호,최초등록일,정비명령시작일자,정비명령종료일자,형식승인번호
0,전남04사4670,HHKHHN51KF0003348,전남04사4670_01,04,지게차,4677037027,전라남도 고흥군 점암면 모룡길 71,김생빈,전라남도 고흥군 점암면 모룡길 71,550128-1637518,NaN,20211220,NaN,NaN,40403951400
1,대구04다3337,부산-04나2699,부산04나2699_00,04,지게차,2771026526,대구광역시 달성군 현풍읍 현풍서로 6,서정목,대구광역시 달성군 현풍읍 현풍서로 6,650221-1768621,NaN,20060203,NaN,NaN,10402580000
2,충남04로3577,D30S5-0703,충남04고6222_00,04,지게차,4420040029,충청남도 아산시 도고면 도고산로74번길 33,이기항,충청남도 아산시 도고면 도고산로74번길 33,560319-1476710,011-2900-0254,20071017,NaN,NaN,40401390100
3,부산04라1089,FDA0R-2340-01820,부산04라1089_01,04,지게차,2620011400,부산광역시 영도구 하나길 113(신선동3가),고봉우,부산광역시 영도구 하나길 152(신선동3가),750310-1105828,051-414-7579,20160804,NaN,NaN,40403660200
4,대구04가5770,D45S-201283,부산04가7506_00,04,지게차,2729012100,대구광역시 달서구 성서로36길 16(월암동),김형수,"경상북도 칠곡군 왜관읍 전원1길 20, 108동 102호(로얄맨션)",650814-1796711,053-581-4111,20040203,NaN,NaN,10402710000


In [88]:
ri_lift04 = ri_lift['Data_3']
ri_lift04.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,건설기계등록번호,건설기계차대번호,건설기계관리번호,건설기계코드,건설기계명,사용본거지법정동코드,사용본거지주소,대표소유자명,대표소유자주소,대표소유자번호,대표소유자전화번호,최초등록일,정비명령시작일자,정비명령종료일자,형식승인번호
2,전남04라1280,H81012887,전남04라1280_00,04,지게차,4615038032,전라남도 순천시 낙안면 이곡1길 56,김면수,전라남도 순천시 낙안면 이곡1길 56,551110-1635111,NaN,20030905,20181010,20181109,40400210100
3,004나9225,FDA2G-5122-00497,004나9225_01,04,지게차,4825025000,경상남도 김해시 진영읍 본산로241번길 6,두산물류 주식회사,경상남도 김해시 진영읍 본산로241번길 6,195511-0150212,055-328-4000,20231130,NaN,NaN,40404740000
4,004나5921,FDB0D100001658,서울04라4072_01,04,지게차,4615032000,전라남도 순천시 서면 매천로 221(대성중기매매상사 송영미-상품용),송영미,"전라남도 순천시 조례1길 60, 104동 403호(조례동, 남양휴튼아파트)",710801-2573422,061-724-5886,20111012,NaN,NaN,40401790100


In [89]:
ri_lift04.columns = ri_lift04.iloc[1, :].values
ri_lift04 = ri_lift04.iloc[2:, :].reset_index(drop=True)
ri_lift04.head()

,건설기계등록번호,건설기계차대번호,건설기계관리번호,건설기계코드,건설기계명,사용본거지법정동코드,사용본거지주소,대표소유자명,대표소유자주소,대표소유자번호,대표소유자전화번호,최초등록일,정비명령시작일자,정비명령종료일자,형식승인번호
0,전남04라1280,H81012887,전남04라1280_00,04,지게차,4615038032,전라남도 순천시 낙안면 이곡1길 56,김면수,전라남도 순천시 낙안면 이곡1길 56,551110-1635111,NaN,20030905,20181010,20181109,40400210100
1,004나9225,FDA2G-5122-00497,004나9225_01,04,지게차,4825025000,경상남도 김해시 진영읍 본산로241번길 6,두산물류 주식회사,경상남도 김해시 진영읍 본산로241번길 6,195511-0150212,055-328-4000,20231130,NaN,NaN,40404740000
2,004나5921,FDB0D100001658,서울04라4072_01,04,지게차,4615032000,전라남도 순천시 서면 매천로 221(대성중기매매상사 송영미-상품용),송영미,"전라남도 순천시 조례1길 60, 104동 403호(조례동, 남양휴튼아파트)",710801-2573422,061-724-5886,20111012,NaN,NaN,40401790100
3,004나8803,FDB0V-2610-03218,경기04로8878_01,04,지게차,4719011800,"경상북도 구미시 금오대로22길 29-6, 1층(오태동)",서현진,"경상북도 구미시 흥안로 43, 103동 702호(옥계동, 동화타운)",770326-2675924,NaN,20190108,NaN,NaN,40403930000
4,전북04나2758,D70S-00152,전북04나2758_00,04,지게차,4514011700,전라북도 익산시 목천동 719번지 5호,이영희,"전라북도 완주군 삼례읍 삼봉5로 55, 302동 312호(완주삼봉엘에이치아파트)",510121-1495717,NaN,19971007,20101011,20110110,10402340000


In [90]:
ri_lift = pd.concat([ri_lift01, ri_lift02, ri_lift03, ri_lift04], ignore_index=True)
ri_lift.shape

(215852, 15)

In [91]:
len(ri_lift['건설기계차대번호'].unique())

215762

In [92]:
ri_lift.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215852 entries, 0 to 215851
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   건설기계등록번호    215852 non-null  object
 1   건설기계차대번호    215852 non-null  object
 2   건설기계관리번호    215852 non-null  object
 3   건설기계코드      215852 non-null  object
 4   건설기계명       215852 non-null  object
 5   사용본거지법정동코드  215852 non-null  object
 6   사용본거지주소     215852 non-null  object
 7   대표소유자명      215852 non-null  object
 8   대표소유자주소     215852 non-null  object
 9   대표소유자번호     215525 non-null  object
 10  대표소유자전화번호   124098 non-null  object
 11  최초등록일       215852 non-null  object
 12  정비명령시작일자    32404 non-null   object
 13  정비명령종료일자    32404 non-null   object
 14  형식승인번호      215852 non-null  object
dtypes: object(15)
memory usage: 24.7+ MB


##### [출력] 등록_지게차 시트 통합

In [94]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20240111'

In [95]:
# 0.7s
ri_lift.to_csv(os.path.join(raw2_fold, f'통합_등록정보_지게차_{today_date}.csv'), index=False)

In [96]:
# 33.1s
ri_lift.to_excel(os.path.join(raw2_fold, f'통합_등록정보_지게차_{today_date}.xlsx'), index=False)

#### 제원_건설기계

In [97]:
name = '제원정보_건설기계'
file_name = f'{name}.xlsx'
si_cm = pd.read_excel(os.path.join(raw2_fold, file_name), sheet_name=None)

In [98]:
si_cm.keys()

dict_keys(['Data'])

In [99]:
si_cm = si_cm['Data']
si_cm.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,형식승인번호,형식승인구분코드,형식승인일,형식명,제작국가명,제작자구분코드,제작업체코드,제작업체명,수입업체코드,규격명세,...,기계너비,기계높이,총중량값,엔진형식,엔진기통수값,엔진정격출력값,엔진정격RPM값,사용연료코드,주행방식코드,주행전동기형식
2,10400300000,1,19821124,FB20-2,대한민국,1,0002,대우종합기계(주),NaN,2ton,...,1598,2220,3717,NaN,NaN,6.5kw,1230,NaN,1,직류직권
3,10200200000,1,19830622,DH02S,대한민국,1,0002,대우종합기계(주),NaN,4.4ton,...,1840,2440,4431,C240-FEA,4,35,2400,2,2,NaN
4,10300080000,1,19820719,FR20,대한민국,1,0081,한국중공업(주),NaN,3.0㎥,...,2890,3504,19300,FIAT8215,6,226,2200,2,1,NaN


In [100]:
si_cm.columns = si_cm.iloc[1, :].values
si_cm = si_cm.iloc[2:, :].reset_index(drop=True)
si_cm.head()

,형식승인번호,형식승인구분코드,형식승인일,형식명,제작국가명,제작자구분코드,제작업체코드,제작업체명,수입업체코드,규격명세,...,기계너비,기계높이,총중량값,엔진형식,엔진기통수값,엔진정격출력값,엔진정격RPM값,사용연료코드,주행방식코드,주행전동기형식
0,10400300000,1,19821124,FB20-2,대한민국,1,0002,대우종합기계(주),NaN,2ton,...,1598,2220,3717,NaN,NaN,6.5kw,1230,NaN,1,직류직권
1,10200200000,1,19830622,DH02S,대한민국,1,0002,대우종합기계(주),NaN,4.4ton,...,1840,2440,4431,C240-FEA,4,35,2400,2,2,NaN
2,10300080000,1,19820719,FR20,대한민국,1,0081,한국중공업(주),NaN,3.0㎥,...,2890,3504,19300,FIAT8215,6,226,2200,2,1,NaN
3,11500050000,1,19830829,HD8C-1,대한민국,1,0025,현대자동차(주),NaN,66㎥/h,...,2475,3650,14145,D0846HM,6,185,2500,2,1,NaN
4,10800010000,1,19820602,FG-85,대한민국,1,0081,한국중공업(주),NaN,3.6m,...,2306,3264,14091,FIAT 8365,6,145,2200,2,1,NaN


In [101]:
si_cm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32081 entries, 0 to 32080
Data columns (total 21 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   형식승인번호    32081 non-null  object
 1   형식승인구분코드  32081 non-null  object
 2   형식승인일     32075 non-null  object
 3   형식명       32077 non-null  object
 4   제작국가명     27778 non-null  object
 5   제작자구분코드   29726 non-null  object
 6   제작업체코드    29220 non-null  object
 7   제작업체명     28951 non-null  object
 8   수입업체코드    13066 non-null  object
 9   규격명세      31879 non-null  object
 10  기계길이      29391 non-null  object
 11  기계너비      29391 non-null  object
 12  기계높이      29356 non-null  object
 13  총중량값      31487 non-null  object
 14  엔진형식      29090 non-null  object
 15  엔진기통수값    28377 non-null  object
 16  엔진정격출력값   28450 non-null  object
 17  엔진정격RPM값  28363 non-null  object
 18  사용연료코드    28353 non-null  object
 19  주행방식코드    29939 non-null  object
 20  주행전동기형식   680 non-null    object
dtypes: object(21

##### [출력] 등록_건설기계 시트 통합

In [102]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20240111'

In [103]:
# 0.0s
si_cm.to_csv(os.path.join(raw2_fold, f'통합_제원정보_건설기계_{today_date}.csv'), index=False)

In [104]:
# 6.4s
si_cm.to_excel(os.path.join(raw2_fold, f'통합_제원정보_건설기계_{today_date}.xlsx'), index=False)

## 전처리

### 인증자료
- tcmaster 
    - 배출인증번호 중복 제거
        - 기준 : 배출인증번호, 처분내용(인증완료 우선)
- eng_tefspecm
    - ADM_NO 중복 제거
    - ENG_CAR_GB 컬럼 제거

In [172]:
tcmaster.columns

Index(['ADM_NO', 'FACT_SNO', 'CERTI_DIV', 'REG_DATETIME', 'RECT_DATE',
       'EMIS_CERTI_NO', 'NOISE_CERTI_NO', 'EMIS_OMIT_NO', 'NOISE_OMIT_NO',
       'EMIS_CERTI_DATE',
       ...
       'SNOTE2', 'EMIS_CANCEL_DATE', 'NOISE_CANCEL_DATE', 'REG_ID', 'MOD_ID',
       'MOD_DATETIME', 'ENG_CAR_GB', 'EMIS_NOTE2', 'EMIS_STD_NOTE',
       'NOISE_STD_NOTE'],
      dtype='object', length=159)

In [189]:
temp = tcmaster.copy()
temp['CERTI_STATE'] = temp['CERTI_STATE'].replace({'90':'인증완료'})
print(temp.CERTI_STATE.unique())
display(temp.sort_values(['EMIS_CERTI_NO', 'CERTI_STATE'], ascending=[True, False])[['EMIS_CERTI_NO', 'CERTI_STATE']])

[nan '인증완료' 'C0' '10' '42' '46' '41']


,EMIS_CERTI_NO,CERTI_STATE
421,08EN*VO*01,인증완료
195,09EN*CA*01,인증완료
341,09EN*CA*02,NaN
333,09EN*CA*04,인증완료
152,09EN*CA*05,인증완료
...,...,...
442,NaN,NaN
443,NaN,NaN
447,NaN,NaN
448,NaN,NaN


In [190]:
tcmaster['CERTI_STATE'].value_counts(dropna=False)

CERTI_STATE
90     417
NaN    174
C0      19
42      11
10       1
46       1
41       1
Name: count, dtype: int64

In [333]:
tcmasterm = tcmaster.copy()
tcmasterm['CERTI_STATE'] = tcmasterm['CERTI_STATE'].replace({'90':'인증완료'})
tcmasterm = tcmasterm.sort_values(['EMIS_CERTI_NO', 'CERTI_STATE'], ascending=[True, False]).drop_duplicates(['EMIS_CERTI_NO']).reset_index(drop=True)
tcmasterm['CERTI_STATE'] = tcmasterm['CERTI_STATE'].replace({'인증완료':'90'})
tcmasterm['CERTI_STATE'].value_counts(dropna=False)

CERTI_STATE
90     417
C0       8
NaN      1
Name: count, dtype: int64

In [399]:
tcmasterm['ENG_CAR_GB'].unique()

array(['C', nan, 'A', 'CA'], dtype=object)

In [407]:
tcmasterm.loc[(tcmasterm['ENG_CAR_GB'] == 'C') & (tcmasterm['CERTI_STATE'] == '90'), 'EMIS_CERTI_NO'].isnull().sum()

1

In [410]:
tcmasterm.loc[(tcmasterm['ENG_CAR_GB'] == 'C') & (tcmasterm['CERTI_STATE'] == '90'), 'EMIS_CERTI_NO'].notnull().sum()

236

In [408]:
tcmasterm.loc[(tcmasterm['ENG_CAR_GB'] == 'A') & (tcmasterm['CERTI_STATE'] == '90'), 'EMIS_CERTI_NO'].isnull().sum()

0

In [404]:
len(tcmasterm.loc[tcmasterm['ENG_CAR_GB'] == 'A', 'CERTI_STATE'].unique())

2

In [409]:
tcmasterm.loc[(tcmasterm['ENG_CAR_GB'] == 'CA') & (tcmasterm['CERTI_STATE'] == '90'), 'EMIS_CERTI_NO'].isnull().sum()

0

In [405]:
len(tcmasterm.loc[tcmasterm['ENG_CAR_GB'] == 'CA', 'CERTI_STATE'].unique())

1

In [460]:
eng_tefspecm = eng_tefspec.drop_duplicates(['ADM_NO']).reset_index(drop=True)
eng_tefspecm = eng_tefspecm.drop(['ENG_CAR_GB'], axis=1)
eng_tefspecm.shape

(472, 107)

### 인증대장

#### 엔진배기량 구분

In [1282]:
cert_ld34.shape

(5971, 64)

In [1283]:
cert_ld34['엔진배기량'].isnull().sum()

109

In [1284]:
cert_ld34['엔진배기량'].dtype

dtype('O')

In [1285]:
cert_ld34.loc[cert_ld34['엔진배기량'] == '"', '엔진배기량'] = np.nan

In [1286]:
for one in cert_ld34.loc[cert_ld34['엔진배기량'].notnull(), '엔진배기량']:
    if type(one) != int:
        print(one)

In [1287]:
cert_ld34['엔진배기량'] = pd.to_numeric(cert_ld34['엔진배기량'], errors='coerce')

In [1288]:
cert_ld34['엔진배기량'].describe()

count    5861.00
mean     6241.37
std      3820.31
min       479.00
25%      3387.00
50%      5132.00
75%      7698.00
max     27030.00
Name: 엔진배기량, dtype: float64

In [1289]:
cert_ld34['엔진배기량'].isnull().sum()

110

In [1290]:
div = 2000
tf_col = '엔진배기량'
tfed_col = '엔진배기량(cc)구간'
n = int(cert_ld34[tf_col].max() // div)
for i in range(n + 1):
    cert_ld34.loc[(cert_ld34[tf_col] >= i*div) & (cert_ld34[tf_col] < (i+1)*div), tfed_col] = f'{i*div:,}이상 ~ {(i+1)*div:,}미만'
cert_ld34[tfed_col].unique()

array(['10,000이상 ~ 12,000미만', nan, '4,000이상 ~ 6,000미만',
       '6,000이상 ~ 8,000미만', '14,000이상 ~ 16,000미만', '8,000이상 ~ 10,000미만',
       '2,000이상 ~ 4,000미만', '12,000이상 ~ 14,000미만', '18,000이상 ~ 20,000미만',
       '26,000이상 ~ 28,000미만', '0이상 ~ 2,000미만', '22,000이상 ~ 24,000미만',
       '16,000이상 ~ 18,000미만'], dtype=object)

In [1291]:
cert_ld5.shape

(2055, 74)

In [1292]:
cert_ld5['엔진배기량'].isnull().sum()

1

In [1293]:
cert_ld5['엔진배기량'].dtype

dtype('float64')

In [1294]:
cert_ld5['엔진배기량'].describe()

count    2054.00
mean     5433.84
std      3630.69
min       479.00
25%      2615.00
50%      4399.00
75%      7014.00
max     18131.00
Name: 엔진배기량, dtype: float64

In [1295]:
div = 2000
tf_col = '엔진배기량'
tfed_col = '엔진배기량(cc)구간'
n = int(cert_ld5[tf_col].max() // div)
for i in range(n + 1):
    cert_ld5.loc[(cert_ld5[tf_col] >= i*div) & (cert_ld5[tf_col] < (i+1)*div), tfed_col] = f'{i*div:,}이상 ~ {(i+1)*div:,}미만'
cert_ld5[tfed_col].unique()

array(['6,000이상 ~ 8,000미만', '4,000이상 ~ 6,000미만', '2,000이상 ~ 4,000미만',
       '0이상 ~ 2,000미만', '10,000이상 ~ 12,000미만', '12,000이상 ~ 14,000미만',
       '16,000이상 ~ 18,000미만', nan, '14,000이상 ~ 16,000미만',
       '18,000이상 ~ 20,000미만', '8,000이상 ~ 10,000미만'], dtype=object)

#### 최대출력_KW 구분

In [1296]:
cert_ld34.shape

(5971, 65)

In [1297]:
cert_ld34['최대출력_KW'].isnull().sum()

5705

In [1298]:
cert_ld34['최대출력_KW'].dtype

dtype('O')

In [1299]:
cert_ld34['최대출력_KW'].head()

3    250(325ps)
4    250(325ps)
5    250(325ps)
6    250(325ps)
7    250(325ps)
Name: 최대출력_KW, dtype: object

In [1300]:
cert_ld34['최대출력_KW'] = cert_ld34['최대출력_KW'].str.split('(').str[0]
cert_ld34['최대출력_KW'].head()

3    250
4    250
5    250
6    250
7    250
Name: 최대출력_KW, dtype: object

In [1301]:
for one in cert_ld34.loc[cert_ld34['최대출력_KW'].notnull(), '최대출력_KW']:
    try:
        one = int(one)
        if type(one) != int:
            print(one)
        elif np.isnan(one):
            pass
    except:
        pass

In [1302]:
cert_ld34['최대출력_KW'] = pd.to_numeric(cert_ld34['최대출력_KW'], errors='coerce')
cert_ld34['최대출력_KW'].dtype

dtype('float64')

In [1303]:
cert_ld34['최대출력_KW'].isnull().sum()

5706

In [1304]:
cert_ld34['최대출력_KW'].describe()

count   265.00
mean    195.56
std      76.54
min      24.50
25%     129.00
50%     194.00
75%     250.00
max     478.00
Name: 최대출력_KW, dtype: float64

In [1305]:
div = 100
tf_col = '최대출력_KW'
tfed_col = '최대출력(KW)구간'
n = int(cert_ld34[tf_col].max() // div)
for i in range(n + 1):
    cert_ld34.loc[(cert_ld34[tf_col] >= i*div) & (cert_ld34[tf_col] < (i+1)*div), tfed_col] = f'{i*div:,}이상 ~ {(i+1)*div:,}미만'
cert_ld34[tfed_col].unique()

array(['200이상 ~ 300미만', nan, '0이상 ~ 100미만', '100이상 ~ 200미만',
       '300이상 ~ 400미만', '400이상 ~ 500미만'], dtype=object)

In [1306]:
cert_ld5.shape

(2055, 75)

In [1307]:
cert_ld5['최대출력_KW'].isnull().sum()

6

In [1308]:
cert_ld5['최대출력_KW'].dtype

dtype('O')

In [1309]:
cert_ld5['최대출력_KW'].head()

3    191
4    221
5    221
6    199
7    199
Name: 최대출력_KW, dtype: object

In [1310]:
for one in cert_ld5.loc[cert_ld5['최대출력_KW'].notnull(), '최대출력_KW']:
    try:
        one = int(one)
        if type(one) != int:
            print(one)
        elif np.isnan(one):
            pass
    except:
        pass

In [1311]:
cert_ld5['최대출력_KW'] = pd.to_numeric(cert_ld5['최대출력_KW'], errors='coerce')
cert_ld5['최대출력_KW'].dtype

dtype('float64')

In [1312]:
cert_ld5['최대출력_KW'].isnull().sum()

8

In [1313]:
cert_ld5['최대출력_KW'].describe()

count   2047.00
mean     130.20
std      103.35
min        5.90
25%       50.55
50%      104.50
75%      185.00
max      522.00
Name: 최대출력_KW, dtype: float64

In [1314]:
div = 100
tf_col = '최대출력_KW'
tfed_col = '최대출력(KW)구간'
n = int(cert_ld5[tf_col].max() // div)
for i in range(n + 1):
    cert_ld5.loc[(cert_ld5[tf_col] >= i*div) & (cert_ld5[tf_col] < (i+1)*div), tfed_col] = f'{i*div:,}이상 ~ {(i+1)*div:,}미만'
cert_ld5[tfed_col].unique()

array(['100이상 ~ 200미만', '200이상 ~ 300미만', nan, '0이상 ~ 100미만',
       '300이상 ~ 400미만', '400이상 ~ 500미만', '500이상 ~ 600미만'], dtype=object)

#### [test] tier 3~4 와 stage5 병합

In [1315]:
cert_ld34.shape[0] + cert_ld5.shape[0]

8026

In [1316]:
total_cert = pd.concat([cert_ld34, cert_ld5], ignore_index=True)
total_cert.shape

(8026, 88)

In [1317]:
len(total_cert['인증번호'].unique())

490

In [1318]:
total_cert.sort_values('구분').drop_duplicates('인증번호').shape

(490, 88)

In [1319]:
total_cert['구분'].unique()

array(['대표(최초)', '대표(정정)', '대표(보고)', '동일(보고)', '동일(최초2)', '동일(최초3)',
       '동일(최초4)', '동일(최초5)', '동일(최초6)', '동일(정정)', '동일(최초7)', '동일(최초8)',
       '동일(최초10)', '동일(최초11)', '동일(최초12)', '동일(최초13)', '동일(최초14)',
       '동일(최초15)', '동일(최초17)', '동일(최초18)', '동일(최초19)', '동일(최초20)',
       '동일(최초21)', '동일(변경)', '동일(최초1)', '동일(최초9)', '동일(최초16)', '동일(최초22)',
       '동일(최초23)', '대표(변경)', '동일(최초24)', '동일(최초25)', '동일(최초26)',
       '동일(최초27)', '동일(최초28)', '동일(최초30)', '동일(최초31)', '동일(최초29)',
       '동일(최초33)', '동일(최초34)', '동일(최초35)', '동일(최초36)', '동일(최초37)',
       '동일(최초38)', '동일(최초40)', '동일(최초41)', '동일(최초42)', '동일(최초43)',
       '동일(최초44)', '동일(최초45)', '동일(최초46)', '동일(최초47)', '동일(최초48)',
       '동일(최초49)', '동일(최초32)', '동일(최초39)', '동일(최초50)', '동일(최초51)',
       '동일(최초52)', '동일(최초53)', '동일(최초55)', '동일(최초56)', '동일(최초57)',
       '동일(최초58)', '동일(최초59)', '동일(최초60)', '동일(최초61)', '동일(최초)', nan],
      dtype=object)

In [1320]:
len(total_cert.loc[(total_cert['구분'] == '대표(최초)') | (total_cert['구분'] == '대표(정정)'), '인증번호'].unique())

410

In [1321]:
len(total_cert.loc[(total_cert['구분'] == '대표(최초)') | (total_cert['구분'] == '대표(보고)'), '인증번호'].unique())

467

In [1322]:
len(total_cert.loc[(total_cert['구분'] == '대표(최초)') | (total_cert['구분'] == '대표(변경)'), '인증번호'].unique())

420

In [1323]:
len(total_cert.loc[(total_cert['구분'] == '대표(최초)') | (total_cert['구분'] == '대표(정정)') | (total_cert['구분'] == '대표(변경)'), '인증번호'].unique())

438

In [1324]:
len(total_cert.loc[total_cert['구분'] == '대표(보고)', '인증번호'].unique())

168

In [1325]:
len(total_cert.loc[total_cert['구분'] == '대표(최초)', '인증번호'].unique())

387

## [❌] 2-1 STD_BD21_1_CTCQ_MNGNO
- 건설기계 관리번호 통계

In [334]:
tcmasterm.shape, len(tcmasterm.ADM_NO.unique())

((426, 159), 426)

In [335]:
tcmasterm.ENG_CAR_GB.unique()

array(['C', nan, 'A', 'CA'], dtype=object)

In [339]:
pr21_1 = tcmasterm.groupby(['ENG_CAR_GB', 'CERTI_STATE'], dropna=False)['ADM_NO'].count().reset_index()
pr21_1

,ENG_CAR_GB,CERTI_STATE,ADM_NO
0,A,90,110
1,A,C0,8
2,C,90,237
3,CA,90,69
4,NaN,90,1
5,NaN,NaN,1


In [340]:
pr21_1['ENG_CAR_GB'] = pr21_1['ENG_CAR_GB'].replace({'A':'농업기계', 'C':'0건설기계', 'CA':'건설/농업기계'})
pr21_1['CERTI_STATE'] = pr21_1['CERTI_STATE'].replace({'90':'인증완료'})
pr21_1 = pr21_1.rename(columns={'ENG_CAR_GB':'건설기계/농업기계 구분', 'CERTI_STATE':'처분내용', 'ADM_NO':'관리번호 개수'})
pr21_1

,건설기계/농업기계 구분,처분내용,관리번호 개수
0,농업기계,인증완료,110
1,농업기계,C0,8
2,0건설기계,인증완료,237
3,건설/농업기계,인증완료,69
4,NaN,인증완료,1
5,NaN,NaN,1


In [342]:
pr21_2 = pr21_1.sort_values(['건설기계/농업기계 구분', '처분내용', '관리번호 개수'], ascending=[True, False, False]).reset_index(drop=True)
pr21_2['건설기계/농업기계 구분'] = pr21_2['건설기계/농업기계 구분'].replace({'0건설기계':'건설기계'})
pr21_2 = pr21_2[pr21_2['처분내용'] == '인증완료'].reset_index(drop=True)
pr21_2

,건설기계/농업기계 구분,처분내용,관리번호 개수
0,건설기계,인증완료,237
1,건설/농업기계,인증완료,69
2,농업기계,인증완료,110
3,NaN,인증완료,1


In [343]:
pr21_2['관리번호 개수'].sum()

417

In [344]:
pr21_2.columns

Index(['건설기계/농업기계 구분', '처분내용', '관리번호 개수'], dtype='object')

In [345]:
today_date = datetime.today().strftime('%Y%m%d')
today_date

'20240108'

In [346]:
pr21_2['테이블생성일자'] = today_date

In [347]:
pr21_2 = pr21_2[[
    '건설기계/농업기계 구분', 
    '처분내용', 
    '관리번호 개수', 
    '테이블생성일자', 
    ]]
pr21_2.head()

,건설기계/농업기계 구분,처분내용,관리번호 개수,테이블생성일자
0,건설기계,인증완료,237,20240108
1,건설/농업기계,인증완료,69,20240108
2,농업기계,인증완료,110,20240108
3,NaN,인증완료,1,20240108


### [출력] STD_BD21_CTCQ_MNGNO

In [ ]:
today_date_dtl = datetime.today().strftime('%Y%m%d_%p_%I%M%S')
today_date_dtl

In [ ]:
# chd21 = {
#     '건설기계/농업기계 구분':'', 
#     '처분내용':'', 
#     '관리번호 개수':'', 
#     '테이블생성일자':'', 
# }
# STD_BD21_CTCQ_MNGNO = pr21_2.rename(columns=chd21)
# STD_BD21_CTCQ_MNGNO['LOAD_DT'] = datetime.today().strftime('%Y%m%d')
# print(STD_BD21_CTCQ_MNGNO.columns)

# #
# if export_opt == 'y':
#     STD_BD21_CTCQ_MNGNO.to_csv(os.path.join(df4_fold, f'STD_BD21_CTCQ_MNGNO_{today_date_dtl}.csv'), index=False)
#     print('STD_BD21_CTCQ_MNGNO 출력 완료.')
# else:
#     pass

In [348]:
if export_opt == 'y':
    pr21_2.to_csv(os.path.join(df4_fold, f'STD_BD21_CTCQ_MNGNO_{today_date_dtl}.csv'), index=False)
    pr21_2.to_excel(os.path.join(df4_fold, f'STD_BD21_CTCQ_MNGNO_{today_date_dtl}.xlsx'), index=False)
    print('STD_BD21_CTCQ_MNGNO 출력 완료.')
else:
    pass

## 2-2 STD_BD22_EXHST_GAS_CERT_NO_NOCS
- 배출인증번호 건수 통계

In [349]:
tcmasterm['EMIS_CERTI_NO'].notnull().sum()

425

In [350]:
tcmasterm.shape

(426, 159)

In [351]:
len(tcmasterm['EMIS_CERTI_NO'].unique())

426

In [411]:
# 주의 ENG_CAR_GB : C, CERTI_STATE : 90 의 EMIS_CERTI_NO중 결측 값 존재
    # 따라서 EMIS_CERTI_NO의 개수는 count로 계산(unique() 사용시 nan 값도 1개로 샘)
pr22_1 = tcmasterm.groupby(['ENG_CAR_GB', 'CERTI_STATE'], dropna=False)['EMIS_CERTI_NO'].count().reset_index()
pr22_1

,ENG_CAR_GB,CERTI_STATE,EMIS_CERTI_NO
0,A,90,110
1,A,C0,8
2,C,90,236
3,CA,90,69
4,NaN,90,1
5,NaN,NaN,1


In [412]:
pr22_1['ENG_CAR_GB'] = pr22_1['ENG_CAR_GB'].replace({'A':'농업기계', 'C':'0건설기계', 'CA':'건설/농업기계'})
pr22_1['CERTI_STATE'] = pr22_1['CERTI_STATE'].replace({'90':'인증완료'})
pr22_1 = pr22_1.rename(columns={'ENG_CAR_GB':'건설기계/농업기계 구분', 'CERTI_STATE':'처분내용', 'EMIS_CERTI_NO':'배출인증번호 개수'})
pr22_2 = pr22_1.sort_values(['건설기계/농업기계 구분', '처분내용', '배출인증번호 개수'], ascending=[True, False, False]).reset_index(drop=True)
pr22_2['건설기계/농업기계 구분'] = pr22_2['건설기계/농업기계 구분'].replace({'0건설기계':'건설기계'})
pr22_2 = pr22_2[pr22_2['처분내용'] == '인증완료'].reset_index(drop=True)
pr22_2

,건설기계/농업기계 구분,처분내용,배출인증번호 개수
0,건설기계,인증완료,236
1,건설/농업기계,인증완료,69
2,농업기계,인증완료,110
3,NaN,인증완료,1


In [413]:
pr22_2['배출인증번호 개수'].sum()

416

In [414]:
pr22_2['테이블생성일자'] = today_date

In [415]:
pr22_2 = pr22_2[[
    '건설기계/농업기계 구분', 
    '처분내용', 
    '배출인증번호 개수', 
    '테이블생성일자', 
    ]]
# chd22_2 = {
#     '건설기계/농업기계 구분':'', 
#     '처분내용':'', 
#     '배출인증번호 개수':'', 
#     '테이블생성일자':'', 
# }
# pr22_2 = pr22_2.rename(columns=chd22_2)
pr22_2.head()

,건설기계/농업기계 구분,처분내용,배출인증번호 개수,테이블생성일자
0,건설기계,인증완료,236,20240108
1,건설/농업기계,인증완료,69,20240108
2,농업기계,인증완료,110,20240108
3,NaN,인증완료,1,20240108


### [출력] STD_BD22_EXHST_GAS_CERT_NO_NOCS

In [416]:
today_date = datetime.today().strftime('%Y%m%d')
today_date

'20240108'

In [417]:
STD_BD22_EXHST_GAS_CERT_NO_NOCS = pr22_2.copy()
STD_BD22_EXHST_GAS_CERT_NO_NOCS.to_csv(os.path.join(df2_fold, f'STD_BD22_EXHST_GAS_CERT_NO_NOCS_{today_date}.csv'), index=False)
STD_BD22_EXHST_GAS_CERT_NO_NOCS.to_excel(os.path.join(df2_fold, f'STD_BD22_EXHST_GAS_CERT_NO_NOCS_{today_date}.xlsx'), index=False)

## [❌] 2-3 STD_BD23_FUEL_CERT_NOCS
- 연료별 인증건수 통계

In [418]:
tcmasterm['TEST_FUEL'].value_counts(dropna=False)

TEST_FUEL
NaN    426
Name: count, dtype: int64

In [419]:
'TEST_FUEL' in tcmasterm.columns

True

In [425]:
pr23_1 = tcmasterm.groupby(['ENG_CAR_GB', 'CERTI_STATE', 'TEST_FUEL'], dropna=False)['EMIS_CERTI_NO'].count().reset_index()
pr23_1

,ENG_CAR_GB,CERTI_STATE,TEST_FUEL,EMIS_CERTI_NO
0,A,90,NaN,110
1,A,C0,NaN,8
2,C,90,NaN,236
3,CA,90,NaN,69
4,NaN,90,NaN,1
5,NaN,NaN,NaN,1


In [426]:
pr23_1['ENG_CAR_GB'] = pr23_1['ENG_CAR_GB'].replace({'A':'농업기계', 'C':'0건설기계', 'CA':'건설/농업기계'})
pr23_1['CERTI_STATE'] = pr23_1['CERTI_STATE'].replace({'90':'인증완료'})
pr23_1 = pr23_1.rename(columns={'ENG_CAR_GB':'건설기계/농업기계 구분', 'CERTI_STATE':'처분내용', 'EMIS_CERTI_NO':'배출인증번호 개수', 'TEST_FUEL':'시험연료'})
pr23_2 = pr23_1.sort_values(['건설기계/농업기계 구분', '처분내용', '배출인증번호 개수'], ascending=[True, False, False]).reset_index(drop=True)
pr23_2['건설기계/농업기계 구분'] = pr23_2['건설기계/농업기계 구분'].replace({'0건설기계':'건설기계'})
pr23_2 = pr23_2[pr23_2['처분내용'] == '인증완료'].reset_index(drop=True)
pr23_2

,건설기계/농업기계 구분,처분내용,시험연료,배출인증번호 개수
0,건설기계,인증완료,NaN,236
1,건설/농업기계,인증완료,NaN,69
2,농업기계,인증완료,NaN,110
3,NaN,인증완료,NaN,1


In [427]:
pr23_2['배출인증번호 개수'].sum()

416

In [428]:
pr23_2['테이블생성일자'] = today_date

In [429]:
pr23_2 = pr23_2[[
    '건설기계/농업기계 구분', 
    '처분내용', 
    '시험연료', 
    '배출인증번호 개수', 
    '테이블생성일자', 
    ]]
# chd23_2 = {
#     '건설기계/농업기계 구분':'', 
#     '처분내용':'', 
#     '시험연료':'', 
#     '배출인증번호 개수':'', 
#     '테이블생성일자':'', 
# }
# pr23_2 = pr23_2.rename(columns=chd23_2)
pr23_2.head()

,건설기계/농업기계 구분,처분내용,시험연료,배출인증번호 개수,테이블생성일자
0,건설기계,인증완료,NaN,236,20240108
1,건설/농업기계,인증완료,NaN,69,20240108
2,농업기계,인증완료,NaN,110,20240108
3,NaN,인증완료,NaN,1,20240108


### [출력] STD_BD23_FUEL_CERT_NOCS

In [430]:
# STD_BD23_FUEL_CERT_NOCS = pr23_2.copy()
# STD_BD23_FUEL_CERT_NOCS.to_csv(os.path.join(df2_fold, f'STD_BD23_FUEL_CERT_NOCS_{today_date}.csv'), index=False)
# STD_BD23_FUEL_CERT_NOCS.to_excel(os.path.join(df2_fold, f'STD_BD23_FUEL_CERT_NOCS_{today_date}.xlsx'), index=False)

## 2-4 STD_BD24_OTPT_CERT_NOCS
- 출력별 인증건수 통계

In [431]:
eng_tefspec['RATED_OUT'].value_counts(dropna=False)

RATED_OUT
129     179
168     141
18.2    118
265     110
151     108
       ... 
67.3      1
62.2      1
69.2      1
71.0      1
332       1
Name: count, Length: 821, dtype: int64

In [432]:
eng_tefspec.shape, len(eng_tefspec['ADM_NO'].unique())

((8646, 108), 472)

In [433]:
eng_tefspec.loc[eng_tefspec['EF_SNO'].duplicated(), ['ADM_NO', 'EF_SNO', 'RATED_OUT', 'ENG_DISPL']].sort_values(['ADM_NO', 'EF_SNO'])

,ADM_NO,EF_SNO,RATED_OUT,ENG_DISPL
5021,52346,0,405,12742.00
5018,52346,1,264,12742.00
5019,52346,2,283,12742.00
5020,52346,3,202,9291.00
5023,52346,7,331,12742.00
...,...,...,...,...
7871,59500,3,55.4,2799.00
8314,59571,0,18.4,1267.00
8315,59571,1,18.4,1267.00
7903,59573,0,268.0,15239.00


In [434]:
eng_tefspec.drop_duplicates(['ADM_NO']).shape

(472, 108)

In [436]:
eng_tefspecm['RATED_OUT'].describe()

count     472
unique    288
top        55
freq       18
Name: RATED_OUT, dtype: object

In [437]:
eng_tefspecm['RATED_OUT'].dtype

dtype('O')

In [438]:
eng_tefspecm.loc[eng_tefspecm['RATED_OUT'] == '42.4 (43.7)', 'RATED_OUT'] = '42.4'

In [439]:
eng_tefspecm['RATED_OUT'] = eng_tefspecm['RATED_OUT'].astype('float')

In [440]:
# 5.9 ~ 552
    # 100 단위
eng_tefspecm['RATED_OUT'].describe()

count   472.00
mean    115.67
std     113.84
min       9.80
25%      41.00
50%      63.45
75%     150.02
max     552.00
Name: RATED_OUT, dtype: float64

In [441]:
eng_tefspecm['RATED_OUT'].max()

552.0

In [442]:
div = 100
n = int(eng_tefspecm['RATED_OUT'].max() // div)
for i in range(n + 1):
    eng_tefspecm.loc[(eng_tefspecm['RATED_OUT'] >= i*div) & (eng_tefspecm['RATED_OUT'] < (i+1)*div), '정격출력(kW)구분'] = f'{i*div}이상 ~ {(i+1)*div}미만'
eng_tefspecm['정격출력(kW)구분'].unique()

array(['0이상 ~ 100미만', '100이상 ~ 200미만', '400이상 ~ 500미만', '200이상 ~ 300미만',
       '300이상 ~ 400미만', '500이상 ~ 600미만'], dtype=object)

In [443]:
tcmasterm.shape

(426, 159)

In [444]:
tcmasterm.columns

Index(['ADM_NO', 'FACT_SNO', 'CERTI_DIV', 'REG_DATETIME', 'RECT_DATE',
       'EMIS_CERTI_NO', 'NOISE_CERTI_NO', 'EMIS_OMIT_NO', 'NOISE_OMIT_NO',
       'EMIS_CERTI_DATE',
       ...
       'SNOTE2', 'EMIS_CANCEL_DATE', 'NOISE_CANCEL_DATE', 'REG_ID', 'MOD_ID',
       'MOD_DATETIME', 'ENG_CAR_GB', 'EMIS_NOTE2', 'EMIS_STD_NOTE',
       'NOISE_STD_NOTE'],
      dtype='object', length=159)

In [445]:
'ENG_CAR_GB' in eng_tefspecm.columns

True

In [446]:
pr24_1 = tcmasterm[['ADM_NO', 'ENG_CAR_GB', 'CERTI_STATE', 'EMIS_CERTI_NO']].merge(eng_tefspecm[['ADM_NO', '정격출력(kW)구분']], on='ADM_NO', how='left')
pr24_1
pr24_1.shape

(426, 5)

In [447]:
len(pr24_1['EMIS_CERTI_NO'].unique())

426

In [448]:
pr24_2 = pr24_1.groupby(['ENG_CAR_GB', 'CERTI_STATE', '정격출력(kW)구분'], dropna=False)['EMIS_CERTI_NO'].count().reset_index()
pr24_2

,ENG_CAR_GB,CERTI_STATE,정격출력(kW)구분,EMIS_CERTI_NO
0,A,90,0이상 ~ 100미만,84
1,A,90,100이상 ~ 200미만,23
2,A,90,200이상 ~ 300미만,3
3,A,C0,0이상 ~ 100미만,8
4,C,90,0이상 ~ 100미만,117
5,C,90,100이상 ~ 200미만,45
6,C,90,200이상 ~ 300미만,36
7,C,90,300이상 ~ 400미만,20
8,C,90,400이상 ~ 500미만,13
9,C,90,500이상 ~ 600미만,5


In [449]:
pr24_2['ENG_CAR_GB'] = pr24_2['ENG_CAR_GB'].replace({'A':'농업기계', 'C':'0건설기계', 'CA':'건설/농업기계'})
pr24_2['CERTI_STATE'] = pr24_2['CERTI_STATE'].replace({'90':'인증완료'})
pr24_2 = pr24_2.rename(columns={'ENG_CAR_GB':'건설기계/농업기계 구분', 'CERTI_STATE':'처분내용', 'EMIS_CERTI_NO':'배출인증번호 개수'})
pr24_3 = pr24_2.sort_values(['처분내용', '건설기계/농업기계 구분', '정격출력(kW)구분'], ascending=[False, True, True]).reset_index(drop=True)
pr24_3['건설기계/농업기계 구분'] = pr24_3['건설기계/농업기계 구분'].replace({'0건설기계':'건설기계'})
pr24_3 = pr24_3[pr24_3['처분내용'] == '인증완료'].reset_index(drop=True)
pr24_3

,건설기계/농업기계 구분,처분내용,정격출력(kW)구분,배출인증번호 개수
0,건설기계,인증완료,0이상 ~ 100미만,117
1,건설기계,인증완료,100이상 ~ 200미만,45
2,건설기계,인증완료,200이상 ~ 300미만,36
3,건설기계,인증완료,300이상 ~ 400미만,20
4,건설기계,인증완료,400이상 ~ 500미만,13
5,건설기계,인증완료,500이상 ~ 600미만,5
6,건설/농업기계,인증완료,0이상 ~ 100미만,55
7,건설/농업기계,인증완료,100이상 ~ 200미만,12
8,건설/농업기계,인증완료,200이상 ~ 300미만,1
9,건설/농업기계,인증완료,400이상 ~ 500미만,1


In [450]:
pr24_3['배출인증번호 개수'].sum()

416

In [451]:
pr24_3['테이블생성일자'] = today_date

In [452]:
pr24_3 = pr24_3[[
    '건설기계/농업기계 구분', 
    '처분내용', 
    '정격출력(kW)구분', 
    '배출인증번호 개수', 
    '테이블생성일자', 
    ]]
# chd24_3 = {
#     '건설기계/농업기계 구분':'', 
#     '처분내용':'', 
#     '정격출력(kW)구분':'', 
#     '배출인증번호 개수':'', 
#     '테이블생성일자':'', 
# }
# pr24_3 = pr24_3.rename(columns=chd24_3)
pr24_3.head()

,건설기계/농업기계 구분,처분내용,정격출력(kW)구분,배출인증번호 개수,테이블생성일자
0,건설기계,인증완료,0이상 ~ 100미만,117,20240108
1,건설기계,인증완료,100이상 ~ 200미만,45,20240108
2,건설기계,인증완료,200이상 ~ 300미만,36,20240108
3,건설기계,인증완료,300이상 ~ 400미만,20,20240108
4,건설기계,인증완료,400이상 ~ 500미만,13,20240108


### [출력] STD_BD24_OTPT_CERT_NOCS

In [453]:
STD_BD24_OTPT_CERT_NOCS = pr24_3.copy()
STD_BD24_OTPT_CERT_NOCS.to_csv(os.path.join(df2_fold, f'STD_BD24_OTPT_CERT_NOCS_{today_date}.csv'), index=False)
STD_BD24_OTPT_CERT_NOCS.to_excel(os.path.join(df2_fold, f'STD_BD24_OTPT_CERT_NOCS_{today_date}.xlsx'), index=False)

## 2-5 STD_BD25_DSPLVL_CERT_NOCS
- 배기량별 인증건수 통계

In [471]:
# 479 ~ 23,152
eng_tefspecm['ENG_DISPL'].describe()

count     472.00
mean     4994.30
std      4084.75
min       479.00
25%      2286.75
50%      3398.00
75%      6692.50
max     23152.00
Name: ENG_DISPL, dtype: float64

In [472]:
div = 2000
tf_col = 'ENG_DISPL'
tfed_col = '배기량(cc)구분'
n = int(eng_tefspecm[tf_col].max() // div)
for i in range(n + 1):
    eng_tefspecm.loc[(eng_tefspecm[tf_col] >= i*div) & (eng_tefspecm[tf_col] < (i+1)*div), tfed_col] = f'{i*div:,}이상 ~ {(i+1)*div:,}미만'
eng_tefspecm[tfed_col].unique()

array(['0이상 ~ 2,000미만', '4,000이상 ~ 6,000미만', '14,000이상 ~ 16,000미만',
       '6,000이상 ~ 8,000미만', '8,000이상 ~ 10,000미만', '2,000이상 ~ 4,000미만',
       '10,000이상 ~ 12,000미만', '16,000이상 ~ 18,000미만',
       '12,000이상 ~ 14,000미만', '18,000이상 ~ 20,000미만',
       '22,000이상 ~ 24,000미만'], dtype=object)

In [473]:
pr25_1 = tcmasterm[['ADM_NO', 'ENG_CAR_GB', 'CERTI_STATE', 'EMIS_CERTI_NO']].merge(eng_tefspecm[['ADM_NO', tfed_col]], on='ADM_NO', how='left')
pr25_1.shape

(426, 5)

In [483]:
pr25_2 = pr25_1.groupby(['ENG_CAR_GB', 'CERTI_STATE', tfed_col], dropna=False)['EMIS_CERTI_NO'].count().reset_index()
pr25_2

,ENG_CAR_GB,CERTI_STATE,배기량(cc)구분,EMIS_CERTI_NO
0,A,90,"0이상 ~ 2,000미만",28
1,A,90,"2,000이상 ~ 4,000미만",47
2,A,90,"4,000이상 ~ 6,000미만",11
3,A,90,"6,000이상 ~ 8,000미만",24
4,A,C0,"0이상 ~ 2,000미만",3
5,A,C0,"2,000이상 ~ 4,000미만",5
6,C,90,"0이상 ~ 2,000미만",30
7,C,90,"10,000이상 ~ 12,000미만",15
8,C,90,"12,000이상 ~ 14,000미만",20
9,C,90,"14,000이상 ~ 16,000미만",11


In [484]:
pr25_2['ENG_CAR_GB'] = pr25_2['ENG_CAR_GB'].replace({'A':'농업기계', 'C':'0건설기계', 'CA':'건설/농업기계'})
pr25_2['CERTI_STATE'] = pr25_2['CERTI_STATE'].replace({'90':'인증완료'})
pr25_2 = pr25_2.rename(columns={'ENG_CAR_GB':'건설기계/농업기계 구분', 'CERTI_STATE':'처분내용', 'EMIS_CERTI_NO':'배출인증번호 개수'})
pr25_3 = pr25_2.sort_values(['처분내용', '건설기계/농업기계 구분', tfed_col], ascending=[False, True, True]).reset_index(drop=True)
pr25_3['건설기계/농업기계 구분'] = pr25_3['건설기계/농업기계 구분'].replace({'0건설기계':'건설기계'})
pr25_3 = pr25_3[pr25_3['처분내용'] == '인증완료'].reset_index(drop=True)
pr25_3

,건설기계/농업기계 구분,처분내용,배기량(cc)구분,배출인증번호 개수
0,건설기계,인증완료,"0이상 ~ 2,000미만",30
1,건설기계,인증완료,"10,000이상 ~ 12,000미만",15
2,건설기계,인증완료,"12,000이상 ~ 14,000미만",20
3,건설기계,인증완료,"14,000이상 ~ 16,000미만",11
4,건설기계,인증완료,"16,000이상 ~ 18,000미만",7
5,건설기계,인증완료,"18,000이상 ~ 20,000미만",3
6,건설기계,인증완료,"2,000이상 ~ 4,000미만",80
7,건설기계,인증완료,"22,000이상 ~ 24,000미만",1
8,건설기계,인증완료,"4,000이상 ~ 6,000미만",28
9,건설기계,인증완료,"6,000이상 ~ 8,000미만",27


In [485]:
pr25_3['배출인증번호 개수'].sum()

416

In [486]:
pr25_3['테이블생성일자'] = today_date

In [489]:
pr25_3 = pr25_3[[
    '건설기계/농업기계 구분', 
    '처분내용', 
    '배기량(cc)구분', 
    '배출인증번호 개수', 
    '테이블생성일자', 
    ]]
# chd25_3 = {
#     '건설기계/농업기계 구분':'', 
#     '처분내용':'', 
#     '시험연료':'', 
#     '배기량(cc)구분':'', 
#     '테이블생성일자':'', 
# }
# pr25_3 = pr25_3.rename(columns=chd25_3)
pr25_3.head()

,건설기계/농업기계 구분,처분내용,배기량(cc)구분,배출인증번호 개수,테이블생성일자
0,건설기계,인증완료,"0이상 ~ 2,000미만",30,20240108
1,건설기계,인증완료,"10,000이상 ~ 12,000미만",15,20240108
2,건설기계,인증완료,"12,000이상 ~ 14,000미만",20,20240108
3,건설기계,인증완료,"14,000이상 ~ 16,000미만",11,20240108
4,건설기계,인증완료,"16,000이상 ~ 18,000미만",7,20240108


### [출력] STD_BD25_DSPLVL_CERT_NOCS

In [490]:
STD_BD25_DSPLVL_CERT_NOCS = pr25_3.copy()
STD_BD25_DSPLVL_CERT_NOCS.to_csv(os.path.join(df2_fold, f'STD_BD25_DSPLVL_CERT_NOCS_{today_date}.csv'), index=False)
STD_BD25_DSPLVL_CERT_NOCS.to_excel(os.path.join(df2_fold, f'STD_BD25_DSPLVL_CERT_NOCS_{today_date}.xlsx'), index=False)

## 2-6 STD_BD26_NTN_CERT_NOCS
- 국내외별 인증건수 통계

In [491]:
pr26_1 = tcmaster.groupby(['ENG_CAR_GB', 'CERTI_STATE', 'CERTI_DIV'], dropna=False)['EMIS_CERTI_NO'].count().reset_index()
pr26_1

,ENG_CAR_GB,CERTI_STATE,CERTI_DIV,EMIS_CERTI_NO
0,A,42,1,0
1,A,90,1,41
2,A,90,2,69
3,A,C0,1,17
4,A,NaN,2,0
5,C,10,1,0
6,C,41,2,0
7,C,42,2,0
8,C,46,1,0
9,C,90,1,41


In [492]:
pr26_1['ENG_CAR_GB'] = pr26_1['ENG_CAR_GB'].replace({'A':'농업기계', 'C':'0건설기계', 'CA':'건설/농업기계'})
pr26_1['CERTI_STATE'] = pr26_1['CERTI_STATE'].replace({'90':'인증완료'})
pr26_1['CERTI_DIV'] = pr26_1['CERTI_DIV'].replace({1:'국내', 2:'수입'})
pr26_1 = pr26_1.rename(columns={'ENG_CAR_GB':'건설기계/농업기계 구분', 'CERTI_STATE':'처분내용', 'EMIS_CERTI_NO':'배출인증번호 개수', 'CERTI_DIV':'인증구분'})
pr26_2 = pr26_1.sort_values(['처분내용', '건설기계/농업기계 구분', '배출인증번호 개수'], ascending=[False, True, False]).reset_index(drop=True)
pr26_2['건설기계/농업기계 구분'] = pr26_2['건설기계/농업기계 구분'].replace({'0건설기계':'건설기계'})
pr26_2 = pr26_2[pr26_2['처분내용'] == '인증완료'].reset_index(drop=True)
pr26_2

,건설기계/농업기계 구분,처분내용,인증구분,배출인증번호 개수
0,건설기계,인증완료,수입,195
1,건설기계,인증완료,국내,41
2,건설/농업기계,인증완료,수입,56
3,건설/농업기계,인증완료,국내,13
4,농업기계,인증완료,수입,69
5,농업기계,인증완료,국내,41
6,NaN,인증완료,국내,1


In [493]:
pr26_2['배출인증번호 개수'].sum()

416

In [494]:
pr26_2['테이블생성일자'] = today_date

In [495]:
pr26_2 = pr26_2[[
    '건설기계/농업기계 구분', 
    '처분내용', 
    '인증구분', 
    '배출인증번호 개수', 
    '테이블생성일자', 
    ]]
# chd26_2 = {
#     '건설기계/농업기계 구분':'', 
#     '처분내용':'', 
#     '인증구분':'', 
#     '배출인증번호 개수':'', 
#     '테이블생성일자':'', 
# }
# pr26_2 = pr26_2.rename(columns=chd26_2)
pr26_2.head()

,건설기계/농업기계 구분,처분내용,인증구분,배출인증번호 개수,테이블생성일자
0,건설기계,인증완료,수입,195,20240108
1,건설기계,인증완료,국내,41,20240108
2,건설/농업기계,인증완료,수입,56,20240108
3,건설/농업기계,인증완료,국내,13,20240108
4,농업기계,인증완료,수입,69,20240108


### [출력] STD_BD26_NTN_CERT_NOCS

In [496]:
STD_BD26_NTN_CERT_NOCS = pr26_2.copy()
STD_BD26_NTN_CERT_NOCS.to_csv(os.path.join(df2_fold, f'STD_BD26_NTN_CERT_NOCS_{today_date}.csv'), index=False)
STD_BD26_NTN_CERT_NOCS.to_excel(os.path.join(df2_fold, f'STD_BD26_NTN_CERT_NOCS_{today_date}.xlsx'), index=False)

## 2-7 STD_BD27_FBCTN_NTN_CERT_NOCS
- 제작국별 인증건수 통계

In [501]:
pr27_1 = tcmaster.groupby(['ENG_CAR_GB', 'CERTI_STATE', 'NATION'], dropna=False)['EMIS_CERTI_NO'].count().reset_index()
pr27_1

,ENG_CAR_GB,CERTI_STATE,NATION,EMIS_CERTI_NO
0,A,42,KR,0
1,A,90,FN,6
2,A,90,FR,8
3,A,90,GM,15
4,A,90,IT,12
5,A,90,JP,26
6,A,90,KR,41
7,A,90,US,2
8,A,C0,KR,17
9,A,NaN,FN,0


In [502]:
pr27_1['ENG_CAR_GB'] = pr27_1['ENG_CAR_GB'].replace({'A':'농업기계', 'C':'0건설기계', 'CA':'건설/농업기계'})
pr27_1['CERTI_STATE'] = pr27_1['CERTI_STATE'].replace({'90':'인증완료'})
pr27_1 = pr27_1.rename(columns={'ENG_CAR_GB':'건설기계/농업기계 구분', 'CERTI_STATE':'처분내용', 'EMIS_CERTI_NO':'배출인증번호 개수', 'NATION':'제작국가'})
pr27_2 = pr27_1.sort_values(['처분내용', '건설기계/농업기계 구분', '배출인증번호 개수'], ascending=[False, True, False]).reset_index(drop=True)
pr27_2['건설기계/농업기계 구분'] = pr27_2['건설기계/농업기계 구분'].replace({'0건설기계':'건설기계'})
pr27_2 = pr27_2[pr27_2['처분내용'] == '인증완료'].reset_index(drop=True)
pr27_2

,건설기계/농업기계 구분,처분내용,제작국가,배출인증번호 개수
0,건설기계,인증완료,JP,75
1,건설기계,인증완료,KR,41
2,건설기계,인증완료,GM,28
3,건설기계,인증완료,US,28
4,건설기계,인증완료,UK,24
5,건설기계,인증완료,SW,20
6,건설기계,인증완료,CN,8
7,건설기계,인증완료,IT,8
8,건설기계,인증완료,SU,4
9,건설/농업기계,인증완료,JP,37


In [503]:
pr27_2['배출인증번호 개수'].sum()

416

In [780]:
pr27_2['제작국가'].unique()

array(['JP', 'KR', 'GM', 'US', 'UK', 'SW', 'CN', 'IT', 'SU', 'FR', 'ID',
       'FN'], dtype=object)

In [504]:
pr27_2['테이블생성일자'] = today_date

In [505]:
pr27_2 = pr27_2[[
    '건설기계/농업기계 구분', 
    '처분내용', 
    '제작국가', 
    '배출인증번호 개수', 
    '테이블생성일자', 
    ]]
# chd27_2 = {
#     '건설기계/농업기계 구분':'', 
#     '처분내용':'', 
#     '제작국가':'', 
#     '배출인증번호 개수':'', 
#     '테이블생성일자':'', 
# }
# pr27_2 = pr27_2.rename(columns=chd27_2)
pr27_2.head()

,건설기계/농업기계 구분,처분내용,제작국가,배출인증번호 개수,테이블생성일자
0,건설기계,인증완료,JP,75,20240108
1,건설기계,인증완료,KR,41,20240108
2,건설기계,인증완료,GM,28,20240108
3,건설기계,인증완료,US,28,20240108
4,건설기계,인증완료,UK,24,20240108


### [출력] STD_BD27_FBCTN_NTN_CERT_NOCS

In [506]:
STD_BD27_FBCTN_NTN_CERT_NOCS = pr27_2.copy()
STD_BD27_FBCTN_NTN_CERT_NOCS.to_csv(os.path.join(df2_fold, f'STD_BD27_FBCTN_NTN_CERT_NOCS_{today_date}.csv'), index=False)
STD_BD27_FBCTN_NTN_CERT_NOCS.to_excel(os.path.join(df2_fold, f'STD_BD27_FBCTN_NTN_CERT_NOCS_{today_date}.xlsx'), index=False)

## 2-8 STD_BD28_CERT_YR_CERT_NOCS
- 인증연도별 인증 건수 통계

In [507]:
pr28_1 = tcmaster.copy()

In [508]:
pr28_1['REG_DATETIME'].dtype

dtype('O')

In [509]:
pr28_1['REG_DATETIME'].head()

0    19/01/24 08:47:12.000000000
1    19/01/31 16:26:07.000000000
2    19/04/19 14:16:55.000000000
3    19/04/19 14:22:44.000000000
4    19/04/19 14:46:46.000000000
Name: REG_DATETIME, dtype: object

In [510]:
pr28_1['REG_DATETIME'] = pr28_1['REG_DATETIME'].str.split('.').str[0]
pr28_1['REG_DATETIME'].head()

0    19/01/24 08:47:12
1    19/01/31 16:26:07
2    19/04/19 14:16:55
3    19/04/19 14:22:44
4    19/04/19 14:46:46
Name: REG_DATETIME, dtype: object

In [511]:
pr28_1['REG_DATETIME'].isnull().sum()

0

In [512]:
pr28_1['REG_DATETIME'] = pd.to_datetime(pr28_1['REG_DATETIME'], yearfirst=True)
pr28_1['REG_DATETIME'].head()

0   2019-01-24 08:47:12
1   2019-01-31 16:26:07
2   2019-04-19 14:16:55
3   2019-04-19 14:22:44
4   2019-04-19 14:46:46
Name: REG_DATETIME, dtype: datetime64[ns]

In [513]:
pr28_1['REG_DATETIME_year'] = pr28_1['REG_DATETIME'].dt.year
pr28_1[['REG_DATETIME', 'REG_DATETIME_year']].head()

,REG_DATETIME,REG_DATETIME_year
0,2019-01-24 08:47:12,2019
1,2019-01-31 16:26:07,2019
2,2019-04-19 14:16:55,2019
3,2019-04-19 14:22:44,2019
4,2019-04-19 14:46:46,2019


In [514]:
pr28_2 = pr28_1.groupby(['ENG_CAR_GB', 'CERTI_STATE', 'REG_DATETIME_year'], dropna=False)['EMIS_CERTI_NO'].count().reset_index()
pr28_2

,ENG_CAR_GB,CERTI_STATE,REG_DATETIME_year,EMIS_CERTI_NO
0,A,42,2021,0
1,A,90,2014,52
2,A,90,2015,23
3,A,90,2016,12
4,A,90,2017,2
5,A,90,2018,5
6,A,90,2019,3
7,A,90,2020,1
8,A,90,2021,1
9,A,90,2022,9


In [515]:
pr28_2['ENG_CAR_GB'] = pr28_2['ENG_CAR_GB'].replace({'A':'농업기계', 'C':'0건설기계', 'CA':'건설/농업기계'})
pr28_2['CERTI_STATE'] = pr28_2['CERTI_STATE'].replace({'90':'인증완료'})
pr28_2 = pr28_2.rename(columns={'ENG_CAR_GB':'건설기계/농업기계 구분', 'CERTI_STATE':'처분내용', 'EMIS_CERTI_NO':'배출인증번호 개수', 'REG_DATETIME_year':'인증연도'})
pr28_3 = pr28_2.sort_values(['처분내용', '건설기계/농업기계 구분', '인증연도', '배출인증번호 개수'], ascending=[False, True, True, False]).reset_index(drop=True)
pr28_3['건설기계/농업기계 구분'] = pr28_3['건설기계/농업기계 구분'].replace({'0건설기계':'건설기계'})
pr28_3 = pr28_3[pr28_3['처분내용'] == '인증완료'].reset_index(drop=True)
pr28_3

,건설기계/농업기계 구분,처분내용,인증연도,배출인증번호 개수
0,건설기계,인증완료,2014,83
1,건설기계,인증완료,2015,49
2,건설기계,인증완료,2016,24
3,건설기계,인증완료,2017,9
4,건설기계,인증완료,2018,9
5,건설기계,인증완료,2019,17
6,건설기계,인증완료,2020,7
7,건설기계,인증완료,2021,13
8,건설기계,인증완료,2022,24
9,건설기계,인증완료,2023,1


In [516]:
pr28_3['배출인증번호 개수'].sum()

416

In [781]:
pr28_3['인증연도'].unique()

array([2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023])

In [517]:
pr28_3['테이블생성일자'] = today_date

In [518]:
pr28_3 = pr28_3[[
    '건설기계/농업기계 구분', 
    '처분내용', 
    '인증연도', 
    '배출인증번호 개수', 
    '테이블생성일자', 
    ]]
# chd27_3 = {
#     '건설기계/농업기계 구분':'', 
#     '처분내용':'', 
#     '인증연도':'', 
#     '배출인증번호 개수':'', 
#     '테이블생성일자':'', 
# }
# pr28_3 = pr28_3.rename(columns=chd27_3)
pr28_3.head()

,건설기계/농업기계 구분,처분내용,인증연도,배출인증번호 개수,테이블생성일자
0,건설기계,인증완료,2014,83,20240108
1,건설기계,인증완료,2015,49,20240108
2,건설기계,인증완료,2016,24,20240108
3,건설기계,인증완료,2017,9,20240108
4,건설기계,인증완료,2018,9,20240108


### [출력] STD_BD28_CERT_YR_CERT_NOCS

In [519]:
STD_BD28_CERT_YR_CERT_NOCS = pr28_3.copy()
STD_BD28_CERT_YR_CERT_NOCS.to_csv(os.path.join(df2_fold, f'STD_BD28_CERT_YR_CERT_NOCS_{today_date}.csv'), index=False)
STD_BD28_CERT_YR_CERT_NOCS.to_excel(os.path.join(df2_fold, f'STD_BD28_CERT_YR_CERT_NOCS_{today_date}.xlsx'), index=False)

## [❌] 2-9 STD_BD29_EXHST_GAS_PRMT_CRTR_MEVLU
- 제작사별 배출가스 측정치 통계

In [208]:
teemissn_eng_res.shape, len(teemissn_eng_res['ADM_NO'].unique())

((2761, 101), 487)

In [209]:
tcmaster.shape

(624, 159)

In [210]:
pr29_1 = tcmaster[['ADM_NO', 'ENG_CAR_GB', 'CERTI_STATE', 'EMIS_CERTI_NO']].merge(teemissn_eng_res, on='ADM_NO', how='left')
pr29_1.shape

(2710, 104)

In [211]:
pr29_2 = pr29_1.merge(eng_tefspec[['ADM_NO', 'ENG_MAKER']], on='ADM_NO', how='left')
pr29_2.shape

(55785, 105)

In [212]:
pr29_3 = pr29_2.groupby(['ENG_CAR_GB', 'CERTI_STATE', 'ENG_MAKER', 'EMIS_CERTI_NO'], dropna=False).agg({'CO_FINAL_RST':'mean', 'NMHC_FINAL_RST':'mean', 'NOX_FINAL_RST':'mean', 'PM_FINAL_RST':'mean', 'PN_FINAL_RST':'mean', 'NH3_FINAL_RST':'mean', 'NMHCNOX_FINAL_RST':'mean'}).reset_index()
pr29_3

,ENG_CAR_GB,CERTI_STATE,ENG_MAKER,EMIS_CERTI_NO,CO_FINAL_RST,NMHC_FINAL_RST,NOX_FINAL_RST,PM_FINAL_RST,PN_FINAL_RST,NH3_FINAL_RST,NMHCNOX_FINAL_RST
0,A,42,국제종합기계,NaN,0.05,NaN,NaN,0.01,6.56,NaN,4.09
1,A,90,(주)대동,22EN*DD*02,0.15,0.04,3.49,0.00,153000000000.00,NaN,3.53
2,A,90,AGCO POWER,22EN*AP*01,0.01,0.01,0.33,0.01,710000000000.00,NaN,NaN
3,A,90,AGCO POWER,23EN*AP*01,0.01,0.02,0.24,0.00,718800000000.00,NaN,NaN
4,A,90,Agco Sisu Power,14EN*AP*01,0.07,NaN,NaN,0.01,NaN,NaN,1.54
...,...,...,...,...,...,...,...,...,...,...,...
515,CA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
516,NaN,90,제작사1,14EN*SS*01,1.50,NaN,NaN,0.10,NaN,NaN,1.10
517,NaN,NaN,CATERPILLAR,09EN*CA*02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
518,NaN,NaN,Caterpillar,09EN*CA*02,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [213]:
pr29_3['ENG_CAR_GB'] = pr29_3['ENG_CAR_GB'].replace({'A':'농업기계', 'C':'0건설기계', 'CA':'건설/농업기계'})
pr29_3['CERTI_STATE'] = pr29_3['CERTI_STATE'].replace({'90':'인증완료'})
pr29_3 = pr29_3.rename(columns={'ENG_CAR_GB':'건설기계/농업기계 구분', 'CERTI_STATE':'처분내용', 'EMIS_CERTI_NO':'배출인증번호', 'ENG_MAKER':'제작사', 'CO_FINAL_RST':'CO_FINAL_RST_평균', 'NMHC_FINAL_RST':'NMHC_FINAL_RST_평균', 'NOX_FINAL_RST':'NOX_FINAL_RST_평균', 'PM_FINAL_RST':'PM_FINAL_RST_평균', 'PN_FINAL_RST':'PN_FINAL_RST_평균', 'NH3_FINAL_RST':'NH3_FINAL_RST_평균', 'NMHCNOX_FINAL_RST':'NMHCNOX_FINAL_RST_평균'})
pr29_4 = pr29_3.sort_values(['처분내용', '건설기계/농업기계 구분', '제작사'], ascending=[False, True, True]).reset_index(drop=True)
pr29_4['건설기계/농업기계 구분'] = pr29_4['건설기계/농업기계 구분'].replace({'0건설기계':'건설기계'})
pr29_4

,건설기계/농업기계 구분,처분내용,제작사,배출인증번호,CO_FINAL_RST_평균,NMHC_FINAL_RST_평균,NOX_FINAL_RST_평균,PM_FINAL_RST_평균,PN_FINAL_RST_평균,NH3_FINAL_RST_평균,NMHCNOX_FINAL_RST_평균
0,건설기계,인증완료,(주)대동,21EN*DD*02,0.01,0.03,3.39,0.01,34400000000.00,NaN,3.42
1,건설기계,인증완료,(주)코마츠제작소,09EN*KO*01,0.90,NaN,NaN,0.10,NaN,NaN,3.51
2,건설기계,인증완료,B4.5,19EN*CU*05,0.07,0.04,0.12,0.01,142689000000000.00,0.00,NaN
3,건설기계,인증완료,CATERPILLAR,09EN*CA*01,2.38,NaN,NaN,0.12,NaN,NaN,3.47
4,건설기계,인증완료,CATERPILLAR,09EN*CA*04,2.51,NaN,NaN,0.11,NaN,NaN,3.77
...,...,...,...,...,...,...,...,...,...,...,...
515,건설/농업기계,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
516,농업기계,NaN,NaN,NaN,0.07,0.00,0.28,0.01,NaN,NaN,NaN
517,NaN,NaN,CATERPILLAR,09EN*CA*02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
518,NaN,NaN,Caterpillar,09EN*CA*02,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [214]:
pr29_4.iloc[:, 4:] = pr29_4.iloc[:, 4:].round(2)
pr29_4.head()

,건설기계/농업기계 구분,처분내용,제작사,배출인증번호,CO_FINAL_RST_평균,NMHC_FINAL_RST_평균,NOX_FINAL_RST_평균,PM_FINAL_RST_평균,PN_FINAL_RST_평균,NH3_FINAL_RST_평균,NMHCNOX_FINAL_RST_평균
0,건설기계,인증완료,(주)대동,21EN*DD*02,0.01,0.03,3.39,0.01,34400000000.00,NaN,3.42
1,건설기계,인증완료,(주)코마츠제작소,09EN*KO*01,0.90,NaN,NaN,0.10,NaN,NaN,3.51
2,건설기계,인증완료,B4.5,19EN*CU*05,0.07,0.04,0.12,0.01,142689000000000.00,0.00,NaN
3,건설기계,인증완료,CATERPILLAR,09EN*CA*01,2.38,NaN,NaN,0.12,NaN,NaN,3.47
4,건설기계,인증완료,CATERPILLAR,09EN*CA*04,2.51,NaN,NaN,0.11,NaN,NaN,3.77


In [215]:
pr29_4['테이블생성일자'] = today_date

In [216]:
pr29_4 = pr29_4[[
    '건설기계/농업기계 구분', 
    '처분내용', 
    '제작사', 
    '배출인증번호', 
    'CO_FINAL_RST_평균', 
    'NMHC_FINAL_RST_평균', 
    'NOX_FINAL_RST_평균', 
    'PM_FINAL_RST_평균', 
    'PN_FINAL_RST_평균', 
    'NH3_FINAL_RST_평균', 
    'NMHCNOX_FINAL_RST_평균', 
    '테이블생성일자', 
    ]]
# chd29_4 = {
#     '건설기계/농업기계 구분':'', 
#     '처분내용':'', 
#     '인증연도':'', 
#     '배출인증번호':'', 
#     'CO_FINAL_RST_평균':'', 
#     'NMHC_FINAL_RST_평균':'', 
#     'NOX_FINAL_RST_평균':'', 
#     'PM_FINAL_RST_평균':'', 
#     'PN_FINAL_RST_평균':'', 
#     'NH3_FINAL_RST_평균':'', 
#     'NMHCNOX_FINAL_RST_평균':'', 
#     '테이블생성일자':'', 
# }
# pr29_4 = pr29_4.rename(columns=chd29_4)
pr29_4.head()

,건설기계/농업기계 구분,처분내용,제작사,배출인증번호,CO_FINAL_RST_평균,NMHC_FINAL_RST_평균,NOX_FINAL_RST_평균,PM_FINAL_RST_평균,PN_FINAL_RST_평균,NH3_FINAL_RST_평균,NMHCNOX_FINAL_RST_평균,테이블생성일자
0,건설기계,인증완료,(주)대동,21EN*DD*02,0.01,0.03,3.39,0.01,34400000000.00,NaN,3.42,2023.12.14
1,건설기계,인증완료,(주)코마츠제작소,09EN*KO*01,0.90,NaN,NaN,0.10,NaN,NaN,3.51,2023.12.14
2,건설기계,인증완료,B4.5,19EN*CU*05,0.07,0.04,0.12,0.01,142689000000000.00,0.00,NaN,2023.12.14
3,건설기계,인증완료,CATERPILLAR,09EN*CA*01,2.38,NaN,NaN,0.12,NaN,NaN,3.47,2023.12.14
4,건설기계,인증완료,CATERPILLAR,09EN*CA*04,2.51,NaN,NaN,0.11,NaN,NaN,3.77,2023.12.14


### [출력] STD_BD29_EXHST_GAS_PRMT_CRTR_MEVLU

In [217]:
STD_BD29_EXHST_GAS_PRMT_CRTR_MEVLU = pr29_4.copy()
STD_BD29_EXHST_GAS_PRMT_CRTR_MEVLU.to_csv(os.path.join(df2_fold, f'STD_BD29_EXHST_GAS_PRMT_CRTR_MEVLU_{today_date}.csv'), index=False)
STD_BD29_EXHST_GAS_PRMT_CRTR_MEVLU.to_excel(os.path.join(df2_fold, f'STD_BD29_EXHST_GAS_PRMT_CRTR_MEVLU_{today_date}.xlsx'), index=False)

## 2-10 STD_BD210_CERT_REGSTR_CERT_NO
- 인증대장 인증번호 통계

In [1326]:
cert_ld34.shape, len(cert_ld34['인증번호'].unique())

((5971, 66), 368)

In [1327]:
cert_ld5.shape, len(cert_ld5['인증번호'].unique())

((2055, 76), 131)

In [1328]:
srt210_1 = cert_ld34[(cert_ld34['구분'] == '대표(최초)') | (cert_ld34['구분'] == '대표(정정)')].drop_duplicates('인증번호').reset_index(drop=True)

In [1329]:
srt210_1.columns

Index(['배출기준', '상호명', '인증번호', '최초인증일자', '재교부일자(변경일자)', '접수번호', '접수일자', '종류',
       '구분', '상세내역', '엔진형식(엔진모델명)', '엔진코드', '엔진배기량', '최대출력_KW', '최대출력_RPM',
       '엔진제작사', '엔진제작국', '건설기계수입사(대표자)', '건설기계제작사', '건설기계명칭', '건설/농업기계_형식',
       '비고', '정정내역', '기준치_CO', '기준치_NMHC+Nox', '기준치_NMHC', '기준치_Nox', '기준치_PM',
       '기준치_CO2', '시험결과_ISO_8178_C1_CO', '시험결과_ISO_8178_C1_HC+Nox',
       '시험결과_ISO_8178_C1_NMHC', '시험결과_ISO_8178_C1_Nox', '시험결과_ISO_8178_C1_PM',
       '시험결과_ISO_8178_C1_CO2', '시험결과_ISO_8178_C1_검사일시',
       '시험결과_ISO_8178_C1_최대출력_kw', '시험결과_ISO_8178_C1_rpm', '시험결과_NRSC_CO',
       '시험결과_NRSC_HC+Nox', '시험결과_NRSC_NMHC', '시험결과_NRSC_Nox', '시험결과_NRSC_PM',
       '시험결과_NRSC_CO2', '시험결과_NRSC_검사일시', '시험결과_NRSC_최대출력_kw',
       '시험결과_NRSC_최대출력_rpm', '시험결과_NRTC_CO', '시험결과_NRTC_HC+Nox',
       '시험결과_NRTC_NMHC', '시험결과_NRTC_Nox', '시험결과_NRTC_PM', '시험결과_NRTC_CO2',
       '시험결과_NRTC_검사일시', '시험결과_NRTC_최대출력_kw', '시험결과_NRTC_최대출력_rpm', '자체시험여부',
       '적용기술_EGR', '적용기술_SCR', '적용기술_DPF', '적용기술_ECU',

In [1330]:
srt210_1['최대출력_KW'].max()

382.0

In [1331]:
srt210_1 = cert_ld34[(cert_ld34['구분'] == '대표(최초)') | (cert_ld34['구분'] == '대표(정정)')].drop_duplicates('인증번호').reset_index(drop=True)
srt210_2 = cert_ld5[(cert_ld5['구분'] == '대표(최초)') | (cert_ld5['구분'] == '대표(정정)')].drop_duplicates('인증번호').reset_index(drop=True)
srt210_1 = srt210_1.sort_values(['구분', '최대출력_KW']).drop_duplicates(['인증번호']).reset_index(drop=True)
srt210_2 = srt210_2.sort_values(['구분', '최대출력_KW']).drop_duplicates(['인증번호']).reset_index(drop=True)
srt210_1.shape, srt210_2.shape

((288, 66), (129, 76))

In [1332]:
srt210_1.shape[0] + srt210_2.shape[0]

417

In [817]:
grp210_1 = srt210_1.groupby(['구분', '배출기준', '종류'], dropna=False)['인증번호'].count().reset_index()
grp210_1 = grp210_1.rename(columns={'인증번호':'샘플개수'})
grp210_1.head()

,구분,배출기준,종류,샘플개수
0,대표(정정),tier-3,건설기계,13
1,대표(정정),tier-3,농업기계,2
2,대표(정정),tier-4,건설기계,4
3,대표(정정),tier-4,농업기계,4
4,대표(최초),tier-3,건설기계,87


In [818]:
grp210_1['샘플개수'].sum()

288

In [819]:
grp210_2 = srt210_2.groupby(['구분', '배출기준', '종류'], dropna=False)['인증번호'].count().reset_index()
grp210_2 = grp210_2.rename(columns={'인증번호':'샘플개수'})
grp210_2.head()

,구분,배출기준,종류,샘플개수
0,대표(최초),-,건설기계,7
1,대표(최초),Stage-5,건설기계,88
2,대표(최초),Stage-5,농업기계,34


In [820]:
grp210_2['샘플개수'].sum()

129

In [821]:
tgrp210_1 = pd.concat([grp210_1, grp210_2], ignore_index=True)
tgrp210_1.loc[tgrp210_1.shape[0], ['구분', '샘플개수']] = ['계', tgrp210_1['샘플개수'].sum()]
tgrp210_1.tail()

,구분,배출기준,종류,샘플개수
7,대표(최초),tier-4,농업기계,33.00
8,대표(최초),-,건설기계,7.00
9,대표(최초),Stage-5,건설기계,88.00
10,대표(최초),Stage-5,농업기계,34.00
11,계,NaN,NaN,417.00


In [822]:
today_date = datetime.today().strftime('%Y%m%d')
today_date

'20240108'

In [823]:
tgrp210_1['테이블생성일자'] = today_date

In [824]:
tgrp210_1 = tgrp210_1[[
    '구분', 
    '배출기준', 
    '종류', 
    '샘플개수', 
    '테이블생성일자', 
    ]]
# chd210 = {
#     '구분':'', 
#     '배출기준':'', 
#     '종류':'', 
#     '샘플개수':'', 
#     '테이블생성일자':'', 
# }
# tgrp210_1 = tgrp210_1.rename(columns=chd210)
tgrp210_1.head()

,구분,배출기준,종류,샘플개수,테이블생성일자
0,대표(정정),tier-3,건설기계,13.00,20240108
1,대표(정정),tier-3,농업기계,2.00,20240108
2,대표(정정),tier-4,건설기계,4.00,20240108
3,대표(정정),tier-4,농업기계,4.00,20240108
4,대표(최초),tier-3,건설기계,87.00,20240108


### [출력] STD_BD210_CERT_REGSTR_CERT_NO

In [825]:
STD_BD210_CERT_REGSTR_CERT_NO = tgrp210_1.copy()
STD_BD210_CERT_REGSTR_CERT_NO.to_csv(os.path.join(df2_fold, f'STD_BD210_CERT_REGSTR_CERT_NO_{today_date}.csv'), index=False)
STD_BD210_CERT_REGSTR_CERT_NO.to_excel(os.path.join(df2_fold, f'STD_BD210_CERT_REGSTR_CERT_NO_{today_date}.xlsx'), index=False)

## 2-11 STD_BD211_DSPLVL_CERT_REGSTR_CERT_NO
- 엔진배기량별 인증대장 인증번호 통계

In [1156]:
srt210_1.shape, srt210_2.shape

((288, 66), (129, 77))

In [1160]:
grp211_1 = srt210_1.groupby(['배출기준', '구분', '종류', '엔진배기량(cc)구간'], dropna=False)['인증번호'].count().reset_index()
grp211_1 = grp211_1.rename(columns={'인증번호':'인증번호개수'})
grp211_1.head()

,배출기준,구분,종류,엔진배기량(cc)구간,인증번호개수
0,tier-3,대표(정정),건설기계,"0이상 ~ 2,000미만",4
1,tier-3,대표(정정),건설기계,"10,000이상 ~ 12,000미만",1
2,tier-3,대표(정정),건설기계,"12,000이상 ~ 14,000미만",1
3,tier-3,대표(정정),건설기계,"14,000이상 ~ 16,000미만",1
4,tier-3,대표(정정),건설기계,"2,000이상 ~ 4,000미만",5


In [1161]:
grp211_1['인증번호개수'].sum()

288

In [1162]:
grp211_2 = srt210_2.groupby(['배출기준', '구분', '종류', '엔진배기량(cc)구간'], dropna=False)['인증번호'].count().reset_index()
grp211_2 = grp211_2.rename(columns={'인증번호':'인증번호개수'})
grp211_2.head()

,배출기준,구분,종류,엔진배기량(cc)구간,인증번호개수
0,-,대표(최초),건설기계,"10,000이상 ~ 12,000미만",1
1,-,대표(최초),건설기계,"2,000이상 ~ 4,000미만",2
2,-,대표(최초),건설기계,"4,000이상 ~ 6,000미만",2
3,-,대표(최초),건설기계,"6,000이상 ~ 8,000미만",1
4,-,대표(최초),건설기계,"8,000이상 ~ 10,000미만",1


In [1163]:
grp211_2['인증번호개수'].sum()

129

In [1164]:
tgrp211_1 = pd.concat([grp211_1, grp211_2], ignore_index=True)
tgrp211_1.loc[tgrp211_1.shape[0], ['배출기준', '인증번호개수']] = ['계', tgrp211_1['인증번호개수'].sum()]
tgrp211_1.tail()

,배출기준,구분,종류,엔진배기량(cc)구간,인증번호개수
58,Stage-5,대표(최초),농업기계,"0이상 ~ 2,000미만",10.00
59,Stage-5,대표(최초),농업기계,"2,000이상 ~ 4,000미만",19.00
60,Stage-5,대표(최초),농업기계,"4,000이상 ~ 6,000미만",2.00
61,Stage-5,대표(최초),농업기계,"6,000이상 ~ 8,000미만",3.00
62,계,NaN,NaN,NaN,417.00


In [1165]:
# tgrp211_1['인증번호개수'].sum()

In [1166]:
tgrp211_1['테이블생성일자'] = today_date

In [1167]:
tgrp211_1 = tgrp211_1[[
    '배출기준',
    '구분', 
    '종류', 
    '엔진배기량(cc)구간', 
    '인증번호개수', 
    '테이블생성일자', 
    ]]
# chd211 = {
#     '배출기준':'', 
#     '구분':'', 
#     '종류':'',
#     '엔진배기량(cc)구간':'',  
#     '인증번호개수':'', 
#     '테이블생성일자':'', 
# }
# tgrp211_1 = tgrp211_1.rename(columns=chd211)
tgrp211_1.head()

,배출기준,구분,종류,엔진배기량(cc)구간,인증번호개수,테이블생성일자
0,tier-3,대표(정정),건설기계,"0이상 ~ 2,000미만",4.00,2024.01.08
1,tier-3,대표(정정),건설기계,"10,000이상 ~ 12,000미만",1.00,2024.01.08
2,tier-3,대표(정정),건설기계,"12,000이상 ~ 14,000미만",1.00,2024.01.08
3,tier-3,대표(정정),건설기계,"14,000이상 ~ 16,000미만",1.00,2024.01.08
4,tier-3,대표(정정),건설기계,"2,000이상 ~ 4,000미만",5.00,2024.01.08


### [출력] STD_BD211_DSPLVL_CERT_REGSTR_CERT_NO

In [1168]:
STD_BD211_DSPLVL_CERT_REGSTR_CERT_NO = tgrp211_1.copy()
STD_BD211_DSPLVL_CERT_REGSTR_CERT_NO.to_csv(os.path.join(df2_fold, f'STD_BD211_DSPLVL_CERT_REGSTR_CERT_NO_{today_date}.csv'), index=False)
STD_BD211_DSPLVL_CERT_REGSTR_CERT_NO.to_excel(os.path.join(df2_fold, f'STD_BD211_DSPLVL_CERT_REGSTR_CERT_NO_{today_date}.xlsx'), index=False)

## 2-12 STD_BD212_OTPT_CERT_REGSTR_CERT_NO
- 출력별 인증대장 인증번호 통계

In [1333]:
srt210_1.shape, srt210_2.shape

((288, 66), (129, 76))

In [1336]:
srt210_1['최대출력_KW'].isnull().sum(), srt210_2['최대출력_KW'].isnull().sum()

(270, 1)

In [1169]:
grp212_1 = srt210_1.groupby(['배출기준', '구분', '종류', '최대출력(KW)구간'], dropna=False)['인증번호'].count().reset_index()
grp212_1 = grp212_1.rename(columns={'인증번호':'인증번호개수'})
grp212_1.head()

,배출기준,구분,종류,최대출력(KW)구간,인증번호개수
0,tier-3,대표(정정),건설기계,300이상 ~ 400미만,1
1,tier-3,대표(정정),건설기계,NaN,12
2,tier-3,대표(정정),농업기계,NaN,2
3,tier-3,대표(최초),건설기계,0이상 ~ 100미만,3
4,tier-3,대표(최초),건설기계,100이상 ~ 200미만,2


In [1170]:
grp212_1['인증번호개수'].sum()

288

In [1171]:
grp212_2 = srt210_2.groupby(['배출기준', '구분', '종류', '최대출력(KW)구간'], dropna=False)['인증번호'].count().reset_index()
grp212_2 = grp212_2.rename(columns={'인증번호':'인증번호개수'})
grp212_2.head()

,배출기준,구분,종류,최대출력(KW)구간,인증번호개수
0,-,대표(최초),건설기계,0이상 ~ 100미만,1
1,-,대표(최초),건설기계,100이상 ~ 200미만,3
2,-,대표(최초),건설기계,300이상 ~ 400미만,1
3,-,대표(최초),건설기계,400이상 ~ 500미만,1
4,-,대표(최초),건설기계,NaN,1


In [1172]:
grp212_2['인증번호개수'].sum()

129

In [1173]:
tgrp212_1 = pd.concat([grp212_1, grp212_2], ignore_index=True)
tgrp212_1.loc[tgrp212_1.shape[0], ['배출기준', '인증번호개수']] = ['계', tgrp212_1['인증번호개수'].sum()]
tgrp212_1.tail()

,배출기준,구분,종류,최대출력(KW)구간,인증번호개수
27,Stage-5,대표(최초),건설기계,500이상 ~ 600미만,2.00
28,Stage-5,대표(최초),농업기계,0이상 ~ 100미만,27.00
29,Stage-5,대표(최초),농업기계,100이상 ~ 200미만,6.00
30,Stage-5,대표(최초),농업기계,200이상 ~ 300미만,1.00
31,계,NaN,NaN,NaN,417.00


In [1174]:
tgrp212_1['테이블생성일자'] = today_date

In [1175]:
tgrp212_1 = tgrp212_1[[
    '배출기준', 
    '구분', 
    '종류', 
    '최대출력(KW)구간', 
    '인증번호개수', 
    '테이블생성일자', 
    ]]
# chd212 = {
#     '배출기준':'', 
#     '구분':'', 
#     '종류':'',
#     '최대출력(KW)구간':'',  
#     '인증번호개수':'', 
#     '테이블생성일자':'', 
# }
# tgrp212_1 = tgrp212_1.rename(columns=chd212)
tgrp212_1.head()

,배출기준,구분,종류,최대출력(KW)구간,인증번호개수,테이블생성일자
0,tier-3,대표(정정),건설기계,300이상 ~ 400미만,1.00,2024.01.08
1,tier-3,대표(정정),건설기계,NaN,12.00,2024.01.08
2,tier-3,대표(정정),농업기계,NaN,2.00,2024.01.08
3,tier-3,대표(최초),건설기계,0이상 ~ 100미만,3.00,2024.01.08
4,tier-3,대표(최초),건설기계,100이상 ~ 200미만,2.00,2024.01.08


### [출력] STD_BD212_OTPT_CERT_REGSTR_CERT_NO

In [1176]:
STD_BD212_OTPT_CERT_REGSTR_CERT_NO = tgrp212_1.copy()
STD_BD212_OTPT_CERT_REGSTR_CERT_NO.to_csv(os.path.join(df2_fold, f'STD_BD212_OTPT_CERT_REGSTR_CERT_NO_{today_date}.csv'), index=False)
STD_BD212_OTPT_CERT_REGSTR_CERT_NO.to_excel(os.path.join(df2_fold, f'STD_BD212_OTPT_CERT_REGSTR_CERT_NO_{today_date}.xlsx'), index=False)

## 2-13 STD_BD213_MNFCTR_CERT_REGSTR_CERT_NO
- 제작사별 인증대장 인증번호 통계

In [846]:
srt210_1.shape, srt210_2.shape

((288, 66), (129, 77))

In [864]:
grp213_1 = srt210_1.groupby(['배출기준', '종류', '엔진제작사'], dropna=False)['인증번호'].count().reset_index()
grp213_1 = grp213_1.rename(columns={'인증번호':'인증번호개수'})
grp213_1.head()

,배출기준,종류,엔진제작사,인증번호개수
0,tier-3,건설기계,CATERPILLAR,4
1,tier-3,건설기계,Mitsubishi Fuso Truck And Bus Corporation,1
2,tier-3,건설기계,Perkins,3
3,tier-3,건설기계,구보다(KUBOTA),18
4,tier-3,건설기계,대동공업㈜,2


In [865]:
grp213_1['인증번호개수'].sum()

288

In [866]:
grp213_2 = srt210_2.groupby(['배출기준', '종류', '엔진제작사'], dropna=False)['인증번호'].count().reset_index()
grp213_2 = grp213_2.rename(columns={'인증번호':'인증번호개수'})
grp213_2.head()

,배출기준,종류,엔진제작사,인증번호개수
0,-,건설기계,커민스,7
1,Stage-5,건설기계,(주)대동,2
2,Stage-5,건설기계,FPT Industrial S.p.A,6
3,Stage-5,건설기계,구보다,14
4,Stage-5,건설기계,도이츠,7


In [867]:
grp213_2['인증번호개수'].sum()

129

In [868]:
tgrp213_1 = pd.concat([grp213_1, grp213_2], ignore_index=True)
tgrp213_1.loc[tgrp213_1.shape[0], ['배출기준', '인증번호개수']] = ['계', tgrp213_1['인증번호개수'].sum()]
tgrp213_1.tail()

,배출기준,종류,엔진제작사,인증번호개수
93,Stage-5,농업기계,도이츠,2.00
94,Stage-5,농업기계,엘에스엠트론,3.00
95,Stage-5,농업기계,콜러/롬바르디니,1.00
96,Stage-5,농업기계,티와이엠,7.00
97,계,NaN,NaN,417.00


In [869]:
tgrp213_1['테이블생성일자'] = today_date

In [870]:
tgrp213_1 = tgrp213_1[[
    '배출기준', 
    # '구분', 
    '종류', 
    '엔진제작사', 
    '인증번호개수', 
    '테이블생성일자', 
    ]]
# chd213 = {
#     '배출기준':'', 
#     '구분':'', 
#     '종류':'',
#     '엔진제작사':'',  
#     '인증번호개수':'', 
#     '테이블생성일자':'', 
# }
# tgrp213_1 = tgrp213_1.rename(columns=chd213)
tgrp213_1.head()

,배출기준,종류,엔진제작사,인증번호개수,테이블생성일자
0,tier-3,건설기계,CATERPILLAR,4.00,20240108
1,tier-3,건설기계,Mitsubishi Fuso Truck And Bus Corporation,1.00,20240108
2,tier-3,건설기계,Perkins,3.00,20240108
3,tier-3,건설기계,구보다(KUBOTA),18.00,20240108
4,tier-3,건설기계,대동공업㈜,2.00,20240108


### [출력] STD_BD213_MNFCTR_CERT_REGSTR_CERT_NO

In [871]:
STD_BD213_MNFCTR_CERT_REGSTR_CERT_NO = tgrp213_1.copy()
STD_BD213_MNFCTR_CERT_REGSTR_CERT_NO.to_csv(os.path.join(df2_fold, f'STD_BD213_MNFCTR_CERT_REGSTR_CERT_NO_{today_date}.csv'), index=False)
STD_BD213_MNFCTR_CERT_REGSTR_CERT_NO.to_excel(os.path.join(df2_fold, f'STD_BD213_MNFCTR_CERT_REGSTR_CERT_NO_{today_date}.xlsx'), index=False)

## 2-14 STD_BD214_FBCTN_NTN_CERT_REGSTR_CERT_NO
- 제작국별 인증대장 인증번호 통계

In [872]:
srt210_1.shape, srt210_2.shape

((288, 66), (129, 77))

In [873]:
grp214_1 = srt210_1.groupby(['배출기준', '종류', '엔진제작국'], dropna=False)['인증번호'].count().reset_index()
grp214_1 = grp214_1.rename(columns={'인증번호':'인증번호개수'})
grp214_1.head()

,배출기준,종류,엔진제작국,인증번호개수
0,tier-3,건설기계,독일,8
1,tier-3,건설기계,미국,9
2,tier-3,건설기계,스웨덴,16
3,tier-3,건설기계,영국,5
4,tier-3,건설기계,이탈리아,3


In [874]:
grp214_1['인증번호개수'].sum()

288

In [875]:
grp214_2 = srt210_2.groupby(['배출기준', '종류', '엔진제작국'], dropna=False)['인증번호'].count().reset_index()
grp214_2 = grp214_2.rename(columns={'인증번호':'인증번호개수'})
grp214_2.head()

,배출기준,종류,엔진제작국,인증번호개수
0,-,건설기계,미국,3
1,-,건설기계,영국,2
2,-,건설기계,중국,2
3,Stage-5,건설기계,독일,7
4,Stage-5,건설기계,미국,6


In [876]:
grp214_2['인증번호개수'].sum()

129

In [877]:
tgrp214_1 = pd.concat([grp214_1, grp214_2], ignore_index=True)
tgrp214_1.loc[tgrp214_1.shape[0], ['배출기준', '인증번호개수']] = ['계', tgrp214_1['인증번호개수'].sum()]
tgrp214_1.tail()

,배출기준,종류,엔진제작국,인증번호개수
47,Stage-5,농업기계,이태리,5.00
48,Stage-5,농업기계,일본,9.00
49,Stage-5,농업기계,핀란드,2.00
50,Stage-5,농업기계,한국,16.00
51,계,NaN,NaN,417.00


In [878]:
tgrp214_1['테이블생성일자'] = today_date

In [879]:
tgrp214_1 = tgrp214_1[[
    '배출기준', 
    # '구분', 
    '종류', 
    '엔진제작국', 
    '인증번호개수', 
    '테이블생성일자', 
    ]]
# chd214 = {
#     '배출기준':'', 
#     '구분':'', 
#     '종류':'',
#     '엔진제작국':'',  
#     '인증번호개수':'', 
#     '테이블생성일자':'', 
# }
# tgrp214_1 = tgrp214_1.rename(columns=chd214)
tgrp214_1.head()

,배출기준,종류,엔진제작국,인증번호개수,테이블생성일자
0,tier-3,건설기계,독일,8.00,20240108
1,tier-3,건설기계,미국,9.00,20240108
2,tier-3,건설기계,스웨덴,16.00,20240108
3,tier-3,건설기계,영국,5.00,20240108
4,tier-3,건설기계,이탈리아,3.00,20240108


### [출력] STD_BD214_FBCTN_NTN_CERT_REGSTR_CERT_NO

In [880]:
STD_BD214_FBCTN_NTN_CERT_REGSTR_CERT_NO = tgrp214_1.copy()
STD_BD214_FBCTN_NTN_CERT_REGSTR_CERT_NO.to_csv(os.path.join(df2_fold, f'STD_BD214_FBCTN_NTN_CERT_REGSTR_CERT_NO_{today_date}.csv'), index=False)
STD_BD214_FBCTN_NTN_CERT_REGSTR_CERT_NO.to_excel(os.path.join(df2_fold, f'STD_BD214_FBCTN_NTN_CERT_REGSTR_CERT_NO_{today_date}.xlsx'), index=False)

# 과제2 코드 end